In [12]:
# Grok Attempt

import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
from collections import defaultdict

# Adjustable scaling factor for fund family distribution
FUND_FAMILY_SCALE_FACTOR = 5  # Adjust this value to change the influence of fund family data

# Database connection
connection_string = (
    "mssql+pyodbc://JULIANS_LAPTOP\\SQLEXPRESS/"
    "CWA_Fund_Database?driver=ODBC+Driver+18+for+SQL+Server"
    "&trusted_connection=yes&TrustServerCertificate=yes"
)
engine = create_engine(connection_string)

# Define keywords for each category
keywords = {
    "Index Based": ["index fund", "tracks", "replicates", "indexed", "underlying index", "thematic", "passive",
                    "economic characteristics that are substantially", "bond index", "market-cap weighted",
                    "low tracking error", "high correlation", "benchmark", "low-cost", "broad market exposure",
                    "aggregate bond", "mirrors", "equal-weighted", "beta", "value index", "growth index"],
    "Factor/Smart Beta": ["rules-based", "factor-based", "factor tilt", "multi-factor", "factor investing", "momentum",
                          "low volatility", "low vol", "value factor", "quality", "quality factor", "free cash flow",
                          "fcf", "objective", "relatively", "go up", "certain fundamental metrics", "momentum index",
                          "quality index", "relatively lower valuations", "factors", "minimum volatility",
                          "high dividend yield", "enhanced index", "revenue weighted", "dividend weighted",
                          "enhanced returns", "fundamental weighting", "yield weighted", "low volatility", "rotation",
                          "rules based methodology", "cash cows", "alphaDEX", "ranked", "lower volatility",
                          "tilt", "optimized", "component securities", "economic characteristics", "free cash flow yield",
                          "high dividend yields", "ranking system", "consistently increased dividends", "dividend",
                          "dividends", "strong cash", "low debt", "increasing earnings", "earnings", "rising dividend",
                          "achievers", "volatility weighted", "long/cash", "low beta", "low size"],
    "Active Discretionary": ["actively managed", "actively-managed", "manager believes", "manager's judgment",
                             "active bottom‑up", "active strategy", "discretionary", "active management",
                             "active-management", "machine learning", "ai", "research-driven", "fundamental",
                             "strategically", "tactical allocation", "active", "rotation", "judgment", "analysis",
                             "outperform", "selection", "tactical", "trend-following", "trend following", "Bottom-Up Approach"
                            "the advisor", "advisor considers", "long-term", "appraisal"],
    "Quant/Systematic": ["quantitative", "algorithm-driven", "systematic", "levered", "algorithm", "implied volatility",
                        "data-driven", "back-tested", "long-short", "model-based", "rotation", "statistical",
                        "rules-driven", "trend-following", "trend following", "tactical", "machine learning", "ai",
                        "long/short"],
    "Multi-Strategy": ["multi-strategy", "multi-asset", "hybrid strategy", "multi-manager", "dynamic allocation",
                      "absolute return", "blended", "combination", "hybrid", "flexible", "alternative"]
}

# Direct mapping keywords (added to existing keywords)
direct_mappings = {
    "Active Discretionary": ["Active Management", "Actively Managed", "Discretionary", "Active Strategy", "Active",
                             "Active Bottom-Up", "Active-Management", "Actively-Managed", "Actively",
                             "Actively Allocates", "Active Allocation", "Active Trading", "Trading Actively",
                             "Actively Trading"],
    "Factor/Smart Beta": ["Rules-Based", "Factor-Based", "Multi-Factor"],
    "Quant/Systematic": []
}

# Update keywords with direct mappings
for category, terms in direct_mappings.items():
    keywords[category].extend(terms)

# Convert keywords to lowercase for matching
keywords = {cat: [term.lower() for term in terms] for cat, terms in keywords.items()}

# Define meaningful categories for direct classification
meaningful_categories = {
    "Index Based": {
        "YC_Category": ["Target Maturity", "Digital Assets", "Single Currency", "Muni Target Maturity"],
        "CWA_Broad_Category": ["Currency", "Digital Asset", "Single Stock"],
        "YC_Global_Category": ["Currency"]
    },
    "Quant/Systematic": {
        "CWA_Broad_Category": ["Defined Outcome", "Trading/Tactical"],
        "YC_Global_Category": ["Defined Outcome", "Trading Tools", "Systematic Trend"],
        "YC_Category": ["Trading--Inverse Commodities", "Trading--Inverse Debt", "Trading--Inverse Equity",
                        "Trading--Leveraged Commodities", "Trading--Leveraged Debt", "Trading--Leveraged Equity",
                        "Trading—Miscellaneous"]
    }
}

# Simplified assist categories using pattern matching for "Target-Date" and "Muni"
assist_categories = [
    # Pattern-based rules
    {
        "pattern": "Target-Date",
        "cat_type": "YC_Category",
        "actions": {"remove": ["Factor/Smart Beta", "Quant/Systematic"]}
    },
    {
        "pattern": "Target Date",
        "cat_type": "CWA_Broad_Category",
        "actions": {"remove": ["Factor/Smart Beta", "Quant/Systematic"]}
    },
    {
        "pattern": "Target Date",
        "cat_type": "YC_Global_Category",
        "actions": {"remove": ["Factor/Smart Beta", "Quant/Systematic"]}
    },
    {
        "pattern": "Muni",
        "cat_type": "YC_Category",
        "actions": {"remove": ["Multi-Strategy", "Quant/Systematic"]}
    },
    {
        "pattern": "US Municipal Fixed Income",
        "cat_type": "YC_Global_Category",
        "actions": {"remove": ["Multi-Strategy", "Quant/Systematic"]}
    },
    # Exact match rules (non-patterned categories)
    {
        "exact": "Commodity",
        "cat_type": "CWA_Broad_Category",
        "actions": {"remove": ["Factor/Smart Beta"]}
    },
    {
        "exact": "Commodities Broad Basket",
        "cat_type": "YC_Category",
        "actions": {"remove": ["Factor/Smart Beta"]}
    },
    {
        "exact": "Commodities Focused",
        "cat_type": "YC_Category",
        "actions": {"remove": ["Factor/Smart Beta"]}
    },
    {
        "exact": "Commodities Broad Basket",
        "cat_type": "YC_Global_Category",
        "actions": {"remove": ["Factor/Smart Beta"]}
    },
    {
        "exact": "Commodities Specified",
        "cat_type": "YC_Global_Category",
        "actions": {"remove": ["Factor/Smart Beta"]}
    },
    {
        "exact": "Long/Short Equity",
        "cat_type": "YC_Global_Category",
        "actions": {"boost": ["Active Discretionary", "Quant/Systematic"],
                    "remove": ["Multi-Strategy", "Factor/Smart Beta"]}
    },
    {
        "exact": "inflation-protected bond",
        "cat_type": "YC_Category",
        "actions": {"boost": ["Active Discretionary", "Index Based"],
                    "remove": ["Multi-Strategy", "Factor/Smart Beta", "Quant/Systematic"]}
    },
    {
        "exact": "Nontraditional",
        "cat_type": "CWA_Broad_Category",
        "actions": {"boost": ["Active Discretionary", "Factor/Smart Beta", "Quant/Systematic"],
                    "remove": ["Index Based"]}
    },
    {
        "exact": "Nontraditional Equity",
        "cat_type": "YC_Broad_Asset_Class",
        "actions": {"boost": ["Active Discretionary", "Factor/Smart Beta", "Quant/Systematic"]}
    },
    {
        "exact": "Sector/Industry",
        "cat_type": "CWA_Broad_Category",
        "actions": {"boost": ["Active Discretionary", "Index Based", "Factor/Smart Beta"]}
    }
]

# Load data from database (select only ID columns)
query_funds = """
SELECT SymbolCUSIP, ProductName, fund_family, investment_strategy, FS_insight, index_fund,
       inverse_fund, leveraged_fund, socially_responsible_fund, synthetic_replication_fund,
       fund_of_funds, ycharts_url, YC_Category_ID, CWA_Broad_Category_ID,
       YC_Global_Category_ID, YC_Broad_Asset_Class_ID,
       currency_hedged_fund
FROM Funds_to_Screen
"""
funds_df = pd.read_sql(query_funds, engine)

# Load category mappings with exact column names from your prompt
category_mappings = {
    "CWA_Broad_Category": pd.read_sql("SELECT ID, CWA_Broad_Category_Name FROM CWA_Broad_Category_List", engine),
    "YC_Category": pd.read_sql("SELECT ID, Category_Name FROM YC_Category_List", engine),
    "YC_Global_Category": pd.read_sql("SELECT ID, Global_Category_Name FROM YC_Global_Category_List", engine),
    "YC_Broad_Asset_Class": pd.read_sql("SELECT ID, YC_Broad_Asset_Class_Name FROM YC_Broad_Asset_Class_List", engine)
}

# Merge category names into funds_df using the ID columns, drop 'ID' after each merge
funds_df = funds_df.merge(category_mappings["CWA_Broad_Category"], left_on="CWA_Broad_Category_ID", right_on="ID", how="left").drop(columns=["ID"])
funds_df = funds_df.merge(category_mappings["YC_Category"], left_on="YC_Category_ID", right_on="ID", how="left").drop(columns=["ID"])
funds_df = funds_df.merge(category_mappings["YC_Global_Category"], left_on="YC_Global_Category_ID", right_on="ID", how="left").drop(columns=["ID"])
funds_df = funds_df.merge(category_mappings["YC_Broad_Asset_Class"], left_on="YC_Broad_Asset_Class_ID", right_on="ID", how="left").drop(columns=["ID"])

# Normalize Boolean fields (True/False and 1/0 to 1/0) and add debugging
boolean_cols = ["index_fund", "inverse_fund", "leveraged_fund", "socially_responsible_fund",
               "synthetic_replication_fund", "fund_of_funds", "currency_hedged_fund"]
# Debug raw values before normalization
print("Raw index_fund values before normalization:", funds_df["index_fund"].head().tolist())
# Normalize with broader type handling
for col in boolean_cols:
    funds_df[col] = funds_df[col].apply(lambda x: 1 if str(x).lower() in ['true', '1', 'yes'] else 0)
# Debug after normalization
print("Index_fund values after normalization:", funds_df["index_fund"].head().tolist())
# Specifically check IJH
print("IJH index_fund after normalization:", funds_df[funds_df["SymbolCUSIP"] == "IJH"]["index_fund"].values)

# Initialize dictionaries to track scoring components
categories = ["Index Based", "Factor/Smart Beta", "Active Discretionary", "Quant/Systematic", "Multi-Strategy"]

# Initialize final score columns
for cat in categories:
    score_col = f"score_{cat}"
    funds_df[score_col] = 0.0

# Initialize intermediate score columns for each component
for cat in categories:
    funds_df[f"keyword_score_{cat}"] = 0.0
    funds_df[f"meaningful_score_{cat}"] = 0.0
    funds_df[f"assist_score_{cat}"] = 0.0
    funds_df[f"boolean_score_{cat}"] = 0.0
    funds_df[f"fundfamily_score_{cat}"] = 0.0

# Initialize columns to track matched keywords (concatenate matched keywords as strings)
for cat in categories:
    funds_df[f"matched_keywords_{cat}"] = ""

# Debugging: Print columns to confirm score columns are created
print("Columns after initializing scores:", funds_df.columns.tolist())

# Function to count keywords and return matched keywords
def count_keywords(text, keyword_list):
    if pd.isna(text):
        return 0, ""
    text = text.lower()
    matches = [keyword for keyword in keyword_list if re.search(r'\b' + re.escape(keyword) + r'\b', text)]
    return len(matches), "; ".join(matches)

# Apply scoring based on keywords and track matched keywords
text_columns = ["ProductName", "investment_strategy", "FS_insight"]
for cat, kw_list in keywords.items():
    score_col = f"keyword_score_{cat}"
    matched_col = f"matched_keywords_{cat}"
    for text_col in text_columns:
        counts_and_matches = funds_df[text_col].apply(lambda x: count_keywords(x, kw_list))
        funds_df[score_col] += counts_and_matches.apply(lambda x: x[0])
        funds_df[matched_col] = funds_df[matched_col] + "; " + counts_and_matches.apply(lambda x: x[1])
    # Clean up matched keywords column (remove duplicate semicolons, trim)
    funds_df[matched_col] = funds_df[matched_col].str.replace(r'\s*;\s*;\s*', '; ', regex=True).str.strip('; ')

# Apply meaningful category rules
for cat, mappings in meaningful_categories.items():
    score_col = f"meaningful_score_{cat}"
    for map_type, values in mappings.items():
        col_name = {
            "CWA_Broad_Category": "CWA_Broad_Category_Name",
            "YC_Category": "Category_Name",
            "YC_Global_Category": "Global_Category_Name"
        }[map_type]
        funds_df.loc[funds_df[col_name].isin(values), score_col] += 10  # High score for direct match

# Apply assist category rules with pattern matching
for rule in assist_categories:
    cat_type = rule["cat_type"]
    col_name = {
        "CWA_Broad_Category": "CWA_Broad_Category_Name",
        "YC_Category": "Category_Name",
        "YC_Global_Category": "Global_Category_Name",
        "YC_Broad_Asset_Class": "YC_Broad_Asset_Class_Name"
    }[cat_type]
    actions = rule["actions"]

    if "pattern" in rule:
        pattern = rule["pattern"]
        mask = funds_df[col_name].fillna("").str.contains(pattern, case=False, na=False)
        matches = funds_df[mask][["SymbolCUSIP", col_name]]
        if not matches.empty:
            print(f"Assist category pattern '{pattern}' matches for {cat_type}:")
            print(matches.head())
    else:
        exact_value = rule["exact"]
        mask = funds_df[col_name].fillna("").str.contains(exact_value, case=False, na=False)
        matches = funds_df[mask][["SymbolCUSIP", col_name]]
        if not matches.empty:
            print(f"Assist category exact '{exact_value}' matches for {cat_type}:")
            print(matches.head())

    if "remove" in actions:
        for remove_cat in actions["remove"]:
            score_col = f"assist_score_{remove_cat}"
            funds_df.loc[mask, score_col] = -float('inf')
    if "boost" in actions:
        for boost_cat in actions["boost"]:
            score_col = f"assist_score_{boost_cat}"
            funds_df.loc[mask, score_col] += 5

# Reset any assist scores not explicitly set to 0 (avoid incorrect leftovers like 0.25)
for cat in categories:
    score_col = f"assist_score_{cat}"
    funds_df[score_col] = funds_df[score_col].replace(0.25, 0)  # Explicitly correct erroneous 0.25 values

# Apply Boolean rules and track contributions
# Rule 1: Categorize as Index Based
mask_index = (funds_df["index_fund"] == 1) & (
    (funds_df["inverse_fund"] == 1) |
    (funds_df["leveraged_fund"] == 1) |
    (funds_df["socially_responsible_fund"] == 1) |
    (funds_df["synthetic_replication_fund"] == 1)
)
funds_df.loc[mask_index, "boolean_score_Index Based"] += 20  # High score for direct categorization

# Rule 2: Remove Active Discretionary and Quant/Systematic if index_fund = True
mask_remove = funds_df["index_fund"] == 1
funds_df.loc[mask_remove, "boolean_score_Active Discretionary"] = -float('inf')
funds_df.loc[mask_remove, "boolean_score_Quant/Systematic"] = -float('inf')

# Rule 3: Boost Index Based and Factor/Smart Beta if index_fund = True
funds_df.loc[mask_remove, "boolean_score_Index Based"] += 5
funds_df.loc[mask_remove, "boolean_score_Factor/Smart Beta"] += 5

# Rule 4: Remove Index Based if index_fund = False
funds_df.loc[funds_df["index_fund"] == 0, "boolean_score_Index Based"] = -float('inf')

# Rule 5: Remove Active Discretionary and Quant/Systematic if index_fund and fund_of_funds = True
mask_remove2 = (funds_df["index_fund"] == 1) & (funds_df["fund_of_funds"] == 1)
funds_df.loc[mask_remove2, "boolean_score_Active Discretionary"] = -float('inf')
funds_df.loc[mask_remove2, "boolean_score_Quant/Systematic"] = -float('inf')

# Rule 6: Boost Active Discretionary, Quant/Systematic, Multi-Strategy if fund_of_funds = True
mask_boost = funds_df["fund_of_funds"] == 1
funds_df.loc[mask_boost, "boolean_score_Active Discretionary"] += 5
funds_df.loc[mask_boost, "boolean_score_Quant/Systematic"] += 5
funds_df.loc[mask_boost, "boolean_score_Multi-Strategy"] += 5

# Rule 7: Boost Active Discretionary and Index Based if currency_hedged_fund = True
mask_currency = funds_df["currency_hedged_fund"] == 1
funds_df.loc[mask_currency, "boolean_score_Active Discretionary"] += 5
funds_df.loc[mask_currency, "boolean_score_Index Based"] += 5

# Rule 8: Remove Factor/Smart Beta if currency_hedged_fund = True
funds_df.loc[mask_currency, "boolean_score_Factor/Smart Beta"] = -float('inf')

# Merge FundFamilyData and apply as a weighted factor
fund_family_df = pd.read_sql("SELECT FundFamilyName, Dist_Index, Dist_Active, Dist_Rules_Based, Dist_Quant, Dist_Multi FROM FundFamilyData", engine)
funds_df = funds_df.merge(fund_family_df, left_on="fund_family", right_on="FundFamilyName", how="left")
funds_df["Dist_Index"] = funds_df["Dist_Index"].fillna(0) / 100
funds_df["Dist_Active"] = funds_df["Dist_Active"].fillna(0) / 100
funds_df["Dist_Rules_Based"] = funds_df["Dist_Rules_Based"].fillna(0) / 100
funds_df["Dist_Quant"] = funds_df["Dist_Quant"].fillna(0) / 100
funds_df["Dist_Multi"] = funds_df["Dist_Multi"].fillna(0) / 100

# Apply fund family scoring
funds_df["fundfamily_score_Index Based"] += funds_df["Dist_Index"] * FUND_FAMILY_SCALE_FACTOR
funds_df["fundfamily_score_Active Discretionary"] += funds_df["Dist_Active"] * FUND_FAMILY_SCALE_FACTOR
funds_df["fundfamily_score_Factor/Smart Beta"] += funds_df["Dist_Rules_Based"] * FUND_FAMILY_SCALE_FACTOR
funds_df["fundfamily_score_Quant/Systematic"] += funds_df["Dist_Quant"] * FUND_FAMILY_SCALE_FACTOR
funds_df["fundfamily_score_Multi-Strategy"] += funds_df["Dist_Multi"] * FUND_FAMILY_SCALE_FACTOR

# Sum all intermediate scores into final scores
for cat in categories:
    score_col = f"score_{cat}"
    funds_df[score_col] = (
        funds_df[f"keyword_score_{cat}"] +
        funds_df[f"meaningful_score_{cat}"] +
        funds_df[f"assist_score_{cat}"] +
        funds_df[f"boolean_score_{cat}"] +
        funds_df[f"fundfamily_score_{cat}"]
    )

# Determine final category with tiebreaker based on keyword scores
score_columns = [f"score_{cat}" for cat in categories]
# Initial idxmax
funds_df["Return_Driver"] = funds_df[score_columns].idxmax(axis=1).apply(
    lambda x: x.replace("score_", "") if pd.notnull(x) else "None"
)
# Apply tiebreaker: if scores are tied, prefer category with higher keyword score
for idx in funds_df.index:
    scores = funds_df.loc[idx, score_columns]
    max_score = scores.max()
    tied_categories = [col for col, score in scores.items() if score == max_score]
    if len(tied_categories) > 1:
        # Find category with highest keyword score among tied categories
        keyword_scores = {col: funds_df.loc[idx, f"keyword_score_{col.replace('score_', '')}"] for col in tied_categories}
        max_keyword_score = max(keyword_scores.values())
        best_tied_category = max(keyword_scores, key=keyword_scores.get)
        funds_df.loc[idx, "Return_Driver"] = best_tied_category.replace("score_", "")

# Export to Excel with all details in the specified order
output_columns = (
    # Primary identifiers and results
    ["SymbolCUSIP", "ProductName", "fund_family", "Return_Driver", "ycharts_url"] +
    
    # Final scores for each category
    [f"score_{cat}" for cat in categories] +
    
    # Intermediate scores for each scoring component
    [f"keyword_score_{cat}" for cat in categories] +
    [f"meaningful_score_{cat}" for cat in categories] +
    [f"assist_score_{cat}" for cat in categories] +
    [f"boolean_score_{cat}" for cat in categories] +
    [f"fundfamily_score_{cat}" for cat in categories] +
    
    # Everything else (inputs, matched keywords, etc.)
    [f"matched_keywords_{cat}" for cat in categories] +
    ["CWA_Broad_Category_Name", "Category_Name", "Global_Category_Name", "YC_Broad_Asset_Class_Name"] +
    ["index_fund", "inverse_fund", "leveraged_fund", "socially_responsible_fund", "synthetic_replication_fund", "fund_of_funds", "currency_hedged_fund"] +
    ["Dist_Index", "Dist_Active", "Dist_Rules_Based", "Dist_Quant", "Dist_Multi"]
)

funds_df[output_columns].to_excel(output_path, index=False)
print(f"Results exported to {output_path}")

Raw index_fund values before normalization: ['0', '1', '1', '1', '1']
Index_fund values after normalization: [0, 1, 1, 1, 1]
IJH index_fund after normalization: [1]
Columns after initializing scores: ['SymbolCUSIP', 'ProductName', 'fund_family', 'investment_strategy', 'FS_insight', 'index_fund', 'inverse_fund', 'leveraged_fund', 'socially_responsible_fund', 'synthetic_replication_fund', 'fund_of_funds', 'ycharts_url', 'YC_Category_ID', 'CWA_Broad_Category_ID', 'YC_Global_Category_ID', 'YC_Broad_Asset_Class_ID', 'currency_hedged_fund', 'CWA_Broad_Category_Name', 'Category_Name', 'Global_Category_Name', 'YC_Broad_Asset_Class_Name', 'score_Index Based', 'score_Factor/Smart Beta', 'score_Active Discretionary', 'score_Quant/Systematic', 'score_Multi-Strategy', 'keyword_score_Index Based', 'meaningful_score_Index Based', 'assist_score_Index Based', 'boolean_score_Index Based', 'fundfamily_score_Index Based', 'keyword_score_Factor/Smart Beta', 'meaningful_score_Factor/Smart Beta', 'assist_sco

In [16]:
# Likely final classification of Return Drivers via Grok

import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy.sql import text
import re
from collections import defaultdict

# Adjustable scaling factor for fund family distribution
FUND_FAMILY_SCALE_FACTOR = 5  # Adjust this value to change the influence of fund family data

# Adjustable toggle for Excel output (default off)
write_to_excel = False  # Set to True to generate Excel output, False to skip

# Define output path for Excel (if used)
output_path = r"C:\Users\JulianHeron\Software Projects\Return_Drivers_V1.xlsx"

# Database connection
connection_string = (
    "mssql+pyodbc://JULIANS_LAPTOP\\SQLEXPRESS/"
    "CWA_Fund_Database?driver=ODBC+Driver+18+for+SQL+Server"
    "&trusted_connection=yes&TrustServerCertificate=yes"
)
engine = create_engine(connection_string)

# Define keywords for each category
keywords = {
    "Index Based": ["index fund", "tracks", "replicates", "indexed", "underlying index", "thematic", "passive",
                    "economic characteristics that are substantially", "bond index", "market-cap weighted",
                    "low tracking error", "high correlation", "benchmark", "low-cost", "broad market exposure",
                    "aggregate bond", "mirrors", "equal-weighted", "beta", "value index", "growth index"],
    "Rules Based": ["rules-based", "factor-based", "factor tilt", "multi-factor", "factor investing", "momentum",
                    "low volatility", "low vol", "value factor", "quality", "quality factor", "free cash flow",
                    "fcf", "objective", "relatively", "go up", "certain fundamental metrics", "momentum index",
                    "quality index", "relatively lower valuations", "factors", "minimum volatility",
                    "high dividend yield", "enhanced index", "revenue weighted", "dividend weighted",
                    "enhanced returns", "fundamental weighting", "yield weighted", "low volatility", "rotation",
                    "rules based methodology", "cash cows", "alphaDEX", "ranked", "lower volatility",
                    "tilt", "optimized", "component securities", "economic characteristics", "free cash flow yield",
                    "high dividend yields", "ranking system", "consistently increased dividends", "dividend",
                    "dividends", "strong cash", "low debt", "increasing earnings", "earnings", "rising dividend",
                    "achievers", "volatility weighted", "long/cash", "low beta", "low size"],
    "Active Discretionary": ["actively managed", "actively-managed", "manager believes", "manager's judgment",
                             "active bottom‑up", "active strategy", "discretionary", "active management",
                             "active-management", "machine learning", "ai", "research-driven", "fundamental",
                             "strategically", "tactical allocation", "active", "rotation", "judgment", "analysis",
                             "outperform", "selection", "tactical", "trend-following", "trend following", "Bottom-Up Approach"
                            "the advisor", "advisor considers", "long-term", "appraisal"],
    "Quant Systematic": ["quantitative", "algorithm-driven", "systematic", "levered", "algorithm", "implied volatility",
                        "data-driven", "back-tested", "long-short", "model-based", "rotation", "statistical",
                        "rules-driven", "trend-following", "trend following", "tactical", "machine learning", "ai",
                        "long/short"],
    "Multi Strategy": ["multi-strategy", "multi-asset", "hybrid strategy", "multi-manager", "dynamic allocation",
                      "absolute return", "blended", "combination", "hybrid", "flexible", "alternative"]
}

# Direct mapping keywords (added to existing keywords)
direct_mappings = {
    "Active Discretionary": ["Active Management", "Actively Managed", "Discretionary", "Active Strategy", "Active",
                             "Active Bottom-Up", "Active-Management", "Actively-Managed", "Actively",
                             "Actively Allocates", "Active Allocation", "Active Trading", "Trading Actively",
                             "Actively Trading"],
    "Rules Based": ["Rules-Based", "Factor-Based", "Multi-Factor"],
    "Quant Systematic": []
}

# Update keywords with direct mappings
for category, terms in direct_mappings.items():
    keywords[category].extend(terms)

# Convert keywords to lowercase for matching
keywords = {cat: [term.lower() for term in terms] for cat, terms in keywords.items()}

# Define meaningful categories for direct classification
meaningful_categories = {
    "Index Based": {
        "YC_Category": ["Target Maturity", "Digital Assets", "Single Currency", "Muni Target Maturity"],
        "CWA_Broad_Category": ["Currency", "Digital Asset", "Single Stock"],
        "YC_Global_Category": ["Currency"]
    },
    "Quant Systematic": {
        "CWA_Broad_Category": ["Defined Outcome", "Trading/Tactical"],
        "YC_Global_Category": ["Defined Outcome", "Trading Tools", "Systematic Trend"],
        "YC_Category": ["Trading--Inverse Commodities", "Trading--Inverse Debt", "Trading--Inverse Equity",
                        "Trading--Leveraged Commodities", "Trading--Leveraged Debt", "Trading--Leveraged Equity",
                        "Trading—Miscellaneous"]
    }
}

# Simplified assist categories using pattern matching for "Target-Date" and "Muni"
assist_categories = [
    # Pattern-based rules
    {
        "pattern": "Target-Date",
        "cat_type": "YC_Category",
        "actions": {"remove": ["Rules Based", "Quant Systematic"]}
    },
    {
        "pattern": "Target Date",
        "cat_type": "CWA_Broad_Category",
        "actions": {"remove": ["Rules Based", "Quant Systematic"]}
    },
    {
        "pattern": "Target Date",
        "cat_type": "YC_Global_Category",
        "actions": {"remove": ["Rules Based", "Quant Systematic"]}
    },
    {
        "pattern": "Muni",
        "cat_type": "YC_Category",
        "actions": {"remove": ["Multi Strategy", "Quant Systematic"]}
    },
    {
        "pattern": "US Municipal Fixed Income",
        "cat_type": "YC_Global_Category",
        "actions": {"remove": ["Multi Strategy", "Quant Systematic"]}
    },
    # Exact match rules (non-patterned categories)
    {
        "exact": "Commodity",
        "cat_type": "CWA_Broad_Category",
        "actions": {"remove": ["Rules Based"]}
    },
    {
        "exact": "Commodities Broad Basket",
        "cat_type": "YC_Category",
        "actions": {"remove": ["Rules Based"]}
    },
    {
        "exact": "Commodities Focused",
        "cat_type": "YC_Category",
        "actions": {"remove": ["Rules Based"]}
    },
    {
        "exact": "Commodities Broad Basket",
        "cat_type": "YC_Global_Category",
        "actions": {"remove": ["Rules Based"]}
    },
    {
        "exact": "Commodities Specified",
        "cat_type": "YC_Global_Category",
        "actions": {"remove": ["Rules Based"]}
    },
    {
        "exact": "Long/Short Equity",
        "cat_type": "YC_Global_Category",
        "actions": {"boost": ["Active Discretionary", "Quant Systematic"],
                    "remove": ["Multi Strategy", "Rules Based"]}
    },
    {
        "exact": "inflation-protected bond",
        "cat_type": "YC_Category",
        "actions": {"boost": ["Active Discretionary", "Index Based"],
                    "remove": ["Multi Strategy", "Rules Based", "Quant Systematic"]}
    },
    {
        "exact": "Nontraditional",
        "cat_type": "CWA_Broad_Category",
        "actions": {"boost": ["Active Discretionary", "Rules Based", "Quant Systematic"],
                    "remove": ["Index Based"]}
    },
    {
        "exact": "Nontraditional Equity",
        "cat_type": "YC_Broad_Asset_Class",
        "actions": {"boost": ["Active Discretionary", "Rules Based", "Quant Systematic"]}
    },
    {
        "exact": "Sector/Industry",
        "cat_type": "CWA_Broad_Category",
        "actions": {"boost": ["Active Discretionary", "Index Based", "Rules Based"]}
    }
]

# Load data from database (select only ID columns along with necessary fields)
query_funds = """
SELECT SymbolCUSIP, ProductName, fund_family, investment_strategy, FS_insight, index_fund,
       inverse_fund, leveraged_fund, socially_responsible_fund, synthetic_replication_fund,
       fund_of_funds, ycharts_url, YC_Category_ID, CWA_Broad_Category_ID,
       YC_Global_Category_ID, YC_Broad_Asset_Class_ID,
       currency_hedged_fund
FROM Funds_to_Screen
"""
funds_df = pd.read_sql(query_funds, engine)

# Load category mappings with exact column names from your schema
category_mappings = {
    "CWA_Broad_Category": pd.read_sql("SELECT ID, CWA_Broad_Category_Name FROM CWA_Broad_Category_List", engine),
    "YC_Category": pd.read_sql("SELECT ID, Category_Name FROM YC_Category_List", engine),
    "YC_Global_Category": pd.read_sql("SELECT ID, Global_Category_Name FROM YC_Global_Category_List", engine),
    "YC_Broad_Asset_Class": pd.read_sql("SELECT ID, YC_Broad_Asset_Class_Name FROM YC_Broad_Asset_Class_List", engine)
}

# Merge category names into funds_df using the ID columns, drop 'ID' after each merge
funds_df = funds_df.merge(category_mappings["CWA_Broad_Category"], left_on="CWA_Broad_Category_ID", right_on="ID", how="left").drop(columns=["ID"])
funds_df = funds_df.merge(category_mappings["YC_Category"], left_on="YC_Category_ID", right_on="ID", how="left").drop(columns=["ID"])
funds_df = funds_df.merge(category_mappings["YC_Global_Category"], left_on="YC_Global_Category_ID", right_on="ID", how="left").drop(columns=["ID"])
funds_df = funds_df.merge(category_mappings["YC_Broad_Asset_Class"], left_on="YC_Broad_Asset_Class_ID", right_on="ID", how="left").drop(columns=["ID"])

# Normalize Boolean fields (True/False and 1/0 to 1/0) and add debugging
boolean_cols = ["index_fund", "inverse_fund", "leveraged_fund", "socially_responsible_fund",
               "synthetic_replication_fund", "fund_of_funds", "currency_hedged_fund"]
# Debug raw values before normalization
print("Raw index_fund values before normalization:", funds_df["index_fund"].head().tolist())
# Normalize with broader type handling
for col in boolean_cols:
    funds_df[col] = funds_df[col].apply(lambda x: 1 if str(x).lower() in ['true', '1', 'yes'] else 0)
# Debug after normalization
print("Index_fund values after normalization:", funds_df["index_fund"].head().tolist())
# Specifically check a few funds
print("VLUE index_fund after normalization:", funds_df[funds_df["SymbolCUSIP"] == "VLUE"]["index_fund"].values)

# Define categories with new names
categories = ["Index Based", "Rules Based", "Active Discretionary", "Quant Systematic", "Multi Strategy"]
category_mapping = {
    "Index Based": "index_based",
    "Rules Based": "rules_based",
    "Active Discretionary": "active_discretionary",
    "Quant Systematic": "quant_systematic",
    "Multi Strategy": "multi_strategy"
}

# Initialize dictionaries to track scoring components
for cat in categories:
    db_cat = category_mapping[cat]
    score_col = f"score_{db_cat}"
    funds_df[score_col] = 0.0

# Initialize intermediate score columns for each component
for cat in categories:
    db_cat = category_mapping[cat]
    funds_df[f"keyword_score_{db_cat}"] = 0.0
    funds_df[f"meaningful_score_{db_cat}"] = 0.0
    funds_df[f"assist_score_{db_cat}"] = 0.0
    funds_df[f"boolean_score_{db_cat}"] = 0.0
    funds_df[f"fundfamily_score_{db_cat}"] = 0.0

# Initialize columns to track matched keywords (concatenate matched keywords as strings)
for cat in categories:
    db_cat = category_mapping[cat]
    funds_df[f"matched_keywords_{db_cat}"] = ""

# Debugging: Print columns to confirm score columns are created
print("Columns after initializing scores:", funds_df.columns.tolist())

# Function to count keywords and return matched keywords
def count_keywords(text, keyword_list):
    if pd.isna(text):
        return 0, ""
    text = text.lower()
    matches = [keyword for keyword in keyword_list if re.search(r'\b' + re.escape(keyword) + r'\b', text)]
    return len(matches), "; ".join(matches)

# Apply scoring based on keywords and track matched keywords
text_columns = ["ProductName", "investment_strategy", "FS_insight"]
for cat, kw_list in keywords.items():
    db_cat = category_mapping[cat]
    score_col = f"keyword_score_{db_cat}"
    matched_col = f"matched_keywords_{db_cat}"
    for text_col in text_columns:
        counts_and_matches = funds_df[text_col].apply(lambda x: count_keywords(x, kw_list))
        funds_df[score_col] += counts_and_matches.apply(lambda x: x[0])
        funds_df[matched_col] = funds_df[matched_col] + "; " + counts_and_matches.apply(lambda x: x[1])
    # Clean up matched keywords column (remove duplicate semicolons, trim)
    funds_df[matched_col] = funds_df[matched_col].str.replace(r'\s*;\s*;\s*', '; ', regex=True).str.strip('; ')

# Apply meaningful category rules
for cat, mappings in meaningful_categories.items():
    db_cat = category_mapping[cat]
    score_col = f"meaningful_score_{db_cat}"
    for map_type, values in mappings.items():
        col_name = {
            "CWA_Broad_Category": "CWA_Broad_Category_Name",
            "YC_Category": "Category_Name",
            "YC_Global_Category": "Global_Category_Name"
        }[map_type]
        matches = funds_df[funds_df[col_name].isin(values)][["SymbolCUSIP", col_name]]
        if not matches.empty:
            print(f"Meaningful category matches for {cat}, {map_type}:")
            print(matches.head())
        funds_df.loc[funds_df[col_name].isin(values), score_col] += 10  # High score for direct match

# Apply assist category rules with pattern matching
for rule in assist_categories:
    cat_type = rule["cat_type"]
    col_name = {
        "CWA_Broad_Category": "CWA_Broad_Category_Name",
        "YC_Category": "Category_Name",
        "YC_Global_Category": "Global_Category_Name",
        "YC_Broad_Asset_Class": "YC_Broad_Asset_Class_Name"
    }[cat_type]
    actions = rule["actions"]

    if "pattern" in rule:
        pattern = rule["pattern"]
        mask = funds_df[col_name].fillna("").str.contains(pattern, case=False, na=False)
        matches = funds_df[mask][["SymbolCUSIP", col_name]]
        if not matches.empty:
            print(f"Assist category pattern '{pattern}' matches for {cat_type}:")
            print(matches.head())
    else:
        exact_value = rule["exact"]
        mask = funds_df[col_name].fillna("").str.contains(exact_value, case=False, na=False)
        matches = funds_df[mask][["SymbolCUSIP", col_name]]
        if not matches.empty:
            print(f"Assist category exact '{exact_value}' matches for {cat_type}:")
            print(matches.head())

    if "remove" in actions:
        for remove_cat in actions["remove"]:
            db_remove_cat = category_mapping[remove_cat]
            score_col = f"assist_score_{db_remove_cat}"
            funds_df.loc[mask, score_col] = -float('inf')
    if "boost" in actions:
        for boost_cat in actions["boost"]:
            db_boost_cat = category_mapping[boost_cat]
            score_col = f"assist_score_{db_boost_cat}"
            funds_df.loc[mask, score_col] += 5

# Reset any assist scores not explicitly set to 0 (avoid incorrect leftovers like 0.25)
for cat in categories:
    db_cat = category_mapping[cat]
    score_col = f"assist_score_{db_cat}"
    funds_df[score_col] = funds_df[score_col].replace(0.25, 0)

# Apply Boolean rules and track contributions
mask_index = (funds_df["index_fund"] == 1) & (
    (funds_df["inverse_fund"] == 1) |
    (funds_df["leveraged_fund"] == 1) |
    (funds_df["socially_responsible_fund"] == 1) |
    (funds_df["synthetic_replication_fund"] == 1)
)
funds_df.loc[mask_index, "boolean_score_index_based"] += 20

mask_remove = funds_df["index_fund"] == 1
funds_df.loc[mask_remove, "boolean_score_active_discretionary"] = -float('inf')
funds_df.loc[mask_remove, "boolean_score_quant_systematic"] = -float('inf')

funds_df.loc[mask_remove, "boolean_score_index_based"] += 5
funds_df.loc[mask_remove, "boolean_score_rules_based"] += 5

funds_df.loc[funds_df["index_fund"] == 0, "boolean_score_index_based"] = -float('inf')

mask_remove2 = (funds_df["index_fund"] == 1) & (funds_df["fund_of_funds"] == 1)
funds_df.loc[mask_remove2, "boolean_score_active_discretionary"] = -float('inf')
funds_df.loc[mask_remove2, "boolean_score_quant_systematic"] = -float('inf')

mask_boost = funds_df["fund_of_funds"] == 1
funds_df.loc[mask_boost, "boolean_score_active_discretionary"] += 5
funds_df.loc[mask_boost, "boolean_score_quant_systematic"] += 5
funds_df.loc[mask_boost, "boolean_score_multi_strategy"] += 5

mask_currency = funds_df["currency_hedged_fund"] == 1
funds_df.loc[mask_currency, "boolean_score_active_discretionary"] += 5
funds_df.loc[mask_currency, "boolean_score_index_based"] += 5

funds_df.loc[mask_currency, "boolean_score_rules_based"] = -float('inf')

# Merge FundFamilyData and apply as a weighted factor
fund_family_df = pd.read_sql("SELECT FundFamilyName, Dist_Index, Dist_Active, Dist_Rules_Based, Dist_Quant, Dist_Multi FROM FundFamilyData", engine)
funds_df = funds_df.merge(fund_family_df, left_on="fund_family", right_on="FundFamilyName", how="left")
funds_df["Dist_Index"] = funds_df["Dist_Index"].fillna(0) / 100
funds_df["Dist_Active"] = funds_df["Dist_Active"].fillna(0) / 100
funds_df["Dist_Rules_Based"] = funds_df["Dist_Rules_Based"].fillna(0) / 100
funds_df["Dist_Quant"] = funds_df["Dist_Quant"].fillna(0) / 100
funds_df["Dist_Multi"] = funds_df["Dist_Multi"].fillna(0) / 100

funds_df["fundfamily_score_index_based"] += funds_df["Dist_Index"] * FUND_FAMILY_SCALE_FACTOR
funds_df["fundfamily_score_active_discretionary"] += funds_df["Dist_Active"] * FUND_FAMILY_SCALE_FACTOR
funds_df["fundfamily_score_rules_based"] += funds_df["Dist_Rules_Based"] * FUND_FAMILY_SCALE_FACTOR
funds_df["fundfamily_score_quant_systematic"] += funds_df["Dist_Quant"] * FUND_FAMILY_SCALE_FACTOR
funds_df["fundfamily_score_multi_strategy"] += funds_df["Dist_Multi"] * FUND_FAMILY_SCALE_FACTOR

# Sum all intermediate scores into final scores
for cat in categories:
    db_cat = category_mapping[cat]
    score_col = f"score_{db_cat}"
    funds_df[score_col] = (
        funds_df[f"keyword_score_{db_cat}"] +
        funds_df[f"meaningful_score_{db_cat}"] +
        funds_df[f"assist_score_{db_cat}"] +
        funds_df[f"boolean_score_{db_cat}"] +
        funds_df[f"fundfamily_score_{db_cat}"]
    )

# Determine final category with tiebreaker based on keyword scores
score_columns = [f"score_{category_mapping[cat]}" for cat in categories]
# Initial idxmax
funds_df["Return_Driver"] = funds_df[score_columns].idxmax(axis=1).apply(
    lambda x: category_mapping.get(x.replace("score_", ""), "None") if pd.notnull(x) else "None"
)
# Apply tiebreaker: if scores are tied, prefer category with higher keyword score
for idx in funds_df.index:
    scores = funds_df.loc[idx, score_columns]
    max_score = scores.max()
    tied_categories = [col for col, score in scores.items() if score == max_score]
    if len(tied_categories) > 1:
        # Find category with highest keyword score among tied categories
        keyword_scores = {col: funds_df.loc[idx, f"keyword_score_{col.replace('score_', '')}"] for col in tied_categories}
        max_keyword_score = max(keyword_scores.values())
        best_tied_category = max(keyword_scores, key=keyword_scores.get)
        funds_df.loc[idx, "Return_Driver"] = best_tied_category.replace("score_", "")



# Write scores and Return_Driver to the database
with engine.connect() as conn:
    for cat in categories:
        db_cat = category_mapping[cat]
        score_col = f"score_{db_cat}"
        # Replace -inf, inf, and NaN with None (NULL in SQL) to avoid invalid float errors
        funds_df[score_col] = funds_df[score_col].replace([np.inf, -np.inf, np.nan], None)
        # Define the SQL query with named parameters
        update_query = text(f"""
            UPDATE Funds_to_Screen
            SET {db_cat} = df.{score_col}
            FROM Funds_to_Screen fts
            JOIN (SELECT :symbol_cusip AS SymbolCUSIP, :score AS {score_col}) df
            ON fts.SymbolCUSIP = df.SymbolCUSIP
        """)
        # Prepare data for update
        score_data = [(row["SymbolCUSIP"], row[score_col]) for _, row in funds_df.iterrows()]
        # Execute the update for each row individually using parameter binding
        for sym, score in score_data:
            # Skip if score is None (already converted from inf/-inf/nan)
            if score is None:
                params = {"symbol_cusip": sym, "score": None}
            else:
                params = {"symbol_cusip": sym, "score": float(score)}  # Ensure float for SQL Server
            conn.execute(update_query, params)
        conn.commit()  # Commit after each category update

    # Update the Return_Driver column
    update_driver_query = text("""
        UPDATE Funds_to_Screen
        SET return_driver = df.Return_Driver
        FROM Funds_to_Screen fts
        JOIN (SELECT :symbol_cusip AS SymbolCUSIP, :return_driver AS Return_Driver) df
        ON fts.SymbolCUSIP = df.SymbolCUSIP
    """)
    driver_data = [(row["SymbolCUSIP"], row["Return_Driver"]) for _, row in funds_df.iterrows()]
    for sym, driver in driver_data:
        conn.execute(update_driver_query, {"symbol_cusip": sym, "return_driver": driver})
    conn.commit()  # Commit after Return_Driver updates

# Export to Excel if toggled on (default off)
if write_to_excel:
    output_columns = (
        ["SymbolCUSIP", "ProductName", "fund_family", "Return_Driver", "ycharts_url"] +
        [f"score_{category_mapping[cat]}" for cat in categories] +
        [f"keyword_score_{category_mapping[cat]}" for cat in categories] +
        [f"meaningful_score_{category_mapping[cat]}" for cat in categories] +
        [f"assist_score_{category_mapping[cat]}" for cat in categories] +
        [f"boolean_score_{category_mapping[cat]}" for cat in categories] +
        [f"fundfamily_score_{category_mapping[cat]}" for cat in categories] +
        [f"matched_keywords_{category_mapping[cat]}" for cat in categories] +
        ["CWA_Broad_Category_Name", "Category_Name", "Global_Category_Name", "YC_Broad_Asset_Class_Name"] +
        ["index_fund", "inverse_fund", "leveraged_fund", "socially_responsible_fund", "synthetic_replication_fund", "fund_of_funds", "currency_hedged_fund"] +
        ["Dist_Index", "Dist_Active", "Dist_Rules_Based", "Dist_Quant", "Dist_Multi"]
    )
    funds_df[output_columns].to_excel(output_path, index=False)
    print(f"Results exported to {output_path}")
else:
    print("Excel output skipped (write_to_excel=False). Data written to database.")

Raw index_fund values before normalization: ['0', '1', '1', '1', '1']
Index_fund values after normalization: [0, 1, 1, 1, 1]
VLUE index_fund after normalization: [1]
Columns after initializing scores: ['SymbolCUSIP', 'ProductName', 'fund_family', 'investment_strategy', 'FS_insight', 'index_fund', 'inverse_fund', 'leveraged_fund', 'socially_responsible_fund', 'synthetic_replication_fund', 'fund_of_funds', 'ycharts_url', 'YC_Category_ID', 'CWA_Broad_Category_ID', 'YC_Global_Category_ID', 'YC_Broad_Asset_Class_ID', 'currency_hedged_fund', 'CWA_Broad_Category_Name', 'Category_Name', 'Global_Category_Name', 'YC_Broad_Asset_Class_Name', 'score_index_based', 'score_rules_based', 'score_active_discretionary', 'score_quant_systematic', 'score_multi_strategy', 'keyword_score_index_based', 'meaningful_score_index_based', 'assist_score_index_based', 'boolean_score_index_based', 'fundfamily_score_index_based', 'keyword_score_rules_based', 'meaningful_score_rules_based', 'assist_score_rules_based', 

In [ ]:
# First attempt at overlay classification



In [17]:
# Simple script to populate cash_long, cash_net, etc. columns in Funds_to_Screen
# Uses YCP API calls and mirrors core functionality from the provided script

import requests
import sqlalchemy
import pandas as pd
import logging
from concurrent.futures import ThreadPoolExecutor, TimeoutError
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Step 1: Configuration
# ---------------------

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Database connection string (same as provided script)
connection_string = (
    "mssql+pyodbc://JULIANS_LAPTOP\\SQLEXPRESS/"
    "CWA_Fund_Database?driver=ODBC+Driver+18+for+SQL+Server"
    "&trusted_connection=yes&TrustServerCertificate=yes"
)

# Create SQLAlchemy engine
engine = sqlalchemy.create_engine(connection_string)

# Test database connection
try:
    with engine.connect() as conn:
        conn.execute(sqlalchemy.text("SELECT 1"))
    print("Database connection successful")
except Exception as e:
    print(f"Database connection failed: {e}")

# YCharts API headers for YCP (POST) requests (same as provided script)
headers_YCP = {
    "X-YCHARTSAUTHORIZATION": "yIIphqbsQysnTvWWxfW33w",  # Replace with your actual API key
    "X-YCHARTSEXCELSESSION": "b645cd897b2446bfa3796acfa3a879db",
    "X-YCHARTSEXCELVERSION": "4.4",
    "X-YCHARTSOPERATINGSYSTEM": "Microsoft Windows NT 10.0.26100.0",
    "Content-Type": "application/x-www-form-urlencoded",
    "Host": "api.ycharts.com",
    "Connection": "Keep-Alive"
}

# Define base YCharts URL
yc_base_url = "https://api.ycharts.com/v3/"b

# Metrics to fetch (all use YCP call type)
metrics_to_fetch = [
    {"yc_metric": "cash_long", "db_column": "cash_long"},
    {"yc_metric": "cash_net", "db_column": "cash_net"},
    {"yc_metric": "cash_short", "db_column": "cash_short"},
    {"yc_metric": "stock_long", "db_column": "stock_long"},
    {"yc_metric": "stock_net", "db_column": "stock_net"},
    {"yc_metric": "stock_short", "db_column": "stock_short"},
    {"yc_metric": "bond_long", "db_column": "bond_long"},
    {"yc_metric": "bond_net", "db_column": "bond_net"},
    {"yc_metric": "bond_short", "db_column": "bond_short"},
    {"yc_metric": "other_long", "db_column": "other_long"},
    {"yc_metric": "other_net", "db_column": "other_net"},
    {"yc_metric": "other_short", "db_column": "other_short"},
]

# Step 2: Core Functions
# ----------------------

def fetch_ycp_metric(symbol, fund_type_id, yc_metric, headers_YCP=headers_YCP):
    """
    Fetch YCP data from YCharts API using a POST request.
    
    Args:
        symbol (str): Fund symbol (e.g., "BALCX").
        fund_type_id (int): Fund type (e.g., 3 for mutual funds).
        yc_metric (str): Metric to fetch (e.g., "cash_long").
        headers_YCP (dict): Headers for YCP request.
    
    Returns:
        Tuple: (symbol, normalized_data) or (None, error_info).
    """
    # Set up YCP URL and payload
    ycp_api_url = "https://api.ycharts.com/v3/excel/points"
    symbol_prefix = f"M:{symbol}" if fund_type_id == 3 else symbol
    data = f"points={symbol_prefix},{yc_metric}"
    logging.debug(f"YCP payload for {symbol}: {data}")

    try:
        # Make POST request with SSL bypass
        response = requests.post(ycp_api_url, headers=headers_YCP, data=data, verify=False)
        if response.status_code != 200:
            logging.error(f"YCP HTTP error for {symbol}: {response.status_code}")
            return None, {"error": f"HTTP {response.status_code}"}
        
        # Parse JSON response
        data = response.json()
        logging.debug(f"YCP response for {symbol}: {data}")

        # Normalize data
        normalized_data = normalize_api_data(data, symbol, yc_metric, fund_type_id=fund_type_id)
        return symbol, normalized_data

    except requests.RequestException as e:
        logging.error(f"YCP request failed for {symbol}: {str(e)}")
        return None, {"error": str(e)}

def normalize_api_data(data, symbol, yc_metric, fund_type_id=None):
    """
    Normalize YCharts API YCP response data into a float.
    
    Args:
        data (dict): Raw JSON response from API.
        symbol (str): Fund symbol.
        yc_metric (str): Metric name.
        fund_type_id (int): Fund type (e.g., 3 for mutual funds).
    
    Returns:
        float or None if parsing fails.
    """
    try:
        # Adjust symbol key for mutual funds
        response_key = f"M:{symbol}" if fund_type_id == 3 else symbol
        if "response" not in data or response_key not in data["response"]:
            logging.warning(f"No response data for {symbol}: {data}")
            return None

        # Extract results
        results = data["response"][response_key].get("results", {})
        if yc_metric not in results:
            logging.warning(f"No {yc_metric} in results for {symbol}: {results}")
            return None

        metric_data = results[yc_metric]
        logging.debug(f"Raw metric data for {yc_metric} ({symbol}): {metric_data}")

        # YCP expects ['']['results'] structure (e.g., ["2025-01-01", 0.123])
        if "" in metric_data and "results" in metric_data[""]:
            data_list = metric_data[""]["results"]
            if isinstance(data_list, list) and len(data_list) >= 1:
                # Take second value if available, else first
                raw_data = data_list[1] if len(data_list) > 1 else data_list[0]
                if raw_data is not None:
                    return float(raw_data)
                logging.info(f"Null value in {yc_metric} for {symbol}: {data_list}")
                return None
            logging.warning(f"Invalid list format for {yc_metric} ({symbol}): {data_list}")
            return None
        else:
            logging.warning(f"No ['']['results'] in {yc_metric} for {symbol}: {metric_data}")
            return None

    except Exception as e:
        logging.error(f"Normalization failed for {yc_metric} ({symbol}): {str(e)}")
        return None

def insert_to_database(df, column_name, batch_size=1000):
    """
    Insert or update data into Funds_to_Screen table in batches.
    
    Args:
        df (DataFrame): DataFrame with SymbolCUSIP and column_name data.
        column_name (str): Name of the column to update.
        batch_size (int): Number of rows per batch.
    
    Returns:
        Tuple: (successes, failures, database_errors)
    """
    successes = 0
    failures = 0
    database_errors = []
    print(f"Updating {column_name} for {len(df)} funds")

    for i in range(0, len(df), batch_size):
        batch = df.iloc[i:i + batch_size]
        try:
            with engine.begin() as conn:
                updates = [{"symbol": row["SymbolCUSIP"], "value": row[column_name]} for _, row in batch.iterrows()]
                conn.execute(
                    sqlalchemy.text(f"""
                        UPDATE Funds_to_Screen 
                        SET {column_name} = :value
                        WHERE SymbolCUSIP = :symbol
                    """),
                    updates
                )
            successes += len(batch)
        except sqlalchemy.exc.SQLAlchemyError as e:
            logging.error(f"Database error: {str(e)}")
            failures += len(batch)
            database_errors.extend(batch["SymbolCUSIP"].tolist())

    return successes, failures, database_errors

# Step 3: Main Function to Update Metrics
# ---------------------------------------

def update_metric(yc_metric, db_column_name):
    """
    Update a specific metric for funds where the column is NULL.
    
    Args:
        yc_metric (str): YCharts metric name (e.g., "cash_long").
        db_column_name (str): Database column name (e.g., "cash_long").
    """
    # Fetch funds where the column is NULL
    query = f"SELECT SymbolCUSIP, Fund_Type_ID FROM Funds_to_Screen WHERE {db_column_name} IS NULL"
    funds = pd.read_sql(query, engine)
    logging.info(f"Found {len(funds)} funds needing {yc_metric} updates")

    if len(funds) == 0:
        print(f"No funds need {yc_metric.replace('_', ' ').title()} updates.")
        return

    print(f"Requesting {yc_metric.replace('_', ' ').title()} for {len(funds)} funds from YCharts...")
    data_list = []
    no_data_count = 0

    def fetch_for_fund(row):
        try:
            symbol, normalized_data = fetch_ycp_metric(row["SymbolCUSIP"], row["Fund_Type_ID"], yc_metric)
            if normalized_data is not None and not isinstance(normalized_data, dict):
                return (symbol, normalized_data)
            return (symbol, None, "No data" if normalized_data is None else normalized_data.get("error", "Unknown error"))
        except Exception as e:
            logging.error(f"Fetch error for {row['SymbolCUSIP']}: {str(e)}")
            return (row["SymbolCUSIP"], None, str(e))

    # Use ThreadPoolExecutor for parallel API calls
    with ThreadPoolExecutor(max_workers=60) as executor:
        future_to_row = {executor.submit(fetch_for_fund, row): row for _, row in funds.iterrows()}
        results = []
        for future in future_to_row:
            try:
                result = future.result(timeout=30)
                results.append(result)
            except TimeoutError:
                symbol = future_to_row[future]["SymbolCUSIP"]
                logging.error(f"Timeout fetching {yc_metric} for {symbol}")
                results.append((symbol, None, "Timeout after 30s"))

    # Process results
    for symbol, normalized_data, *error in results:
        if normalized_data is not None:
            data_list.append((symbol, normalized_data))
        else:
            no_data_count += 1
            if error and error[0] != "No data":
                logging.error(f"API error for {symbol}: {error[0]}")

    if not data_list:
        print(f"No data fetched for {yc_metric}. No updates performed.")
        print(f"Summary - Funds Needing Update: {len(funds)}, No Data: {no_data_count}")
        return

    # Insert data into database
    df = pd.DataFrame(data_list, columns=["SymbolCUSIP", db_column_name])
    logging.info(f"Inserting {len(df)} rows for {yc_metric}")
    insert_successes, insert_failures, database_errors = insert_to_database(df, db_column_name)

    # Print summary
    print(f"\nSummary for {yc_metric}:")
    print(f"  Funds Needing Update: {len(funds)}")
    print(f"  Data Fetched: {len(df)}")
    print(f"  Inserted: {insert_successes}")
    print(f"  No Data: {no_data_count}")
    print(f"  Database Errors: {insert_failures}")
    print(f"  Failed Symbols: {';'.join(database_errors) if database_errors else 'None'}")

# Step 4: Main Execution
# ----------------------

if __name__ == "__main__":
    # Loop through each metric and update
    for config in metrics_to_fetch:
        update_metric(config["yc_metric"], config["db_column"])

2025-03-03 12:40:01,138 - INFO - Found 5586 funds needing cash_long updates


Database connection successful
Requesting Cash Long for 5586 funds from YCharts...


2025-03-03 12:40:04,228 - WARNING - No cash_long in results for AMPD: {}
2025-03-03 12:40:06,157 - WARNING - No cash_long in results for VMOT: {}
2025-03-03 12:40:13,983 - INFO - Null value in cash_long for HSBH: [None, None]
2025-03-03 12:40:13,988 - INFO - Null value in cash_long for AZNH: [None, None]
2025-03-03 12:40:14,259 - INFO - Null value in cash_long for SHEH: [None, None]
2025-03-03 12:40:16,162 - WARNING - No cash_long in results for PP: {}
2025-03-03 12:40:17,679 - WARNING - No cash_long in results for VCAR: {}
2025-03-03 12:40:46,893 - INFO - Null value in cash_long for PSQA: [None, None]
2025-03-03 12:40:56,488 - INFO - Null value in cash_long for APDOX: [None, None]
2025-03-03 12:41:05,993 - WARNING - No cash_long in results for OFIYX: {}
2025-03-03 12:41:45,892 - WARNING - No cash_long in results for TWIO: {}
2025-03-03 12:41:52,126 - INFO - Null value in cash_long for HGER: [None, None]
2025-03-03 12:41:53,009 - INFO - Null value in cash_long for IAUM: [None, None]
20

Updating cash_long for 5536 funds


2025-03-03 12:46:45,224 - INFO - Found 5586 funds needing cash_net updates


...

Updating other_net for 5536 funds


2025-03-03 13:35:03,758 - INFO - Found 5586 funds needing other_short updates



Summary for other_net:
  Funds Needing Update: 5586
  Data Fetched: 5536
  Inserted: 5536
  No Data: 50
  Database Errors: 0
  Failed Symbols: None
Requesting Other Short for 5586 funds from YCharts...


2025-03-03 13:35:07,854 - WARNING - No other_short in results for VMOT: {}
2025-03-03 13:35:09,179 - WARNING - No other_short in results for AMPD: {}
2025-03-03 13:35:13,260 - INFO - Null value in other_short for HSBH: [None, None]
2025-03-03 13:35:13,699 - INFO - Null value in other_short for SHEH: [None, None]
2025-03-03 13:35:13,768 - INFO - Null value in other_short for AZNH: [None, None]
2025-03-03 13:35:15,597 - WARNING - No other_short in results for VCAR: {}
2025-03-03 13:35:15,677 - WARNING - No other_short in results for PP: {}
2025-03-03 13:35:33,577 - INFO - Null value in other_short for PSQA: [None, None]
2025-03-03 13:35:36,860 - INFO - Null value in other_short for APDOX: [None, None]
2025-03-03 13:35:44,165 - WARNING - No other_short in results for OFIYX: {}
2025-03-03 13:35:57,533 - WARNING - No other_short in results for TWIO: {}
2025-03-03 13:36:00,185 - WARNING - No other_short in results for DFRA: {}
2025-03-03 13:36:01,957 - WARNING - No other_short in results for

Updating other_short for 5536 funds

Summary for other_short:
  Funds Needing Update: 5586
  Data Fetched: 5536
  Inserted: 5536
  No Data: 50
  Database Errors: 0
  Failed Symbols: None


In [1]:
# All Exposure metrics verses selected ones 

# Simple script to populate cash_long, cash_net, etc. columns in Funds_to_Screen
# Uses YCP API calls and mirrors core functionality from the provided script

import requests
import sqlalchemy
import pandas as pd
import logging
from concurrent.futures import ThreadPoolExecutor, TimeoutError
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Step 1: Configuration
# ---------------------

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Database connection string (same as provided script)
connection_string = (
    "mssql+pyodbc://JULIANS_LAPTOP\\SQLEXPRESS/"
    "CWA_Fund_Database?driver=ODBC+Driver+18+for+SQL+Server"
    "&trusted_connection=yes&TrustServerCertificate=yes"
)

# Create SQLAlchemy engine
engine = sqlalchemy.create_engine(connection_string)

# Test database connection
try:
    with engine.connect() as conn:
        conn.execute(sqlalchemy.text("SELECT 1"))
    print("Database connection successful")
except Exception as e:
    print(f"Database connection failed: {e}")

# YCharts API headers for YCP (POST) requests (same as provided script)
headers_YCP = {
    "X-YCHARTSAUTHORIZATION": "yIIphqbsQysnTvWWxfW33w",  # Replace with your actual API key
    "X-YCHARTSEXCELSESSION": "b645cd897b2446bfa3796acfa3a879db",
    "X-YCHARTSEXCELVERSION": "4.4",
    "X-YCHARTSOPERATINGSYSTEM": "Microsoft Windows NT 10.0.26100.0",
    "Content-Type": "application/x-www-form-urlencoded",
    "Host": "api.ycharts.com",
    "Connection": "Keep-Alive"
}

# Define base YCharts URL
yc_base_url = "https://api.ycharts.com/v3/"

# Metrics to fetch (all use YCP call type) - Added new preferred and convertible metrics
metrics_to_fetch = [
    {"yc_metric": "cash_long", "db_column": "cash_long"},
    {"yc_metric": "cash_net", "db_column": "cash_net"},
    {"yc_metric": "cash_short", "db_column": "cash_short"},
    {"yc_metric": "stock_long", "db_column": "stock_long"},
    {"yc_metric": "stock_net", "db_column": "stock_net"},
    {"yc_metric": "stock_short", "db_column": "stock_short"},
    {"yc_metric": "bond_long", "db_column": "bond_long"},
    {"yc_metric": "bond_net", "db_column": "bond_net"},
    {"yc_metric": "bond_short", "db_column": "bond_short"},
    {"yc_metric": "other_long", "db_column": "other_long"},
    {"yc_metric": "other_net", "db_column": "other_net"},
    {"yc_metric": "other_short", "db_column": "other_short"},
    {"yc_metric": "preferred_long", "db_column": "preferred_long"},
    {"yc_metric": "preferred_net", "db_column": "preferred_net"},
    {"yc_metric": "preferred_short", "db_column": "preferred_short"},
    {"yc_metric": "convertible_long", "db_column": "convertible_long"},
    {"yc_metric": "convertible_net", "db_column": "convertible_net"},
    {"yc_metric": "convertible_short", "db_column": "convertible_short"},
]

# Step 2: Core Functions
# ----------------------

def fetch_ycp_metric(symbol, fund_type_id, yc_metric, headers_YCP=headers_YCP):
    """
    Fetch YCP data from YCharts API using a POST request.
    
    Args:
        symbol (str): Fund symbol (e.g., "BALCX").
        fund_type_id (int): Fund type (e.g., 3 for mutual funds).
        yc_metric (str): Metric to fetch (e.g., "cash_long").
        headers_YCP (dict): Headers for YCP request.
    
    Returns:
        Tuple: (symbol, normalized_data) or (None, error_info).
    """
    # Set up YCP URL and payload
    ycp_api_url = "https://api.ycharts.com/v3/excel/points"
    symbol_prefix = f"M:{symbol}" if fund_type_id == 3 else symbol
    data = f"points={symbol_prefix},{yc_metric}"
    logging.debug(f"YCP payload for {symbol}: {data}")

    try:
        # Make POST request with SSL bypass
        response = requests.post(ycp_api_url, headers=headers_YCP, data=data, verify=False)
        if response.status_code != 200:
            logging.error(f"YCP HTTP error for {symbol}: {response.status_code}")
            return None, {"error": f"HTTP {response.status_code}"}
        
        # Parse JSON response
        data = response.json()
        logging.debug(f"YCP response for {symbol}: {data}")

        # Normalize data
        normalized_data = normalize_api_data(data, symbol, yc_metric, fund_type_id=fund_type_id)
        return symbol, normalized_data

    except requests.RequestException as e:
        logging.error(f"YCP request failed for {symbol}: {str(e)}")
        return None, {"error": str(e)}

def normalize_api_data(data, symbol, yc_metric, fund_type_id=None):
    """
    Normalize YCharts API YCP response data into a float.
    
    Args:
        data (dict): Raw JSON response from API.
        symbol (str): Fund symbol.
        yc_metric (str): Metric name.
        fund_type_id (int): Fund type (e.g., 3 for mutual funds).
    
    Returns:
        float or None if parsing fails.
    """
    try:
        # Adjust symbol key for mutual funds
        response_key = f"M:{symbol}" if fund_type_id == 3 else symbol
        if "response" not in data or response_key not in data["response"]:
            logging.warning(f"No response data for {symbol}: {data}")
            return None

        # Extract results
        results = data["response"][response_key].get("results", {})
        if yc_metric not in results:
            logging.warning(f"No {yc_metric} in results for {symbol}: {results}")
            return None

        metric_data = results[yc_metric]
        logging.debug(f"Raw metric data for {yc_metric} ({symbol}): {metric_data}")

        # YCP expects ['']['results'] structure (e.g., ["2025-01-01", 0.123])
        if "" in metric_data and "results" in metric_data[""]:
            data_list = metric_data[""]["results"]
            if isinstance(data_list, list) and len(data_list) >= 1:
                # Take second value if available, else first
                raw_data = data_list[1] if len(data_list) > 1 else data_list[0]
                if raw_data is not None:
                    return float(raw_data)
                logging.info(f"Null value in {yc_metric} for {symbol}: {data_list}")
                return None
            logging.warning(f"Invalid list format for {yc_metric} ({symbol}): {data_list}")
            return None
        else:
            logging.warning(f"No ['']['results'] in {yc_metric} for {symbol}: {metric_data}")
            return None

    except Exception as e:
        logging.error(f"Normalization failed for {yc_metric} ({symbol}): {str(e)}")
        return None

def insert_to_database(df, column_name, batch_size=1000):
    """
    Insert or update data into Funds_to_Screen table in batches.
    
    Args:
        df (DataFrame): DataFrame with SymbolCUSIP and column_name data.
        column_name (str): Name of the column to update.
        batch_size (int): Number of rows per batch.
    
    Returns:
        Tuple: (successes, failures, database_errors)
    """
    successes = 0
    failures = 0
    database_errors = []
    print(f"Updating {column_name} for {len(df)} funds")

    for i in range(0, len(df), batch_size):
        batch = df.iloc[i:i + batch_size]
        try:
            with engine.begin() as conn:
                updates = [{"symbol": row["SymbolCUSIP"], "value": row[column_name]} for _, row in batch.iterrows()]
                conn.execute(
                    sqlalchemy.text(f"""
                        UPDATE Funds_to_Screen 
                        SET {column_name} = :value
                        WHERE SymbolCUSIP = :symbol
                    """),
                    updates
                )
            successes += len(batch)
        except sqlalchemy.exc.SQLAlchemyError as e:
            logging.error(f"Database error: {str(e)}")
            failures += len(batch)
            database_errors.extend(batch["SymbolCUSIP"].tolist())

    return successes, failures, database_errors

# Step 3: Main Function to Update Metrics
# ---------------------------------------

def update_metric(yc_metric, db_column_name):
    """
    Update a specific metric for funds where the column is NULL or needs updating.
    
    Args:
        yc_metric (str): YCharts metric name (e.g., "cash_long").
        db_column_name (str): Database column name (e.g., "cash_long").
    """
    # Fetch all funds to check data population
    query = f"""
        SELECT SymbolCUSIP, Fund_Type_ID, {db_column_name}
        FROM Funds_to_Screen
    """
    funds = pd.read_sql(query, engine)
    
    # Filter funds where data is NULL
    funds_to_update = funds[funds[db_column_name].isna()]
    logging.info(f"Found {len(funds_to_update)} funds needing {yc_metric} updates out of {len(funds)} total")

    if len(funds_to_update) == 0:
        print(f"No funds need {yc_metric.replace('_', ' ').title()} updates.")
        return

    print(f"Requesting {yc_metric.replace('_', ' ').title()} for {len(funds_to_update)} funds from YCharts...")
    data_list = []
    no_data_count = 0

    def fetch_for_fund(row):
        try:
            symbol, normalized_data = fetch_ycp_metric(row["SymbolCUSIP"], row["Fund_Type_ID"], yc_metric)
            if normalized_data is not None and not isinstance(normalized_data, dict):
                return (symbol, normalized_data)
            return (symbol, None, "No data" if normalized_data is None else normalized_data.get("error", "Unknown error"))
        except Exception as e:
            logging.error(f"Fetch error for {row['SymbolCUSIP']}: {str(e)}")
            return (row["SymbolCUSIP"], None, str(e))

    # Use ThreadPoolExecutor for parallel API calls
    with ThreadPoolExecutor(max_workers=60) as executor:
        future_to_row = {executor.submit(fetch_for_fund, row): row for _, row in funds_to_update.iterrows()}
        results = []
        for future in future_to_row:
            try:
                result = future.result(timeout=30)
                results.append(result)
            except TimeoutError:
                symbol = future_to_row[future]["SymbolCUSIP"]
                logging.error(f"Timeout fetching {yc_metric} for {symbol}")
                results.append((symbol, None, "Timeout after 30s"))

    # Process results
    for symbol, normalized_data, *error in results:
        if normalized_data is not None:
            data_list.append((symbol, normalized_data))
        else:
            no_data_count += 1
            if error and error[0] != "No data":
                logging.error(f"API error for {symbol}: {error[0]}")

    if not data_list:
        print(f"No data fetched for {yc_metric}. No updates performed.")
        print(f"Summary - Funds Needing Update: {len(funds_to_update)}, No Data: {no_data_count}")
        return

    # Insert data into database
    df = pd.DataFrame(data_list, columns=["SymbolCUSIP", db_column_name])
    logging.info(f"Inserting {len(df)} rows for {yc_metric}")
    insert_successes, insert_failures, database_errors = insert_to_database(df, db_column_name)

    # Print summary
    print(f"\nSummary for {yc_metric}:")
    print(f"  Total Funds: {len(funds)}")
    print(f"  Funds Needing Update: {len(funds_to_update)}")
    print(f"  Data Fetched: {len(df)}")
    print(f"  Inserted: {insert_successes}")
    print(f"  No Data: {no_data_count}")
    print(f"  Database Errors: {insert_failures}")
    print(f"  Failed Symbols: {';'.join(database_errors) if database_errors else 'None'}")

# Step 4: Main Execution
# ----------------------

if __name__ == "__main__":
    # Loop through each metric and update
    for config in metrics_to_fetch:
        update_metric(config["yc_metric"], config["db_column"])

2025-03-10 14:34:52,560 - INFO - Found 50 funds needing cash_long updates out of 5586 total


Database connection successful
Requesting Cash Long for 50 funds from YCharts...


2025-03-10 14:34:53,481 - WARNING - No cash_long in results for IMSI: {}
2025-03-10 14:34:53,678 - WARNING - No cash_long in results for NETZ: {}
2025-03-10 14:34:53,681 - WARNING - No cash_long in results for GPOW: {}
2025-03-10 14:34:53,685 - INFO - Null value in cash_long for FMCE: [None, None]
2025-03-10 14:34:54,081 - WARNING - No cash_long in results for FYLG: {}
2025-03-10 14:34:54,083 - WARNING - No cash_long in results for MUIBX: {}
2025-03-10 14:34:54,101 - WARNING - No cash_long in results for FPPJX: {}
2025-03-10 14:34:54,110 - WARNING - No cash_long in results for DFRA: {}
2025-03-10 14:34:54,176 - INFO - Null value in cash_long for IAUM: [None, None]
2025-03-10 14:34:54,223 - INFO - Null value in cash_long for BCIM: [None, None]
2025-03-10 14:34:54,600 - WARNING - No cash_long in results for PP: {}
2025-03-10 14:34:54,602 - WARNING - No cash_long in results for VCAR: {}
2025-03-10 14:34:54,604 - WARNING - No cash_long in results for USCF: {}
2025-03-10 14:34:54,628 - WARN

Updating cash_long for 5 funds

Summary for cash_long:
  Total Funds: 5586
  Funds Needing Update: 50
  Data Fetched: 5
  Inserted: 5
  No Data: 45
  Database Errors: 0
  Failed Symbols: None
Requesting Cash Net for 50 funds from YCharts...


2025-03-10 14:34:56,713 - WARNING - No cash_net in results for EMCC: {}
2025-03-10 14:34:56,739 - WARNING - No cash_net in results for PP: {}
2025-03-10 14:34:56,751 - WARNING - No cash_net in results for HYLG: {}
2025-03-10 14:34:56,765 - WARNING - No cash_net in results for TTAI: {}
2025-03-10 14:34:56,854 - WARNING - No cash_net in results for TWIO: {}
2025-03-10 14:34:56,881 - WARNING - No cash_net in results for NUSI: {}
2025-03-10 14:34:56,950 - WARNING - No cash_net in results for QYLE: {}
2025-03-10 14:34:56,966 - WARNING - No cash_net in results for VMOT: {}
2025-03-10 14:34:56,966 - WARNING - No cash_net in results for FYLG: {}
2025-03-10 14:34:56,967 - INFO - Null value in cash_net for BCIM: [None, None]
2025-03-10 14:34:56,967 - WARNING - No cash_net in results for XYLE: {}
2025-03-10 14:34:56,967 - INFO - Null value in cash_net for APDOX: [None, None]
2025-03-10 14:34:56,967 - WARNING - No cash_net in results for DFRA: {}
2025-03-10 14:34:56,968 - WARNING - No cash_net in 

...

Updating convertible_net for 5541 funds


2025-03-10 14:55:33,660 - INFO - Found 5586 funds needing convertible_short updates out of 5586 total



Summary for convertible_net:
  Total Funds: 5586
  Funds Needing Update: 5586
  Data Fetched: 5541
  Inserted: 5541
  No Data: 45
  Database Errors: 0
  Failed Symbols: None
Requesting Convertible Short for 5586 funds from YCharts...


2025-03-10 14:55:35,585 - WARNING - No convertible_short in results for AMPD: {}
2025-03-10 14:55:37,118 - WARNING - No convertible_short in results for VMOT: {}
2025-03-10 14:55:43,552 - WARNING - No convertible_short in results for PP: {}
2025-03-10 14:55:43,707 - WARNING - No convertible_short in results for VCAR: {}
2025-03-10 14:56:00,008 - INFO - Null value in convertible_short for APDOX: [None, None]
2025-03-10 14:56:06,554 - WARNING - No convertible_short in results for OFIYX: {}
2025-03-10 14:56:18,874 - WARNING - No convertible_short in results for TWIO: {}
2025-03-10 14:56:21,454 - WARNING - No convertible_short in results for DFRA: {}
2025-03-10 14:56:22,220 - WARNING - No convertible_short in results for DFNV: {}
2025-03-10 14:56:24,950 - INFO - Null value in convertible_short for HGER: [None, None]
2025-03-10 14:56:27,227 - INFO - Null value in convertible_short for BCIM: [None, None]
2025-03-10 14:56:27,412 - INFO - Null value in convertible_short for IAUM: [None, None]


Updating convertible_short for 5541 funds

Summary for convertible_short:
  Total Funds: 5586
  Funds Needing Update: 5586
  Data Fetched: 5541
  Inserted: 5541
  No Data: 45
  Database Errors: 0
  Failed Symbols: None


In [30]:
# Risk Overlays v2 - Classification of Funds Based on Options Usage with Enhanced Debugging

import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
from collections import defaultdict

# Adjustable toggle for Excel output
write_to_excel = True

# Define output path for Excel
output_path = r"C:\Users\JulianHeron\Software Projects\overlay_categories.xlsx"

# Database connection
connection_string = (
    "mssql+pyodbc://JULIANS_LAPTOP\\SQLEXPRESS/"
    "CWA_Fund_Database?driver=ODBC+Driver+18+for+SQL+Server"
    "&trusted_connection=yes&TrustServerCertificate=yes"
)
engine = create_engine(connection_string)

# Expanded and refined keywords for each category
keywords = {
    "Slight/None": [
        "long-only", "no derivatives", "no hedging", "no leverage", "no options", "no short",
        "position adjustment", "occasional hedging", "covered call", "put-write", "light hedge",
        "may include options", "limited use of derivatives", "for risk management purposes",
        "minor hedging", "occasional short positions", "overwrite",
    ],
    "Moderate": [
        "hedged", "currency hedge", "protected", "buffer", "partial hedge", "hedged equity", "covered call",
        "convexity option overlay", "option overlay", "put/spread collar", "forward agreement", "enhanced index strategy",
        "BuyWrite", "Buy-Write", "buy write", "option spread", "volatility hedge", "put options", "enhance",
        "options-based income", "ELN", "premium income", "call option", "FLEX options", "option premium",
        "write calls", "sell calls", "protective puts", "equity-linked notes", "structured notes",
        "risk mitigation", "downside protection", "limited hedging",
        "floating rate",
    ],
    "Persistent Systematic": [
        "tail-risk", "trend-following", "systematic hedging", "overlay", "CTA", "managed futures",
        "defined outcome", "long-short", "market neutral", "systematic strategy", "return stacking",
        "option writing", "straddle", "derivative income", "futures contracts", "swap contract", "forward agreement",
        "enhanced index strategy", "volatility hedge", "put options", "options-based income", "ELN", "option premium",
        "swap", "forward", "futures", "future", "VIX",
        "managed futures strategy", "trend strategy", "quantitative hedging", "systematic options",
        "options overlay strategy", "futures overlay", "swaps-based",
    ],
    "Heavy Amplification": [
        "leveraged", "2x", "3x", "ultra", "amplified", "systematic leverage",
        "double exposure", "triple exposure", "enhanced leverage", "geared",
    ]
}

# Convert keywords to lowercase for matching
keywords = {cat: [term.lower() for term in terms] for cat, terms in keywords.items()}

# Define keyword scoring multipliers to reflect category intensity
keyword_multipliers = {
    "Slight/None": 1.0,
    "Moderate": 2.0,
    "Persistent Systematic": 3.0,
    "Heavy Amplification": 2.0
}

# Define meaningful categories for direct classification
meaningful_categories = {
    "Moderate": {
        "CWA_Broad_Category": [],
        "YC_Global_Category": [],
        "YC_Category": ["Derivative Income", "Trading--Miscellaneous", "Bank Loan"]
    },
    "Persistent Systematic": {
        "CWA_Broad_Category": ["Trading/Tactical", "Alternatives"],
        "YC_Global_Category": ["Options Trading", "Market Neutral", "Long/Short Equity", "Multialternative"],
        "YC_Category": ["Defined Outcome", "Relative Value Arbitrage", "Systematic Trend", "Trading--Inverse Commodities", "Trading--Inverse Debt", "Trading--Inverse Equity"]
    },
    "Heavy Amplification": {
        "YC_Category": ["Trading--Leveraged Commodities", "Trading--Leveraged Debt", "Trading--Leveraged Equity"]
    }
}

# Simplified assist categories
assist_categories = [
    {
        "exact": "Long/Short Equity",
        "cat_type": "YC_Global_Category",
        "actions": {"boost": ["Persistent Systematic"], "remove": ["Slight/None", "Moderate"]}
    },
    {
        "exact": "Market Neutral",
        "cat_type": "YC_Global_Category",
        "actions": {"boost": ["Persistent Systematic"], "remove": ["Slight/None", "Moderate"]}
    },
    {
        "exact": "Systematic Trend",
        "cat_type": "YC_Category",
        "actions": {"boost": ["Persistent Systematic"], "remove": ["Slight/None", "Moderate"]}
    },
    {
        "exact": "Defined Outcome",
        "cat_type": "CWA_Broad_Category",
        "actions": {"boost": ["Persistent Systematic"]}
    },
    {
        "exact": "Defined Outcome",
        "cat_type": "YC_Global_Category",
        "actions": {"boost": ["Persistent Systematic"]}
    },
    {
        "exact": "Defined Outcome",
        "cat_type": "YC_Category",
        "actions": {"boost": ["Persistent Systematic"]}
    },
    {
        "exact": "Trading--Leveraged Equity",
        "cat_type": "YC_Category",
        "actions": {"boost": ["Heavy Amplification"], "remove": ["Slight/None", "Moderate", "Persistent Systematic"]}
    },
    {
        "exact": "Trading--Leveraged Commodities",
        "cat_type": "YC_Category",
        "actions": {"boost": ["Heavy Amplification"], "remove": ["Slight/None", "Moderate", "Persistent Systematic"]}
    },
    {
        "exact": "Trading--Leveraged Debt",
        "cat_type": "YC_Category",
        "actions": {"boost": ["Heavy Amplification"], "remove": ["Slight/None", "Moderate", "Persistent Systematic"]}
    },
    {
        "exact": "Bank Loan",
        "cat_type": "YC_Category",
        "actions": {"boost": ["Moderate"], "remove": ["Heavy Amplification"]}
    },
    {
        "exact": "Options Trading",
        "cat_type": "YC_Global_Category",
        "actions": {"boost": ["Persistent Systematic"], "remove": ["Slight/None"]}
    },
    {
        "exact": "Multialternative",
        "cat_type": "YC_Global_Category",
        "actions": {"boost": ["Persistent Systematic"], "remove": ["Slight/None", "Moderate"]}
    },
    {
        "exact": "Relative Value Arbitrage",
        "cat_type": "YC_Category",
        "actions": {"boost": ["Persistent Systematic"], "remove": ["Slight/None", "Moderate"]}
    },
    {
        "exact": "Trading--Inverse Commodities",
        "cat_type": "YC_Category",
        "actions": {"boost": ["Persistent Systematic"], "remove": ["Slight/None", "Moderate"]}
    },
    {
        "exact": "Trading--Inverse Debt",
        "cat_type": "YC_Category",
        "actions": {"boost": ["Persistent Systematic"], "remove": ["Slight/None", "Moderate"]}
    },
    {
        "exact": "Trading--Inverse Equity",
        "cat_type": "YC_Category",
        "actions": {"boost": ["Persistent Systematic"], "remove": ["Slight/None", "Moderate"]}
    },
    {
        "exact": "Derivative Income",
        "cat_type": "YC_Category",
        "actions": {"boost": ["Persistent Systematic", "Moderate"]}
    },
    {
        "exact": "Trading--Miscellaneous",
        "cat_type": "YC_Category",
        "actions": {"boost": ["Persistent Systematic", "Moderate"]}
    }
]

# Load data from database
query_funds = """
SELECT SymbolCUSIP, ProductName, fund_family, investment_strategy, FS_insight, index_fund,
       inverse_fund, leveraged_fund, socially_responsible_fund, synthetic_replication_fund,
       fund_of_funds, currency_hedged_fund, ycharts_url, YC_Category_ID, CWA_Broad_Category_ID,
       YC_Global_Category_ID, YC_Broad_Asset_Class_ID,
       stock_net, stock_long, stock_short,
       bond_net, bond_long, bond_short,
       cash_net, cash_long, cash_short,
       other_net, other_long, other_short
FROM Funds_to_Screen
"""
funds_df = pd.read_sql(query_funds, engine)

# Debug: Check BFRIX's CWA_Broad_Category_ID before merge
bfrix_row_before_merge = funds_df[funds_df["SymbolCUSIP"] == "BFRIX"]
if not bfrix_row_before_merge.empty:
    print(f"\nBFRIX CWA_Broad_Category_ID before merge: {bfrix_row_before_merge['CWA_Broad_Category_ID'].values[0]}")

# Load category mappings
category_mappings = {
    "CWA_Broad_Category": pd.read_sql("SELECT ID, CWA_Broad_Category_Name FROM CWA_Broad_Category_List", engine),
    "YC_Category": pd.read_sql("SELECT ID, Category_Name FROM YC_Category_List", engine),
    "YC_Global_Category": pd.read_sql("SELECT ID, Global_Category_Name FROM YC_Global_Category_List", engine),
    "YC_Broad_Asset_Class": pd.read_sql("SELECT ID, YC_Broad_Asset_Class_Name FROM YC_Broad_Asset_Class_List", engine)
}

# Debug: Print CWA_Broad_Category_List contents
print("CWA_Broad_Category_List contents:")
print(category_mappings["CWA_Broad_Category"])

# Debug: Check for unmatched CWA_Broad_Category_IDs
unmatched_ids = funds_df[~funds_df["CWA_Broad_Category_ID"].isin(category_mappings["CWA_Broad_Category"]["ID"])]["CWA_Broad_Category_ID"].unique()
print(f"\nUnmatched CWA_Broad_Category_IDs: {unmatched_ids}")

# Merge category names into funds_df
funds_df = funds_df.merge(category_mappings["CWA_Broad_Category"], left_on="CWA_Broad_Category_ID", right_on="ID", how="left").drop(columns=["ID"])
funds_df = funds_df.merge(category_mappings["YC_Category"], left_on="YC_Category_ID", right_on="ID", how="left").drop(columns=["ID"])
funds_df = funds_df.merge(category_mappings["YC_Global_Category"], left_on="YC_Global_Category_ID", right_on="ID", how="left").drop(columns=["ID"])
funds_df = funds_df.merge(category_mappings["YC_Broad_Asset_Class"], left_on="YC_Broad_Asset_Class_ID", right_on="ID", how="left").drop(columns=["ID"])

# Debug: Print unique CWA_Broad_Category_Name values after merge
print("\nUnique CWA_Broad_Category_Name values after merge:")
print(funds_df["CWA_Broad_Category_Name"].unique())

# Debug: Check for NaN values in CWA_Broad_Category_Name
nan_count = funds_df["CWA_Broad_Category_Name"].isna().sum()
print(f"\nNumber of NaN values in CWA_Broad_Category_Name: {nan_count}")

# Debug: Print CWA_Broad_Category_Name for BFRIX after merge
bfrix_row = funds_df[funds_df["SymbolCUSIP"] == "BFRIX"]
if not bfrix_row.empty:
    print(f"\nCWA_Broad_Category_Name for BFRIX after merge: {bfrix_row['CWA_Broad_Category_Name'].values[0]}")

# Normalize Boolean fields
boolean_cols = ["index_fund", "inverse_fund", "leveraged_fund", "socially_responsible_fund",
               "synthetic_replication_fund", "fund_of_funds", "currency_hedged_fund"]
for col in boolean_cols:
    funds_df[col] = funds_df[col].apply(lambda x: 1 if str(x).lower() in ['true', '1', 'yes'] else 0)

# Ensure position columns are numeric and handle missing values
position_cols = [
    "stock_net", "stock_long", "stock_short",
    "bond_net", "bond_long", "bond_short",
    "cash_net", "cash_long", "cash_short",
    "other_net", "other_long", "other_short"
]
for col in position_cols:
    funds_df[col] = pd.to_numeric(funds_df[col], errors='coerce').fillna(0)

# Debug: Check CWA_Broad_Category_Name for BFRIX after normalization
bfrix_row = funds_df[funds_df["SymbolCUSIP"] == "BFRIX"]
if not bfrix_row.empty:
    print(f"\nCWA_Broad_Category_Name for BFRIX after normalization: {bfrix_row['CWA_Broad_Category_Name'].values[0]}")

# Define categories for risk management overlays
categories = ["Slight/None", "Moderate", "Persistent Systematic", "Heavy Amplification"]
category_mapping = {
    "Slight/None": "slight_none",
    "Moderate": "moderate",
    "Persistent Systematic": "persistent_systematic",
    "Heavy Amplification": "heavy_amplification"
}
# Reverse mapping for final classification
reverse_category_mapping = {v: k for k, v in category_mapping.items()}

# Initialize scoring columns
for cat in categories:
    db_cat = category_mapping[cat]
    score_col = f"score_{db_cat}"
    funds_df[score_col] = 0.0

# Initialize intermediate score columns
for cat in categories:
    db_cat = category_mapping[cat]
    funds_df[f"keyword_score_{db_cat}"] = 0.0
    funds_df[f"meaningful_score_{db_cat}"] = 0.0
    funds_df[f"assist_score_{db_cat}"] = 0.0
    funds_df[f"boolean_score_{db_cat}"] = 0.0
    funds_df[f"matched_keywords_{db_cat}"] = ""

# Function to count keywords and return matched keywords
def count_keywords(text, keyword_list):
    if pd.isna(text):
        return 0, ""
    text = text.lower()
    matches = [keyword for keyword in keyword_list if re.search(r'\b' + re.escape(keyword) + r'\b', text)]
    count = len(matches)
    matched_text = "; ".join(matches) if matches else ""
    return count, matched_text

# Apply keyword scoring with multipliers and debugging
text_columns = ["ProductName", "investment_strategy", "FS_insight"]
for cat, kw_list in keywords.items():
    db_cat = category_mapping[cat]
    score_col = f"keyword_score_{db_cat}"
    matched_col = f"matched_keywords_{db_cat}"
    multiplier = keyword_multipliers[cat]
    for text_col in text_columns:
        counts_and_matches = funds_df[text_col].apply(lambda x: count_keywords(x, kw_list))
        increment = counts_and_matches.apply(lambda x: x[0] * multiplier)
        funds_df[score_col] += increment
        matched_increment = counts_and_matches.apply(lambda x: x[1])
        funds_df[matched_col] = funds_df[matched_col] + "; " + matched_increment
        # Debugging: Check for non-zero increments for Heavy Amplification
        if cat == "Heavy Amplification":
            non_zero_increments = funds_df[(increment > 0) & (funds_df["SymbolCUSIP"] == "BFRIX")][["SymbolCUSIP", text_col, matched_col]]
            if not non_zero_increments.empty:
                print(f"\nNon-zero keyword increments for {cat} in {text_col}:")
                print(non_zero_increments)
    funds_df[matched_col] = funds_df[matched_col].str.replace(r'\s*;\s*;\s*', '; ', regex=True).str.strip('; ')

# Debug: Check CWA_Broad_Category_Name for BFRIX after keyword scoring
bfrix_row = funds_df[funds_df["SymbolCUSIP"] == "BFRIX"]
if not bfrix_row.empty:
    print(f"\nCWA_Broad_Category_Name for BFRIX after keyword scoring: {bfrix_row['CWA_Broad_Category_Name'].values[0]}")

# Apply meaningful category rules
for cat, mappings in meaningful_categories.items():
    db_cat = category_mapping[cat]
    score_col = f"meaningful_score_{db_cat}"
    for map_type, values in mappings.items():
        col_name = {
            "CWA_Broad_Category": "CWA_Broad_Category_Name",
            "YC_Category": "Category_Name",
            "YC_Global_Category": "Global_Category_Name"
        }[map_type]
        funds_df.loc[funds_df[col_name].isin(values), score_col] += 10

# Apply assist category rules with debugging
for rule in assist_categories:
    cat_type = rule["cat_type"]
    col_name = {
        "CWA_Broad_Category": "CWA_Broad_Category_Name",
        "YC_Category": "Category_Name",
        "YC_Global_Category": "Global_Category_Name",
        "YC_Broad_Asset_Class": "YC_Broad_Asset_Class_Name"
    }[cat_type]
    actions = rule["actions"]
    exact_value = rule["exact"]
    mask = funds_df[col_name].fillna("").str.contains(exact_value, case=False, na=False)

    if "remove" in actions:
        for remove_cat in actions["remove"]:
            db_remove_cat = category_mapping[remove_cat]
            score_col = f"assist_score_{db_remove_cat}"
            funds_df.loc[mask, score_col] -= 5
    if "boost" in actions:
        for boost_cat in actions["boost"]:
            db_boost_cat = category_mapping[boost_cat]
            score_col = f"assist_score_{db_boost_cat}"
            funds_df.loc[mask, score_col] += 5
            if boost_cat == "Heavy Amplification" and mask.any():
                print(f"Assist rule boosted {boost_cat} for {sum(mask)} funds with {exact_value} in {cat_type}")

# Debug: Check CWA_Broad_Category_Name for BFRIX after assist rules
bfrix_row = funds_df[funds_df["SymbolCUSIP"] == "BFRIX"]
if not bfrix_row.empty:
    print(f"\nCWA_Broad_Category_Name for BFRIX after assist rules: {bfrix_row['CWA_Broad_Category_Name'].values[0]}")

# Apply Boolean rules with refinements
combined_text = funds_df[text_columns].fillna("").apply(lambda row: " ".join(row), axis=1).str.lower()

# Rule 1: Auto-classify funds with explicit leverage terms as Heavy Amplification
amplification_terms = ["2x", "3x", "double exposure", "triple exposure", "geared"]
mask_explicit_amplification = combined_text.str.contains("|".join(amplification_terms), na=False) & (funds_df["leveraged_fund"] == 1)
funds_df.loc[mask_explicit_amplification, "boolean_score_heavy_amplification"] += 50
funds_df.loc[mask_explicit_amplification, "boolean_score_slight_none"] = -float('inf')
funds_df.loc[mask_explicit_amplification, "boolean_score_moderate"] = -float('inf')
funds_df.loc[mask_explicit_amplification, "boolean_score_persistent_systematic"] -= 20

# Rule 2: Leveraged funds with amplification keywords
mask_leveraged_amplify = (
    (funds_df["leveraged_fund"] == 1) &
    (funds_df["keyword_score_heavy_amplification"] > 0) &
    (combined_text.str.contains("leveraged|2x|3x|amplified|geared", na=False))
)
funds_df.loc[mask_leveraged_amplify, "boolean_score_heavy_amplification"] += 30
funds_df.loc[mask_leveraged_amplify, "boolean_score_slight_none"] = -float('inf')
funds_df.loc[mask_leveraged_amplify, "boolean_score_moderate"] = -float('inf')
funds_df.loc[mask_leveraged_amplify, "boolean_score_persistent_systematic"] -= 10

# Rule 3: Leveraged funds without amplification keywords
mask_leveraged_no_amplify = (
    (funds_df["leveraged_fund"] == 1) &
    (funds_df["keyword_score_heavy_amplification"] == 0) &
    (funds_df["keyword_score_persistent_systematic"] > 0)
)
funds_df.loc[mask_leveraged_no_amplify, "boolean_score_persistent_systematic"] += 15
funds_df.loc[mask_leveraged_no_amplify, "boolean_score_slight_none"] = -float('inf')
funds_df.loc[mask_leveraged_no_amplify, "boolean_score_moderate"] -= 5

# Rule 4: Boost Moderate for covered calls and light hedging
mask_moderate_hedging = (
    ((funds_df["keyword_score_moderate"] > 0) |
     (funds_df["currency_hedged_fund"] == 1) |
     ((funds_df[["stock_short", "bond_short", "cash_short", "other_short"]].max(axis=1) > 1) &
      (funds_df[["stock_short", "bond_short", "cash_short", "other_short"]].max(axis=1) <= 5)))
)
funds_df.loc[mask_moderate_hedging, "boolean_score_moderate"] += 20
funds_df.loc[mask_moderate_hedging, "boolean_score_slight_none"] -= 10

# Rule 5: Boost Persistent Systematic for trend-following, CTA, or systematic strategies
mask_systematic = (
    (funds_df["keyword_score_persistent_systematic"] > 0) &
    (combined_text.str.contains("trend-following|trend strategy|CTA|managed futures|systematic", na=False))
) | (
    (funds_df[["stock_short", "bond_short", "cash_short", "other_short"]].max(axis=1) > 5) &
    (funds_df["keyword_score_persistent_systematic"] > 0)
)
funds_df.loc[mask_systematic, "boolean_score_persistent_systematic"] += 20
funds_df.loc[mask_systematic, "boolean_score_slight_none"] = -float('inf')
funds_df.loc[mask_systematic, "boolean_score_moderate"] -= 10

# Rule 6: Inverse or synthetic funds boost Persistent Systematic
mask_inverse = funds_df["inverse_fund"] == 1
mask_synthetic = funds_df["synthetic_replication_fund"] == 1
funds_df.loc[mask_inverse | mask_synthetic, "boolean_score_persistent_systematic"] += 10
funds_df.loc[mask_inverse | mask_synthetic, "boolean_score_slight_none"] = -float('inf')
funds_df.loc[mask_inverse | mask_synthetic, "boolean_score_moderate"] -= 5

# Rule 7: Boost Slight/None for minimal risk (strengthened)
mask_no_risk = (
    (funds_df["inverse_fund"] == 0) &
    (funds_df["leveraged_fund"] == 0) &
    (funds_df["synthetic_replication_fund"] == 0) &
    (funds_df["currency_hedged_fund"] == 0) &
    (funds_df["stock_short"] <= 1) &
    (funds_df["bond_short"] <= 1) &
    (funds_df["cash_short"] <= 1) &
    (funds_df["other_short"] <= 1) &
    (funds_df["stock_long"] <= 100) &
    (funds_df["bond_long"] <= 100) &
    (funds_df["cash_long"] <= 100) &
    (funds_df["other_long"] <= 100)
)
funds_df.loc[mask_no_risk, "boolean_score_slight_none"] += 10

# Rule 8: Boost Persistent Systematic for balanced long/short (market neutrality)
mask_market_neutral = (
    (funds_df["stock_long"] > 10) &
    (funds_df["stock_short"] > 10) &
    (abs(funds_df["stock_net"]) < 20)
)
funds_df.loc[mask_market_neutral, "boolean_score_persistent_systematic"] += 10
funds_df.loc[mask_market_neutral, "boolean_score_slight_none"] = -float('inf')
funds_df.loc[mask_market_neutral, "boolean_score_moderate"] -= 5

# Rule 9: Penalize Heavy Amplification for Bank Loan funds without clear leverage
mask_bank_loan_no_leverage = (
    (funds_df["YC_Category"].str.contains("Bank Loan", na=False)) &
    (funds_df["leveraged_fund"] == 0) &
    (~combined_text.str.contains("2x|3x|amplified|geared", na=False))
)
funds_df.loc[mask_bank_loan_no_leverage, "boolean_score_heavy_amplification"] -= 10
funds_df.loc[mask_bank_loan_no_leverage, "boolean_score_moderate"] += 5

# Debug: Check CWA_Broad_Category_Name for BFRIX before final scoring
bfrix_row = funds_df[funds_df["SymbolCUSIP"] == "BFRIX"]
if not bfrix_row.empty:
    print(f"\nCWA_Broad_Category_Name for BFRIX before final scoring: {bfrix_row['CWA_Broad_Category_Name'].values[0]}")

# Sum all intermediate scores into final scores with debugging
for cat in categories:
    db_cat = category_mapping[cat]
    score_col = f"score_{db_cat}"
    funds_df[score_col] = (
        funds_df[f"keyword_score_{db_cat}"] +
        funds_df[f"meaningful_score_{db_cat}"] +
        funds_df[f"assist_score_{db_cat}"] +
        funds_df[f"boolean_score_{db_cat}"]
    )
    # Debugging for BFRIX
    if cat == "Heavy Amplification":
        bflix_row = funds_df[funds_df["SymbolCUSIP"] == "BFRIX"]
        if not bflix_row.empty:
            print(f"\nBFRIX Heavy Amplification Score Breakdown:")
            print(f"Keyword Score: {bflix_row[f'keyword_score_{db_cat}'].values[0]}")
            print(f"Meaningful Score: {bflix_row[f'meaningful_score_{db_cat}'].values[0]}")
            print(f"Assist Score: {bflix_row[f'assist_score_{db_cat}'].values[0]}")
            print(f"Boolean Score: {bflix_row[f'boolean_score_{db_cat}'].values[0]}")
            print(f"Total Score: {bflix_row[f'score_{db_cat}'].values[0]}")

# Debug: Check CWA_Broad_Category_Name for BFRIX after final scoring
bfrix_row = funds_df[funds_df["SymbolCUSIP"] == "BFRIX"]
if not bfrix_row.empty:
    print(f"\nCWA_Broad_Category_Name for BFRIX after final scoring: {bfrix_row['CWA_Broad_Category_Name'].values[0]}")

# Determine final overlay category with corrected mapping
score_columns = [f"score_{category_mapping[cat]}" for cat in categories]
# Get the column name with the highest score
max_score_col = funds_df[score_columns].idxmax(axis=1)
# Map back to human-readable category using reverse mapping
funds_df["Overlay_Category"] = max_score_col.apply(
    lambda x: reverse_category_mapping.get(x.replace("score_", ""), "Slight/None") if pd.notnull(x) else "Slight/None"
)

# Apply tiebreaker: if scores are tied, prefer category with higher keyword score
for idx in funds_df.index:
    scores = funds_df.loc[idx, score_columns]
    max_score = scores.max()
    tied_categories = [col for col, score in scores.items() if score == max_score]
    if len(tied_categories) > 1:
        keyword_scores = {col: funds_df.loc[idx, f"keyword_score_{col.replace('score_', '')}"] for col in tied_categories}
        max_keyword_score = max(keyword_scores.values())
        best_tied_category = max(keyword_scores, key=keyword_scores.get)
        funds_df.loc[idx, "Overlay_Category"] = reverse_category_mapping.get(best_tied_category.replace("score_", ""), "Slight/None")

# Debug: Check CWA_Broad_Category_Name for BFRIX before export
bfrix_row = funds_df[funds_df["SymbolCUSIP"] == "BFRIX"]
if not bfrix_row.empty:
    print(f"\nCWA_Broad_Category_Name for BFRIX before export: {bfrix_row['CWA_Broad_Category_Name'].values[0]}")

# Validation and Debugging Output
print("Classification Distribution:")
print(funds_df["Overlay_Category"].value_counts())

# Detailed scores for top categories
funds_df["Top_Score_Category"] = funds_df[score_columns].idxmax(axis=1).apply(lambda x: reverse_category_mapping.get(x.replace("score_", ""), "Slight/None"))
funds_df["Top_Score_Value"] = funds_df[score_columns].max(axis=1)
print("\nSample of funds with scores:")
print(funds_df[["SymbolCUSIP", "ProductName", "Overlay_Category", "Top_Score_Category", "Top_Score_Value"] + score_columns].head(10))

# Flag potential misclassifications with stricter criteria
potential_misclass = funds_df[
    (funds_df["Overlay_Category"] == "Slight/None") &
    (
        (funds_df["keyword_score_moderate"] > 0) |
        (funds_df["keyword_score_persistent_systematic"] > 0) |
        (funds_df["keyword_score_heavy_amplification"] > 0) |
        (funds_df[["stock_short", "bond_short", "cash_short", "other_short"]].max(axis=1) > 5) |
        (funds_df["currency_hedged_fund"] == 1 & funds_df["keyword_score_moderate"] > 0)
    )
]
print(f"\nFunds classified as Slight/None but with possible mismatch ({len(potential_misclass)}):")
if not potential_misclass.empty:
    print(potential_misclass[["SymbolCUSIP", "ProductName", "Overlay_Category", "keyword_score_moderate", 
                              "keyword_score_persistent_systematic", "keyword_score_heavy_amplification"]].head())

# Export to Excel with error handling
if write_to_excel:
    priority_columns = [
        "SymbolCUSIP", "ProductName", "fund_family", "Overlay_Category", "ycharts_url"
    ]
    remaining_columns = [
        col for col in (
            [f"score_{category_mapping[cat]}" for cat in categories] +
            [f"keyword_score_{category_mapping[cat]}" for cat in categories] +
            [f"meaningful_score_{category_mapping[cat]}" for cat in categories] +
            [f"assist_score_{category_mapping[cat]}" for cat in categories] +
            [f"boolean_score_{category_mapping[cat]}" for cat in categories] +
            [f"matched_keywords_{category_mapping[cat]}" for cat in categories] +
            ["CWA_Broad_Category_Name", "Category_Name", "Global_Category_Name", "YC_Broad_Asset_Class_Name"] +
            ["index_fund", "inverse_fund", "leveraged_fund", "socially_responsible_fund", "synthetic_replication_fund", "fund_of_funds", "currency_hedged_fund"] +
            ["stock_net", "stock_long", "stock_short", "bond_net", "bond_long", "bond_short", "cash_net", "cash_long", "cash_short", "other_net", "other_long", "other_short"]
        ) if col in funds_df.columns and col not in priority_columns
    ]
    output_columns = priority_columns + remaining_columns
    # Debug: Print CWA_Broad_Category_Name column before export
    print("\nCWA_Broad_Category_Name column before export (first few rows):")
    print(funds_df[["SymbolCUSIP", "CWA_Broad_Category_Name"]].head())
    try:
        funds_df[output_columns].to_excel(output_path, index=False)
        print(f"\nResults exported to {output_path}")
    except PermissionError as e:
        print(f"\nError: Unable to write to {output_path}. Please ensure the file is closed and try again. ({e})")
else:
    print("\nExcel output skipped (write_to_excel=False).")


BFRIX CWA_Broad_Category_ID before merge: 24.0
CWA_Broad_Category_List contents:
    ID CWA_Broad_Category_Name
0    1              Allocation
1    2             Alternative
2    3           Bond Strategy
3    4        Cash Alternative
4    5               Commodity
5    6                 Country
6    7                Currency
7    8         Defined Outcome
8    9                Delisted
9   10           Digital Asset
10  11                Emerging
11  12             Global Bond
12  13           Global Equity
13  14           International
14  15           Miscellaneous
15  16               Municipal
16  17          Nontraditional
17  18   Quantitative/Tactical
18  19                Regional
19  20         Sector/Industry
20  21            Single Stock
21  22               Specialty
22  23               Strategic
23  25             Target Date
24  26         Target Maturity
25  24    Taxable Fixed Income
26  27        Trading/Tactical
27  28               US Equity

Unmatched CWA_Broa

KeyError: 'YC_Category'

In [42]:
# New grok risk overlays code

import pandas as pd
from sqlalchemy import create_engine
import re
import os

# Connection string
connection_string = (
    "mssql+pyodbc://JULIANS_LAPTOP\\SQLEXPRESS/"
    "CWA_Fund_Database?driver=ODBC+Driver+18+for+SQL+Server"
    "&trusted_connection=yes&TrustServerCertificate=yes"
)

# Define keyword mappings
keyword_mappings = {
    "Slight/None": ["long-only", "no derivatives", "no hedging", "no leverage", "no options", "no short",
                    "position adjustment", "occasional hedging", "covered call", "put-write", "light hedge",
                    "may include options", "limited use of derivatives", "for risk management purposes",
                    "minor hedging", "occasional short positions", "overwrite"],
    "Moderate": ["hedged", "currency hedge", "protective put", "buffer", "partial hedge", "hedged equity",
                 "covered call", "convexity option overlay", "option overlay", "put/spread collar",
                 "forward agreement", "enhanced index strategy", "BuyWrite", "Buy-Write", "buy write",
                 "option spread", "volatility hedge", "put options", "enhance", "options-based income",
                 "ELN", "premium income", "call option", "FLEX options", "option premium", "write calls",
                 "sell calls", "protective puts", "equity-linked notes", "structured notes", "risk mitigation",
                 "downside protection", "limited hedging", "multi-asset"],
    "Persistent Systematic": ["tail-risk", "trend-following", "systematic hedging", "overlay", "CTA",
                              "managed futures", "defined outcome", "long-short", "market neutral",
                              "systematic strategy", "return stacking", "option writing", "straddle",
                              "derivative income", "futures contracts", "swap contract", "forward agreement",
                              "enhanced index strategy", "volatility hedge", "put options", "options-based income",
                              "ELN", "option premium", "swap", "forward", "futures", "future", "VIX",
                              "managed futures strategy", "trend strategy", "quantitative hedging",
                              "systematic options", "options overlay strategy", "futures overlay", "swaps-based",
                              "multi-asset", "Flex Options", "Flexible Exchange Options", "YieldMax",
                              "inverse", "synthetic"],
    "Heavy Amplification": ["2x", "3x", "Uncapped Accelerator", "-2x", "-3x", "YieldMax",
                            "inverse", "synthetic"]
}

# Direct mapping keywords (often in ProductName)
direct_keyword_mappings = {
    "Moderate": ["Covered Call", "Interest Rate Hedged", "Hedged Equity", "Currency Hedged"],
    "Persistent Systematic": ["Market Neutral", "managed futures", "Premia", "Return Stacked ETFs"]
}

# Direct mapping categories
direct_category_mappings = {
    "Persistent Systematic": {
        "YC_Category": ["Defined Outcome"],
        "CWA_Broad_Category": ["Defined Outcome"],
        "YC_Global_Category": ["market neutral"]
    },
    "Heavy Amplification": {
        "YC_Category": ["Trading--Leveraged Equity", "Trading--Leveraged Debt", "Trading--Leveraged Commodities"],
        "CWA_Broad_Category": ["Single Stock"]
    }
}

# Helper category mappings (narrow possibilities)
helper_category_mappings = {
    ("Persistent Systematic", "Heavy Amplification"): {
        "YC_Category": ["Trading--Inverse Commodities", "Trading--Inverse Debt", "Trading--Inverse Equity", "Trading--Miscellaneous"],
        "CWA_Broad_Category": ["Trading/Tactical"],
        "YC_Global_Category": ["Trading Tools"]
    },
    ("Persistent Systematic", "Moderate"): {
        "YC_Global_Category": ["Multialternative", "Long/Short Equity"],
        "YC_Category": ["Equity Hedged"]
    },
    ("Moderate",): {
        "YC_Category": ["Derivative Income"]
    }
}

# Weak helper category mappings (may or may not have overlay)
weak_helper_category_mappings = {
    ("Slight/None", "Moderate"): {
        "YC_Global_Category": ["Flexible Allocation", "Alternative Miscellaneous"],
        "return_driver": ["Index Based", "Factor/Smart Beta"]
    },
    ("Slight/None", "Moderate", "Persistent Systematic"): {
        "YC_Category": ["Relative Value Arbitrage"]
    },
    ("Heavy Amplification", "Persistent Systematic", "Moderate", "Slight/None"): {
        "return_driver": ["Quant/Systematic"]
    },
    ("Persistent Systematic", "Moderate", "Slight/None"): {
        "return_driver": ["Active Discretionary", "Multi-Strategy"]
    }
}

# Boolean flags to remove categories
boolean_exclusions = {
    ("Slight/None", "Moderate"): lambda row: row['leveraged_fund'] in [True, 1] or \
                                            row['synthetic_replication_fund'] in [True, 1] or \
                                            row['inverse_fund'] in [True, 1]
}

# Function to calculate exposure-based classification
def exposure_based_classification(row):
    # Convert exposure columns to numeric, handle missing columns gracefully
    exposure_cols = ['cash_long', 'cash_net', 'cash_short', 'stock_long', 'stock_net', 'stock_short',
                    'bond_long', 'bond_net', 'bond_short', 'other_long', 'other_net', 'other_short']
    missing_cols = []
    for col in exposure_cols:
        try:
            value = pd.to_numeric(row[col], errors='coerce')
            row[col] = 0 if pd.isna(value) else value
        except KeyError:
            missing_cols.append(col)
            row[col] = 0  # Set missing column to 0
    
    # If there are missing columns, note them in the audit log
    debug_msg = ""
    if missing_cols:
        debug_msg += f"Missing columns in exposure data: {missing_cols}. Set to 0. "
    
    # Calculate totals
    long_total = row['cash_long'] + row['stock_long'] + row['bond_long'] + row['other_long']
    short_total = row['cash_short'] + row['stock_short'] + row['bond_short'] + row['other_short']

    # Use a tolerance-based comparison instead of strict rounding
    epsilon = 0.001

    # Debug log for exact values
    debug_msg += (f"Exposure Check - long_total: {long_total}, "
                 f"short_total: {short_total}")

    # Rule: Long exposures between 99.5% and 100% (allowing for small floating-point differences) and no shorts -> "Slight/None"
    if (99.5 - epsilon <= long_total <= 100 + epsilon) and abs(short_total) <= epsilon:
        return "Slight/None", f"{debug_msg} -> Long exposures effectively between 99.5% and 100% and no shorts"

    # Existing rules
    if (row['cash_long'] >= 98 and row['stock_long'] == 0 and row['bond_long'] == 0 and 
        row['cash_short'] == 0 and row['stock_short'] == 0 and row['bond_short'] == 0):
        return "Slight/None", "Pure long exposure"
    
    if (row['cash_short'] > 0 and row['cash_long'] < 1 and 
        row['stock_long'] <= 1 and row['bond_long'] <= 1 and row['other_long'] <= 1):
        return "Slight/None", "Minimal short exposure"
    
    if (row['cash_long'] > 50 and row['cash_long'] < 100 and 
        (row['stock_long'] > 0 or row['bond_long'] > 0) and 
        row['cash_short'] == 0 and row['stock_short'] == 0 and row['bond_short'] == 0):
        return "Moderate", "Possible futures usage"
    
    return None, "No clear exposure pattern"

# Function to search text for keywords
def search_keywords(text, keywords):
    if pd.isna(text):
        return 0
    text = str(text).lower()
    return sum(1 for keyword in keywords if re.search(r'\b' + re.escape(keyword.lower()) + r'\b', text))

# Main classification function
def classify_fund(row):
    possible_categories = ["Slight/None", "Moderate", "Persistent Systematic", "Heavy Amplification"]
    scores = {cat: 0 for cat in possible_categories}
    audit_log = []
    any_category_match = False

    # Step 1: Override for Fund Family "Return Stacked ETFs"
    if pd.notna(row['fund_family']) and "return stacked etfs" in row['fund_family'].lower():
        audit_log.append("Override: Fund family 'Return Stacked ETFs' -> Persistent Systematic")
        return "Persistent Systematic", audit_log

    # Step 2: Direct keyword mapping
    for category, keywords in direct_keyword_mappings.items():
        for field in ['ProductName', 'investment_strategy', 'FS_insight']:
            if pd.notna(row[field]):
                match_count = search_keywords(row[field], keywords)
                if match_count > 0:
                    audit_log.append(f"Direct keyword match for {category} in {field}: {match_count} keywords")
                    return category, audit_log

    # Step 3: Direct category mapping
    for category, mappings in direct_category_mappings.items():
        for db_field, values in mappings.items():
            values_lower = [val.lower() for val in values]
            if db_field == "YC_Category" and pd.notna(row['YC_Category_Name']) and row['YC_Category_Name'].lower() in values_lower:
                audit_log.append(f"Direct category match for {category} in YC_Category_Name: {row['YC_Category_Name']}")
                return category, audit_log
            if db_field == "CWA_Broad_Category" and pd.notna(row['CWA_Broad_Category_Name']) and row['CWA_Broad_Category_Name'].lower() in values_lower:
                audit_log.append(f"Direct category match for {category} in CWA_Broad_Category_Name: {row['CWA_Broad_Category_Name']}")
                return category, audit_log
            if db_field == "YC_Global_Category" and pd.notna(row['Global_Category_Name']) and row['Global_Category_Name'].lower() in values_lower:
                audit_log.append(f"Direct category match for {category} in Global_Category_Name: {row['Global_Category_Name']}")
                return category, audit_log

    # Step 4: Apply Boolean exclusions
    for categories, condition in boolean_exclusions.items():
        if condition(row):
            excluded = [cat for cat in possible_categories if cat in categories]
            for cat in excluded:
                possible_categories.remove(cat)
            audit_log.append(f"Excluded {excluded} due to Boolean flags")
            if len(possible_categories) == 1:
                return possible_categories[0], audit_log

    # Step 5: Exposure-based classification
    exposure_class, exposure_reason = exposure_based_classification(row)
    if exposure_class:
        audit_log.append(f"Exposure-based classification: {exposure_class} - {exposure_reason}")
        return exposure_class, audit_log

    # Step 6: Additional exposure-based rules
    # Convert exposure columns to numeric for additional checks
    exposure_cols = ['cash_long', 'cash_net', 'cash_short', 'stock_long', 'stock_net', 'stock_short',
                    'bond_long', 'bond_net', 'bond_short', 'other_long', 'other_net', 'other_short']
    for col in exposure_cols:
        try:
            value = pd.to_numeric(row[col], errors='coerce')
            row[col] = 0 if pd.isna(value) else value
        except KeyError:
            row[col] = 0  # Set missing column to 0

    # Rule: Stocks > 100% cannot be "Slight/None"
    if row['stock_long'] > 100:
        if "Slight/None" in possible_categories:
            possible_categories.remove("Slight/None")
            audit_log.append("Excluded 'Slight/None' due to stock_long > 100%")
    # Rule: Stocks > 200% boosts Systematic, Aggressive, and Moderate
    if row['stock_long'] > 200:
        scores["Persistent Systematic"] += 2
        scores["Heavy Amplification"] += 2
        scores["Moderate"] += 1
        audit_log.append("Boosted 'Persistent Systematic' and 'Heavy Amplification' by +2, 'Moderate' by +1 due to stock_long > 200%")

    # Rule: Short or "other" exposures > 20% drops "Moderate" and "Slight/None"
    short_total = row['cash_short'] + row['stock_short'] + row['bond_short'] + row['other_short']
    other_total = row['other_long'] + row['other_short']
    if short_total > 20 or other_total > 20:
        for cat in ["Moderate", "Slight/None"]:
            if cat in possible_categories:
                possible_categories.remove(cat)
        audit_log.append(f"Excluded 'Moderate' and 'Slight/None' due to short_total ({short_total}%) or other_total ({other_total}%) > 20%")
        if len(possible_categories) == 1:
            return possible_categories[0], audit_log

    # Rule: High long exposure (>150%), high short exposure (>40%), high cash_short (>50%)
    if row['stock_long'] > 150 and short_total > 40 and row['cash_short'] > 50:
        for cat in ["Moderate", "Slight/None"]:
            if cat in possible_categories:
                possible_categories.remove(cat)
        scores["Persistent Systematic"] += 2
        scores["Heavy Amplification"] += 2
        audit_log.append(f"Excluded 'Moderate' and 'Slight/None', boosted 'Persistent Systematic' and 'Heavy Amplification' by +2 due to stock_long ({row['stock_long']}%) > 150, short_total ({short_total}%) > 40, and cash_short ({row['cash_short']}%) > 50")
        if len(possible_categories) == 1:
            return possible_categories[0], audit_log

    # Rule: If inverse_fund=True and short_total > 20, boost Heavy Amplification
    if row['inverse_fund'] in [True, 1] and short_total > 20:
        scores["Heavy Amplification"] += 1
        audit_log.append("Boosted 'Heavy Amplification' by +1 due to inverse_fund=True and short_total > 20")

    # Step 7: Rule for YC_BM_Symbol '^PEATR'
    if pd.notna(row['YC_BM_Symbol']) and row['YC_BM_Symbol'] == '^PEATR':
        for cat in ["Moderate", "Persistent Systematic", "Heavy Amplification"]:
            scores[cat] += 1
        audit_log.append("Boosted 'Moderate', 'Persistent Systematic', and 'Heavy Amplification' by +1 due to YC_BM_Symbol '^PEATR'")

    # Step 8: Rule for YC_Global_Category_Name "Options Trading"
    if pd.notna(row['Global_Category_Name']) and row['Global_Category_Name'].lower() == "options trading":
        scores["Moderate"] += 1
        scores["Persistent Systematic"] += 2
        scores["Heavy Amplification"] += 2
        audit_log.append("Boosted 'Moderate' by +1, 'Persistent Systematic' and 'Heavy Amplification' by +2 due to YC_Global_Category_Name 'Options Trading'")
        any_category_match = True

    # Step 9: Score based on helper categories and keywords
    # Helper categories
    for categories, mappings in helper_category_mappings.items():
        for db_field, values in mappings.items():
            values_lower = [val.lower() for val in values]
            if db_field == "YC_Category" and pd.notna(row['YC_Category_Name']) and row['YC_Category_Name'].lower() in values_lower:
                for cat in categories:
                    scores[cat] += 2
                audit_log.append(f"Helper category match for {categories} in YC_Category_Name: {row['YC_Category_Name']}")
                any_category_match = True
            if db_field == "CWA_Broad_Category" and pd.notna(row['CWA_Broad_Category_Name']) and row['CWA_Broad_Category_Name'].lower() in values_lower:
                for cat in categories:
                    scores[cat] += 2
                audit_log.append(f"Helper category match for {categories} in CWA_Broad_Category_Name: {row['CWA_Broad_Category_Name']}")
                any_category_match = True
            if db_field == "YC_Global_Category" and pd.notna(row['Global_Category_Name']) and row['Global_Category_Name'].lower() in values_lower:
                for cat in categories:
                    scores[cat] += 2
                audit_log.append(f"Helper category match for {categories} in Global_Category_Name: {row['Global_Category_Name']}")
                any_category_match = True

    # Weak helper categories
    for categories, mappings in weak_helper_category_mappings.items():
        for db_field, values in mappings.items():
            values_lower = [val.lower() for val in values]
            if db_field == "YC_Global_Category" and pd.notna(row['Global_Category_Name']) and row['Global_Category_Name'].lower() in values_lower:
                for cat in categories:
                    scores[cat] += 1
                audit_log.append(f"Weak helper category match for {categories} in Global_Category_Name: {row['Global_Category_Name']}")
                any_category_match = True
            if db_field == "YC_Category" and pd.notna(row['YC_Category_Name']) and row['YC_Category_Name'].lower() in values_lower:
                for cat in categories:
                    scores[cat] += 1
                audit_log.append(f"Weak helper category match for {categories} in YC_Category_Name: {row['YC_Category_Name']}")
                any_category_match = True
            if db_field == "return_driver" and pd.notna(row['return_driver']) and row['return_driver'].lower() in values_lower:
                for cat in categories:
                    scores[cat] += 1
                audit_log.append(f"Weak helper category match for {categories} in return_driver: {row['return_driver']}")
                any_category_match = True

    # Step 10: Handle non-category matches
    # Include "Nontraditional" in CWA_Broad_Category as a match for top-tier exclusion
    if pd.notna(row['CWA_Broad_Category_Name']) and row['CWA_Broad_Category_Name'].lower() == "nontraditional":
        any_category_match = True
        audit_log.append("CWA_Broad_Category 'Nontraditional' counts as a category match for top-tier exclusion")

    if not any_category_match:
        audit_log.append("No category matches found. Excluding 'Heavy Amplification' and 'Persistent Systematic'.")
        possible_categories = [cat for cat in possible_categories if cat not in ["Heavy Amplification", "Persistent Systematic"]]
        scores["Slight/None"] += 2
        scores["Moderate"] += 1
        audit_log.append("Boosted 'Slight/None' score by +2 and 'Moderate' score by +1 due to no category matches.")

    # Keyword scoring with increased points for Heavy Amplification
    for category, keywords in keyword_mappings.items():
        keyword_score = 0
        for field in ['ProductName', 'investment_strategy', 'FS_insight']:
            keyword_score += search_keywords(row[field], keywords)
        # Increase score multiplier for "Heavy Amplification" keywords to 3 points per match
        if category == "Heavy Amplification":
            scores[category] += keyword_score * 3
            if keyword_score > 0:
                audit_log.append(f"Keyword score for {category}: {keyword_score} matches x 3 = {keyword_score * 3}")
        else:
            scores[category] += keyword_score
            if keyword_score > 0:
                audit_log.append(f"Keyword score for {category}: {keyword_score}")

    # Determine final classification
    max_score = max(scores[cat] for cat in possible_categories)
    top_categories = [cat for cat in possible_categories if scores[cat] == max_score]
    if len(top_categories) == 1:
        final_classification = top_categories[0]
        audit_log.append(f"Final classification: {final_classification} with score {max_score}")
    else:
        # Primary tiebreaker: Use keyword scores
        keyword_tie_scores = {cat: 0 for cat in top_categories}
        for category in top_categories:
            for field in ['ProductName', 'investment_strategy', 'FS_insight']:
                keyword_tie_scores[category] += search_keywords(row[field], keyword_mappings[category])
        max_keyword_score = max(keyword_tie_scores.values())
        top_keyword_categories = [cat for cat, score in keyword_tie_scores.items() if score == max_keyword_score]
        
        if len(top_keyword_categories) == 1:
            final_classification = top_keyword_categories[0]
            audit_log.append(f"Tiebreaker used - Final classification: {final_classification} with tiebreaker keyword score {keyword_tie_scores[final_classification]}")
        else:
            # Secondary tiebreaker: Prefer more conservative category
            conservative_order = ["Slight/None", "Moderate", "Persistent Systematic", "Heavy Amplification"]
            final_classification = min(top_keyword_categories, key=lambda x: conservative_order.index(x))
            audit_log.append(f"Secondary tiebreaker used - Final classification: {final_classification} (most conservative among {top_keyword_categories})")

    return final_classification, audit_log

# Main script
def main():
    # Create database engine
    engine = create_engine(connection_string)

    # Query to join tables, fixed typo with 'stock_short'
    query = """
    SELECT 
        f.SymbolCUSIP, f.ProductName, f.fund_family, f.investment_strategy, f.FS_insight,
        f.index_fund, f.inverse_fund, f.leveraged_fund, f.synthetic_replication_fund,
        f.fund_of_funds, f.ycharts_url, f.currency_hedged_fund,
        f.cash_long, f.cash_net, f.cash_short, f.stock_long, f.stock_net, f.stock_short,
        f.bond_long, f.bond_net, f.bond_short, f.other_long, f.other_net, f.other_short,
        f.return_driver, f.YC_BM_Symbol,
        cwa.CWA_Broad_Category_Name,
        yc.Category_Name AS YC_Category_Name,
        ycg.Global_Category_Name,
        ycba.YC_Broad_Asset_Class_Name
    FROM Funds_to_Screen f
    LEFT JOIN CWA_Broad_Category_List cwa ON f.CWA_Broad_Category_ID = cwa.ID
    LEFT JOIN YC_Category_List yc ON f.YC_Category_ID = yc.ID
    LEFT JOIN YC_Global_Category_List ycg ON f.YC_Global_Category_ID = ycg.ID
    LEFT JOIN YC_Broad_Asset_Class_List ycba ON f.YC_Broad_Asset_Class_ID = ycba.ID
    """

    # Load data into DataFrame
    df = pd.read_sql(query, engine)

    # Apply classification
    results = []
    for idx, row in df.iterrows():
        classification, audit_log = classify_fund(row)
        result_row = row.copy()
        result_row['Final_Classification'] = classification
        result_row['Audit_Log'] = "; ".join(audit_log)
        results.append(result_row)

    # Convert results to DataFrame
    result_df = pd.DataFrame(results)

    # Ensure desired column order
    output_columns = ['SymbolCUSIP', 'ProductName', 'fund_family', 'Final_Classification', 'ycharts_url'] + \
                     [col for col in result_df.columns if col not in ['SymbolCUSIP', 'ProductName', 'fund_family', 'Final_Classification', 'ycharts_url']]

    result_df = result_df[output_columns]

    # Export to Excel
    output_path = r"C:\Users\JulianHeron\Software Projects\Risk_Overlays_V1.xlsx"
    result_df.to_excel(output_path, index=False)
    print(f"Results exported to {output_path}")

if __name__ == "__main__":
    main()

Results exported to C:\Users\JulianHeron\Software Projects\Risk_Overlays_V1.xlsx


In [50]:
# V3.2 with Grok. Using tiered decision tree and other logic.
import pandas as pd
from sqlalchemy import create_engine
import re
import os

# Connection string
connection_string = (
    "mssql+pyodbc://JULIANS_LAPTOP\\SQLEXPRESS/"
    "CWA_Fund_Database?driver=ODBC+Driver+18+for+SQL+Server"
    "&trusted_connection=yes&TrustServerCertificate=yes"
)

# Helper function to safely convert to lowercase string
def safe_lower(value):
    return value.lower() if isinstance(value, str) else ""

# Define keyword mappings for general scoring
keyword_mappings = {
    "Slight/None": ["long-only", "no derivatives", "no hedging", "no leverage", "no options", "no short", 
                    "position adjustment", "occasional hedging", "covered call", "put-write", "light hedge", 
                    "may include options", "limited use of derivatives", "for risk management purposes", 
                    "minor hedging", "occasional short positions", "overwrite", "investment grade", "core"],
    "Moderate": ["hedged", "currency hedge", "protective put", "partial hedge", "hedged equity", 
                 "covered call", "convexity option overlay", "option overlay", "put/spread collar", 
                 "forward agreement", "enhanced index strategy", "BuyWrite", "Buy-Write", "buy write", 
                 "option spread", "volatility hedge", "put options", "enhance", "options-based income", 
                 "ELN", "premium income", "call option", "FLEX options", "option premium", "write calls", 
                 "sell calls", "protective puts", "equity-linked notes", "structured notes", "risk mitigation", 
                 "downside protection", "limited hedging", "multi-asset"],
    "Persistent Systematic": ["tail-risk", "trend-following", "systematic hedging", "overlay", "CTA", 
                              "managed futures", "defined outcome", "long-short", "market neutral", 
                              "systematic strategy", "return stacking", "option writing", "straddle", 
                              "derivative income", "futures contracts", "swap contract", "forward agreement", 
                              "enhanced index strategy", "volatility hedge", "put options", "options-based income", 
                              "ELN", "option premium", "swap", "forward", "futures", "future", "VIX", 
                              "managed futures strategy", "trend strategy", "quantitative hedging", 
                              "systematic options", "options overlay strategy", "futures overlay", "swaps-based", 
                              "multi-asset", "Flex Options", "Flexible Exchange Options", "YieldMax", "buffer"],
    "Heavy Amplification": ["2x", "3x", "Uncapped Accelerator", "-2x", "-3x", "YieldMax"]
}

# Direct mapping keywords (often in ProductName)
direct_keyword_mappings = {
    "Persistent Systematic": ["Market Neutral", "managed futures", "Premia", "Return Stacked ETFs"]
}

# Direct mapping categories
direct_category_mappings = {
    "Persistent Systematic": {
        "YC_Category": ["Defined Outcome"],
        "CWA_Broad_Category": ["Defined Outcome"],
        "YC_Global_Category": ["market neutral"]
    },
    "Heavy Amplification": {
        "YC_Category": ["Trading--Leveraged Equity", "Trading--Leveraged Debt", "Trading--Leveraged Commodities"],
        "CWA_Broad_Category": ["Single Stock"]
    }
}

# Helper category mappings (narrow possibilities)
helper_category_mappings = {
    ("Persistent Systematic", "Heavy Amplification"): {
        "YC_Category": ["Trading--Inverse Commodities", "Trading--Inverse Debt", "Trading--Inverse Equity", "Trading--Miscellaneous"],
        "CWA_Broad_Category": ["Trading/Tactical"],
        "YC_Global_Category": ["Trading Tools"]
    },
    ("Persistent Systematic", "Moderate"): {
        "YC_Global_Category": ["Multialternative", "Long/Short Equity"],
        "YC_Category": ["Equity Hedged"]
    }
}

# Weak helper category mappings (may or may not have overlay)
weak_helper_category_mappings = {
    ("Slight/None", "Moderate"): {
        "YC_Global_Category": ["Flexible Allocation", "Alternative Miscellaneous"],
        "return_driver": ["Index Based", "Factor/Smart Beta"]
    },
    ("Slight/None", "Moderate", "Persistent Systematic"): {
        "YC_Category": ["Relative Value Arbitrage"]
    },
    ("Heavy Amplification", "Persistent Systematic", "Moderate", "Slight/None"): {
        "return_driver": ["Quant/Systematic"]
    },
    ("Persistent Systematic", "Moderate", "Slight/None"): {
        "return_driver": ["Active Discretionary", "Multi-Strategy"]
    }
}

# Boolean flags to remove categories
boolean_exclusions = {
    ("Slight/None", "Moderate"): lambda row: row['leveraged_fund'] in [True, 1] or \
                                            row['synthetic_replication_fund'] in [True, 1] or \
                                            row['inverse_fund'] in [True, 1]
}

# Function to search text for keywords
def search_keywords(text, keywords):
    if pd.isna(text):
        return 0
    text = str(text).lower()
    return sum(1 for keyword in keywords if re.search(r'\b' + re.escape(keyword.lower()) + r'\b', text))

# Decision Tree for Exposure Classification with Prove/Disprove
def classify_by_exposures_with_disproof(row):
    audit_log = []
    possible_categories = ["Slight/None", "Moderate", "Persistent Systematic", "Heavy Amplification"]
    scores = {cat: 0 for cat in possible_categories}

    # Convert exposure columns to numeric, handle non-numeric gracefully
    exposure_cols = ['cash_long', 'cash_net', 'cash_short', 'stock_long', 'stock_net', 'stock_short',
                    'bond_long', 'bond_net', 'bond_short', 'other_long', 'other_net', 'other_short']
    for col in exposure_cols:
        value = pd.to_numeric(row[col], errors='coerce')
        row[col] = 0 if pd.isna(value) else value

    # Calculate totals
    long_total = row['cash_long'] + row['stock_long'] + row['bond_long'] + row['other_long']
    short_total = row['cash_short'] + row['stock_short'] + row['bond_short'] + row['other_short']
    other_total = row['other_long'] + row['other_short']
    stock_long = row['stock_long']
    cash_short = row['cash_short']

    # Round totals to 4 decimal places for comparison
    long_total_rounded = round(long_total, 4)
    short_total_rounded = round(short_total, 4)
    other_total_rounded = round(other_total, 4)

    # Debug logging for exposure values
    debug_msg = (f"Exposure Values - long_total: {long_total}, long_total_rounded: {long_total_rounded}, "
                 f"short_total: {short_total}, short_total_rounded: {short_total_rounded}, "
                 f"other_total: {other_total}, other_total_rounded: {other_total_rounded}")
    audit_log.append(debug_msg)

    # Define disproof keywords for each branch
    disproof_keywords = {
        "Slight/None": ["derivatives", "swaps", "futures", "short", "hedge", "long-short", "inverse", "leveraged", "tail risk", "tail-risk"],
        "Moderate": ["systematic", "trend-following", "2x", "3x", "market neutral", "quantitative hedging", "managed futures",
                     "managed futures strategy", "trend strategy", "YieldMax", "tail risk", "tail-risk"],
        "Persistent Systematic": ["2x", "3x", "-2x", "-3x", "uncapped accelerator"],
        "Heavy Amplification": ["long-only", "no derivatives", "no short"]
    }

    # Helper function to check disproof keywords with detailed logging
    def has_disproof_keywords(category):
        keywords = disproof_keywords.get(category, [])
        keyword_score = 0
        disproof_details = []
        for field in ['ProductName', 'investment_strategy', 'FS_insight']:
            if pd.notna(row[field]):
                text = str(row[field]).lower()
                found_keywords = [keyword for keyword in keywords if re.search(r'\b' + re.escape(keyword.lower()) + r'\b', text)]
                if found_keywords:
                    disproof_details.append(f"Field '{field}' contains disproof keywords: {found_keywords}")
                    keyword_score += len(found_keywords)
        if keyword_score > 0:
            audit_log.append(f"Disproof Keywords ({category}): {disproof_details}")
        else:
            audit_log.append(f"No Disproof Keywords Found ({category})")
        return keyword_score > 0

    # Helper function to check disproof categories with detailed logging
    def has_disproof_categories(category):
        disproof_details = []
        result = False
        if category == "Slight/None":
            yc_category = safe_lower(row.get('YC_Category_Name', ''))
            cwa_category = safe_lower(row.get('CWA_Broad_Category_Name', ''))
            global_category = safe_lower(row.get('YC_Global_Category_Name', ''))
            disproof_cats = ["derivative income", "long-short equity", "trading--leveraged equity"]
            if yc_category in disproof_cats:
                disproof_details.append(f"YC_Category_Name '{yc_category}' matches disproof categories: {disproof_cats}")
                result = True
            disproof_cats = ["alternative", "nontraditional"]
            if cwa_category in disproof_cats:
                disproof_details.append(f"CWA_Broad_Category_Name '{cwa_category}' matches disproof categories: {disproof_cats}")
                result = True
            disproof_cats = ["options trading", "long/short equity"]
            if global_category in disproof_cats:
                disproof_details.append(f"YC_Global_Category_Name '{global_category}' matches disproof categories: {disproof_cats}")
                result = True
        elif category == "Moderate":
            yc_category = safe_lower(row.get('YC_Category_Name', ''))
            return_driver = safe_lower(row.get('return_driver', ''))
            disproof_cats = ["trading--leveraged equity", "market neutral"]
            if yc_category in disproof_cats:
                disproof_details.append(f"YC_Category_Name '{yc_category}' matches disproof categories: {disproof_cats}")
                result = True
            if return_driver in ["quant/systematic"]:
                disproof_details.append(f"return_driver '{return_driver}' matches disproof categories: ['quant/systematic']")
                result = True
        elif category == "Persistent Systematic":
            yc_category = safe_lower(row.get('YC_Category_Name', ''))
            product_name = safe_lower(row.get('ProductName', ''))
            disproof_cats = ["trading--leveraged equity", "trading--leveraged debt"]
            if yc_category in disproof_cats:
                disproof_details.append(f"YC_Category_Name '{yc_category}' matches disproof categories: {disproof_cats}")
                result = True
            if "leveraged" in product_name:
                disproof_details.append(f"ProductName '{product_name}' contains 'leveraged'")
                result = True
        if result:
            audit_log.append(f"Disproof Categories ({category}): {disproof_details}")
        else:
            audit_log.append(f"No Disproof Categories Found ({category})")
        return result

    # Branch 1: Simple Exposure (Long-Only, No Shorts/Other)
    # Revert to original strict range: 99 <= long_total <= 101
    if abs(short_total_rounded) <= 1 and abs(other_total_rounded) <= 1 and 99 <= long_total_rounded <= 101:
        audit_log.append("Branch 1 Simple Exposure: Minimal shorts/other (<=1%), 99 <= long_total <= 101")
        # Prove with fund family
        if pd.notna(row['fund_family']) and "return stacked" in row['fund_family'].lower():
            audit_log.append("Disproof: Fund family 'Return Stacked' suggests complexity, routing to next branch")
        elif has_disproof_keywords("Slight/None"):
            audit_log.append("Disproof triggered (keywords), routing to next branch")
        elif has_disproof_categories("Slight/None"):
            audit_log.append("Disproof triggered (categories), routing to next branch")
        else:
            audit_log.append("No disproof triggered, classifying as Slight/None")
            return "Slight/None", "Simple Exposure: Minimal shorts/other (<=1%), 99 <= long_total <= 101, passed disproof"
    else:
        reasons = []
        if abs(short_total_rounded) > 1:
            reasons.append(f"short_total_rounded={short_total_rounded} > 1")
        if abs(other_total_rounded) > 1:
            reasons.append(f"other_total_rounded={other_total_rounded} > 1")
        if not (99 <= long_total_rounded <= 101):
            reasons.append(f"long_total_rounded={long_total_rounded} not in [99, 101]")
        audit_log.append(f"Failed Simple Exposure: {', '.join(reasons)}")

    # Branch 2: Moderate Complexity (Minimal Shorts/Other, Balanced Exposures)
    if short_total_rounded <= 10 and other_total_rounded <= 10:
        audit_log.append("Branch 2 Moderate Complexity: Minimal shorts/other (<=10%)")
        if has_disproof_keywords("Moderate"):
            audit_log.append("Disproof triggered (keywords), routing to next branch")
        elif has_disproof_categories("Moderate"):
            audit_log.append("Disproof triggered (categories), routing to next branch")
        else:
            audit_log.append("No disproof triggered, classifying as Moderate")
            return "Moderate", "Moderate Complexity: Minimal shorts/other (<=10%), passed disproof"
    else:
        audit_log.append(f"Failed Moderate Complexity: shorts/other >10% "
                         f"(short_total_rounded={short_total_rounded}, other_total_rounded={other_total_rounded})")

    # Branch 3: Persistent Systematic Complexity (Significant Shorts/Other)
    if short_total_rounded <= 50 or other_total_rounded <= 50:
        audit_log.append("Branch 3 Persistent Systematic Complexity: Shorts/other <=50%")
        excluded = []
        for cat in ["Slight/None", "Moderate"]:
            if cat in possible_categories:
                possible_categories.remove(cat)
                excluded.append(cat)
        if excluded:
            audit_log.append(f"Excluded {excluded} due to shorts/other >10%")
        if has_disproof_keywords("Persistent Systematic"):
            audit_log.append("Disproof triggered (keywords), routing to next branch")
        elif has_disproof_categories("Persistent Systematic"):
            audit_log.append("Disproof triggered (categories), routing to next branch")
        else:
            audit_log.append("No disproof triggered, classifying as Persistent Systematic")
            return "Persistent Systematic", "Persistent Systematic Complexity: Significant shorts/other (<=50%), passed disproof"
    else:
        audit_log.append(f"Failed Persistent Systematic Complexity: shorts/other >50% "
                         f"(short_total_rounded={short_total_rounded}, other_total_rounded={other_total_rounded})")

    # Branch 4: Heavy Amplification Complexity (High Shorts/Other, Amplified Indicators)
    audit_log.append("Branch 4 Heavy Amplification Complexity: Shorts/other >50%")
    excluded = []
    for cat in ["Slight/None", "Moderate", "Persistent Systematic"]:
        if cat in possible_categories:
            possible_categories.remove(cat)
            excluded.append(cat)
    if excluded:
        audit_log.append(f"Excluded {excluded} due to shorts/other >50%")
    if has_disproof_keywords("Heavy Amplification"):
        audit_log.append("Disproof triggered (keywords), routing to alternative evaluation")
        return None, scores, audit_log
    if has_disproof_categories("Heavy Amplification"):
        audit_log.append("Disproof triggered (categories), routing to alternative evaluation")
        return None, scores, audit_log
    audit_log.append("No disproof triggered, classifying as Heavy Amplification")
    return "Heavy Amplification", "Heavy Amplification Complexity: High shorts/other (>50%), passed disproof"

# Alternative Evaluation Method for Ambiguous Cases
def alternative_evaluation(row, scores, audit_log):
    possible_categories = ["Slight/None", "Moderate", "Persistent Systematic", "Heavy Amplification"]
    audit_log.append("Entering Alternative Evaluation: No definitive classification from exposure tree")

    # Adjust scores based on exposure and category data
    if row['inverse_fund'] in [True, 1]:
        scores["Heavy Amplification"] += 2
        scores["Persistent Systematic"] += 1
        audit_log.append("Inverse fund flag True, boosting Heavy Amplification (+2) and Persistent Systematic (+1)")

    # Score based on categories
    yc_category = safe_lower(row.get('YC_Category_Name', ''))
    cwa_category = safe_lower(row.get('CWA_Broad_Category_Name', ''))
    global_category = safe_lower(row.get('YC_Global_Category_Name', ''))
    return_driver = safe_lower(row.get('return_driver', ''))

    # Score Slight/None
    if (yc_category in ["government bond", "corporate bond", "municipal bond"] or
        cwa_category in ["taxable fixed income", "municipal"] or
        return_driver in ["index based"]):
        scores["Slight/None"] += 2
        audit_log.append(f"Category scoring for Slight/None: Matched categories/return_driver, added +2")

    # Score Moderate
    if (yc_category in ["derivative income", "multisector bond"] or
        cwa_category in ["bond strategy", "strategic"] or
        global_category in ["flexible allocation", "us fixed income"]):
        scores["Moderate"] += 2
        audit_log.append(f"Category scoring for Moderate: Matched categories/global categories, added +2")

    # Score Persistent Systematic
    if (yc_category in ["long-short equity", "equity hedged"] or
        cwa_category in ["alternative", "nontraditional"] or
        global_category in ["long/short equity", "multialternative", "options trading"] or
        return_driver in ["quant/systematic", "active discretionary"]):
        scores["Persistent Systematic"] += 2
        audit_log.append(f"Category scoring for Persistent Systematic: Matched categories/global/return_driver, added +2")

    # Score Heavy Amplification
    if (yc_category in ["trading--leveraged equity", "trading--leveraged debt"] or
        "leveraged" in safe_lower(row.get('ProductName', ''))):
        scores["Heavy Amplification"] += 2
        audit_log.append(f"Category scoring for Heavy Amplification: Matched leveraged categories, added +2")

    # Evaluate pairs of categories
    audit_log.append("Alternative Evaluation: Evaluating pairs of categories")
    # Pair 1: Slight/None vs Moderate
    pair_scores = {"Slight/None": scores["Slight/None"], "Moderate": scores["Moderate"]}
    audit_log.append(f"Pair 1 (Slight/None vs Moderate) scores: {pair_scores}")
    max_score = max(pair_scores.values())
    top_pair = [cat for cat, score in pair_scores.items() if score == max_score]
    if len(top_pair) == 1:
        selected = top_pair[0]
        audit_log.append(f"Alternative Evaluation: Slight/None vs Moderate -> Selected {selected} with score {pair_scores[selected]}")
    else:
        keyword_tie_scores = {"Slight/None": 0, "Moderate": 0}
        for category in top_pair:
            for field in ['ProductName', 'investment_strategy', 'FS_insight']:
                keyword_tie_scores[category] += search_keywords(row[field], keyword_mappings[category])
        audit_log.append(f"Pair 1 tiebreaker keyword scores: {keyword_tie_scores}")
        max_keyword_score = max(keyword_tie_scores.values())
        top_keyword_pair = [cat for cat, score in keyword_tie_scores.items() if score == max_keyword_score]
        selected = top_keyword_pair[0] if len(top_keyword_pair) == 1 else min(top_pair, key=lambda x: ["Slight/None", "Moderate"].index(x))
        audit_log.append(f"Alternative Evaluation: Slight/None vs Moderate tiebreak -> Selected {selected}")

    # Double-check against next category
    if selected == "Slight/None":
        next_pair = {"Slight/None": scores["Slight/None"], "Moderate": scores["Moderate"]}
        if scores["Moderate"] >= scores["Slight/None"]:
            selected = "Moderate"
            audit_log.append(f"Double-check: Moderate score ({scores['Moderate']}) >= Slight/None ({scores['Slight/None']}), selecting Moderate")
    elif selected == "Moderate":
        next_pair = {"Moderate": scores["Moderate"], "Persistent Systematic": scores["Persistent Systematic"]}
        if scores["Persistent Systematic"] >= scores["Moderate"]:
            selected = "Persistent Systematic"
            audit_log.append(f"Double-check: Persistent Systematic score ({scores['Persistent Systematic']}) >= Moderate ({scores['Moderate']}), selecting Persistent Systematic")
    elif selected == "Persistent Systematic":
        next_pair = {"Persistent Systematic": scores["Persistent Systematic"], "Heavy Amplification": scores["Heavy Amplification"]}
        if scores["Heavy Amplification"] >= scores["Persistent Systematic"]:
            selected = "Heavy Amplification"
            audit_log.append(f"Double-check: Heavy Amplification score ({scores['Heavy Amplification']}) >= Persistent Systematic ({scores['Persistent Systematic']}), selecting Heavy Amplification")

    return selected, audit_log

# Main classification function
def classify_fund(row):
    possible_categories = ["Slight/None", "Moderate", "Persistent Systematic", "Heavy Amplification"]
    scores = {cat: 0 for cat in possible_categories}
    audit_log = []

    # Step 1: Override for Fund Family "Return Stacked ETFs"
    if pd.notna(row['fund_family']) and "return stacked etfs" in row['fund_family'].lower():
        audit_log.append("Override: Fund family 'Return Stacked ETFs' -> Persistent Systematic")
        return "Persistent Systematic", audit_log

    # Step 2: Direct keyword mapping (immediate classification)
    for category, keywords in direct_keyword_mappings.items():
        for field in ['ProductName', 'investment_strategy', 'FS_insight']:
            if pd.notna(row[field]):
                match_count = search_keywords(row[field], keywords)
                if match_count > 0:
                    audit_log.append(f"Direct keyword match for {category} in {field}: {match_count} keywords")
                    return category, audit_log

    # Step 3: Direct category mapping (immediate classification)
    for category, mappings in direct_category_mappings.items():
        for db_field, values in mappings.items():
            values_lower = [val.lower() for val in values]
            field_value = safe_lower(row.get(db_field, ''))
            if field_value in values_lower:
                audit_log.append(f"Direct category match for {category} in {db_field}: {field_value}")
                return category, audit_log

    # Step 4: Apply Boolean exclusions
    for categories, condition in boolean_exclusions.items():
        if condition(row):
            excluded = [cat for cat in possible_categories if cat in categories]
            for cat in excluded:
                possible_categories.remove(cat)
            audit_log.append(f"Excluded {excluded} due to Boolean flags")
            if len(possible_categories) == 1:
                return possible_categories[0], audit_log

    # Step 5: Exposure Decision Tree with Prove/Disprove
    exposure_result = classify_by_exposures_with_disproof(row)
    if isinstance(exposure_result, tuple) and len(exposure_result) == 2:
        classification, reason = exposure_result
        audit_log.append(f"Exposure-based classification: {classification} - {reason}")
        return classification, audit_log
    elif exposure_result[0] is None:
        # No definitive classification, proceed to alternative evaluation
        scores_update = exposure_result[1]
        for msg in exposure_result[2]:
            audit_log.append(msg)
        for cat in scores_update:
            scores[cat] += scores_update[cat]
        classification, eval_log = alternative_evaluation(row, scores, audit_log)
        for msg in eval_log:
            audit_log.append(msg)
        audit_log.append(f"Alternative Evaluation: Final classification: {classification}")
        return classification, audit_log
    else:
        # Continue with scoring
        possible_categories, scores_update, exposure_log = exposure_result
        for msg in exposure_log:
            audit_log.append(msg)
        for cat in scores_update:
            scores[cat] += scores_update[cat]

    # Step 6: Final Scoring if No Immediate Classification
    max_score = max(scores[cat] for cat in possible_categories)
    top_categories = [cat for cat in possible_categories if scores[cat] == max_score]
    if len(top_categories) == 1:
        final_classification = top_categories[0]
        audit_log.append(f"Final classification: {final_classification} with score {max_score}")
    else:
        keyword_tie_scores = {cat: 0 for cat in top_categories}
        for category in top_categories:
            for field in ['ProductName', 'investment_strategy', 'FS_insight']:
                keyword_tie_scores[category] += search_keywords(row[field], keyword_mappings[category])
        audit_log.append(f"Tiebreaker keyword scores: {keyword_tie_scores}")
        max_keyword_score = max(keyword_tie_scores.values())
        top_keyword_categories = [cat for cat, score in keyword_tie_scores.items() if score == max_keyword_score]
        if len(top_keyword_categories) == 1:
            final_classification = top_keyword_categories[0]
            audit_log.append(f"Tiebreaker used - Final classification: {final_classification} with tiebreaker keyword score {keyword_tie_scores[final_classification]}")
        else:
            conservative_order = ["Slight/None", "Moderate", "Persistent Systematic", "Heavy Amplification"]
            final_classification = min(top_keyword_categories, key=lambda x: conservative_order.index(x))
            audit_log.append(f"Secondary tiebreaker used - Final classification: {final_classification} (most conservative among {top_keyword_categories})")

    return final_classification, audit_log

# Main script
def main():
    # Create database engine
    engine = create_engine(connection_string)

    # Query to join tables
    query = """
    SELECT 
        f.SymbolCUSIP, f.ProductName, f.fund_family, f.investment_strategy, f.FS_insight,
        f.index_fund, f.inverse_fund, f.leveraged_fund, f.synthetic_replication_fund,
        f.fund_of_funds, f.ycharts_url, f.currency_hedged_fund,
        f.cash_long, f.cash_net, f.cash_short, f.stock_long, f.stock_net, f.stock_short,
        f.bond_long, f.bond_net, f.bond_short, f.other_long, f.other_net, f.other_short,
        f.return_driver, f.YC_BM_Symbol,
        cwa.CWA_Broad_Category_Name,
        yc.Category_Name AS YC_Category_Name,
        ycg.Global_Category_Name,
        ycba.YC_Broad_Asset_Class_Name
    FROM Funds_to_Screen f
    LEFT JOIN CWA_Broad_Category_List cwa ON f.CWA_Broad_Category_ID = cwa.ID
    LEFT JOIN YC_Category_List yc ON f.YC_Category_ID = yc.ID
    LEFT JOIN YC_Global_Category_List ycg ON f.YC_Global_Category_ID = ycg.ID
    LEFT JOIN YC_Broad_Asset_Class_List ycba ON f.YC_Broad_Asset_Class_ID = ycba.ID
    """

    # Load data into DataFrame
    df = pd.read_sql(query, engine)

    # Apply classification
    results = []
    for idx, row in df.iterrows():
        classification, audit_log = classify_fund(row)
        result_row = row.copy()
        result_row['Final_Classification'] = classification
        result_row['Audit_Log'] = "; ".join(audit_log)
        results.append(result_row)

    # Convert results to DataFrame
    result_df = pd.DataFrame(results)

    # Ensure desired column order
    output_columns = ['SymbolCUSIP', 'ProductName', 'fund_family', 'Final_Classification', 'ycharts_url'] + \
                     [col for col in result_df.columns if col not in ['SymbolCUSIP', 'ProductName', 'fund_family', 'Final_Classification', 'ycharts_url']]

    result_df = result_df[output_columns]

    # Export to Excel
    output_path = r"C:\Users\JulianHeron\Software Projects\Risk_Overlays_V3.2.xlsx"
    result_df.to_excel(output_path, index=False)
    print(f"Results exported to {output_path}")

if __name__ == "__main__":
    main()

Results exported to C:\Users\JulianHeron\Software Projects\Risk_Overlays_V3.2.xlsx


In [53]:
# This code was close, only issue is the 99.5% vs 100% issue
# placed here to preserve in case of grok issues

# V4 with Grok. Using tiered decision tree and other logic.
import pandas as pd
from sqlalchemy import create_engine
import re
import os

# Connection string
connection_string = (
    "mssql+pyodbc://JULIANS_LAPTOP\\SQLEXPRESS/"
    "CWA_Fund_Database?driver=ODBC+Driver+18+for+SQL+Server"
    "&trusted_connection=yes&TrustServerCertificate=yes"
)

# Helper function to safely convert to lowercase string (moved to top level)
def safe_lower(value):
    return value.lower() if isinstance(value, str) else ""

# Define keyword mappings for general scoring
keyword_mappings = {
    "Slight/None": ["long-only", "no derivatives", "no hedging", "no leverage", "no options", "no short", 
                    "position adjustment", "occasional hedging", "covered call", "put-write", "light hedge", 
                    "may include options", "limited use of derivatives", "for risk management purposes", 
                    "minor hedging", "occasional short positions", "overwrite", "investment grade", "core"],
    "Moderate": ["hedged", "currency hedge", "protective put", "partial hedge", "hedged equity", 
                 "covered call", "convexity option overlay", "option overlay", "put/spread collar", 
                 "forward agreement", "enhanced index strategy", "BuyWrite", "Buy-Write", "buy write", 
                 "option spread", "volatility hedge", "put options", "enhance", "options-based income", 
                 "ELN", "premium income", "call option", "FLEX options", "option premium", "write calls", 
                 "sell calls", "protective puts", "equity-linked notes", "structured notes", "risk mitigation", 
                 "downside protection", "limited hedging", "multi-asset"],
    "Persistent Systematic": ["tail-risk", "trend-following", "systematic hedging", "overlay", "CTA", 
                              "managed futures", "defined outcome", "long-short", "market neutral", 
                              "systematic strategy", "return stacking", "option writing", "straddle", 
                              "derivative income", "futures contracts", "swap contract", "forward agreement", 
                              "enhanced index strategy", "volatility hedge", "put options", "options-based income", 
                              "ELN", "option premium", "swap", "forward", "futures", "future", "VIX", 
                              "managed futures strategy", "trend strategy", "quantitative hedging", 
                              "systematic options", "options overlay strategy", "futures overlay", "swaps-based", 
                              "multi-asset", "Flex Options", "Flexible Exchange Options", "YieldMax", "buffer"],
    "Heavy Amplification": ["2x", "3x", "Uncapped Accelerator", "-2x", "-3x", "YieldMax"]
}

# Direct mapping keywords (often in ProductName)
direct_keyword_mappings = {
    "Persistent Systematic": ["Market Neutral", "managed futures", "Premia", "Return Stacked ETFs"]
}

# Direct mapping categories
direct_category_mappings = {
    "Persistent Systematic": {
        "YC_Category": ["Defined Outcome"],
        "CWA_Broad_Category": ["Defined Outcome"],
        "YC_Global_Category": ["market neutral"]
    },
    "Heavy Amplification": {
        "YC_Category": ["Trading--Leveraged Equity", "Trading--Leveraged Debt", "Trading--Leveraged Commodities"],
        "CWA_Broad_Category": ["Single Stock"]
    }
}

# Helper category mappings (narrow possibilities)
helper_category_mappings = {
    ("Persistent Systematic", "Heavy Amplification"): {
        "YC_Category": ["Trading--Inverse Commodities", "Trading--Inverse Debt", "Trading--Inverse Equity", "Trading--Miscellaneous"],
        "CWA_Broad_Category": ["Trading/Tactical"],
        "YC_Global_Category": ["Trading Tools"]
    },
    ("Persistent Systematic", "Moderate"): {
        "YC_Global_Category": ["Multialternative", "Long/Short Equity"],
        "YC_Category": ["Equity Hedged"]
    }
}

# Weak helper category mappings (may or may not have overlay)
weak_helper_category_mappings = {
    ("Slight/None", "Moderate"): {
        "YC_Global_Category": ["Flexible Allocation", "Alternative Miscellaneous"],
        "return_driver": ["Index Based", "Factor/Smart Beta"]
    },
    ("Slight/None", "Moderate", "Persistent Systematic"): {
        "YC_Category": ["Relative Value Arbitrage"]
    },
    ("Heavy Amplification", "Persistent Systematic", "Moderate", "Slight/None"): {
        "return_driver": ["Quant/Systematic"]
    },
    ("Persistent Systematic", "Moderate", "Slight/None"): {
        "return_driver": ["Active Discretionary", "Multi-Strategy"]
    }
}

# Boolean flags to remove categories
boolean_exclusions = {
    ("Slight/None", "Moderate"): lambda row: row['leveraged_fund'] in [True, 1] or \
                                            row['synthetic_replication_fund'] in [True, 1] or \
                                            row['inverse_fund'] in [True, 1]
}

# Function to search text for keywords
def search_keywords(text, keywords):
    if pd.isna(text):
        return 0
    text = str(text).lower()
    return sum(1 for keyword in keywords if re.search(r'\b' + re.escape(keyword.lower()) + r'\b', text))

# Decision Tree for Exposure Classification with Prove/Disprove
def classify_by_exposures_with_disproof(row):
    audit_log = []
    possible_categories = ["Slight/None", "Moderate", "Persistent Systematic", "Heavy Amplification"]
    scores = {cat: 0 for cat in possible_categories}

    # Convert exposure columns to numeric, handle non-numeric gracefully
    exposure_cols = ['cash_long', 'cash_net', 'cash_short', 'stock_long', 'stock_net', 'stock_short',
                    'bond_long', 'bond_net', 'bond_short', 'other_long', 'other_net', 'other_short']
    for col in exposure_cols:
        value = pd.to_numeric(row[col], errors='coerce')
        row[col] = 0 if pd.isna(value) else value

    # Calculate totals
    long_total = row['cash_long'] + row['stock_long'] + row['bond_long'] + row['other_long']
    short_total = row['cash_short'] + row['stock_short'] + row['bond_short'] + row['other_short']
    other_total = row['other_long'] + row['other_short']
    stock_long = row['stock_long']
    cash_short = row['cash_short']

    # Round totals to 4 decimal places for comparison
    long_total_rounded = round(long_total, 4)
    short_total_rounded = round(short_total, 4)
    other_total_rounded = round(other_total, 4)

    # Debug logging for exposure values
    debug_msg = (f"Exposure Values - long_total: {long_total}, long_total_rounded: {long_total_rounded}, "
                 f"short_total: {short_total}, short_total_rounded: {short_total_rounded}, "
                 f"other_total: {other_total}, other_total_rounded: {other_total_rounded}")
    audit_log.append(debug_msg)

    # Define disproof keywords for each branch
    disproof_keywords = {
        "Slight/None": ["derivatives", "swaps", "futures", "short", "hedge", "long-short", "inverse", "leveraged", "tail risk", "tail-risk"],
        "Moderate": ["systematic", "trend-following", "2x", "3x", "market neutral", "quantitative hedging", "managed futures",
                     "managed futures strategy", "trend strategy", "YieldMax", "tail risk", "tail-risk"],
        "Persistent Systematic": ["2x", "3x", "-2x", "-3x", "uncapped accelerator"],
        "Heavy Amplification": ["long-only", "no derivatives", "no short"]
    }

    # Helper function to check disproof keywords with detailed logging
    def has_disproof_keywords(category):
        keywords = disproof_keywords.get(category, [])
        keyword_score = 0
        disproof_details = []
        for field in ['ProductName', 'investment_strategy', 'FS_insight']:
            if pd.notna(row[field]):
                text = str(row[field]).lower()
                found_keywords = [keyword for keyword in keywords if re.search(r'\b' + re.escape(keyword.lower()) + r'\b', text)]
                if found_keywords:
                    disproof_details.append(f"Field '{field}' contains disproof keywords: {found_keywords}")
                    keyword_score += len(found_keywords)
        if keyword_score > 0:
            audit_log.append(f"Disproof Keywords ({category}): {disproof_details}")
        else:
            audit_log.append(f"No Disproof Keywords Found ({category})")
        return keyword_score > 0

    # Helper function to check disproof categories with detailed logging
    def has_disproof_categories(category):
        disproof_details = []
        result = False
        if category == "Slight/None":
            yc_category = safe_lower(row.get('YC_Category_Name', ''))
            cwa_category = safe_lower(row.get('CWA_Broad_Category_Name', ''))
            global_category = safe_lower(row.get('YC_Global_Category_Name', ''))
            disproof_cats = ["derivative income", "long-short equity", "trading--leveraged equity"]
            if yc_category in disproof_cats:
                disproof_details.append(f"YC_Category_Name '{yc_category}' matches disproof categories: {disproof_cats}")
                result = True
            disproof_cats = ["alternative", "nontraditional"]
            if cwa_category in disproof_cats:
                disproof_details.append(f"CWA_Broad_Category_Name '{cwa_category}' matches disproof categories: {disproof_cats}")
                result = True
            disproof_cats = ["options trading", "long/short equity"]
            if global_category in disproof_cats:
                disproof_details.append(f"YC_Global_Category_Name '{global_category}' matches disproof categories: {disproof_cats}")
                result = True
        elif category == "Moderate":
            yc_category = safe_lower(row.get('YC_Category_Name', ''))
            return_driver = safe_lower(row.get('return_driver', ''))
            disproof_cats = ["trading--leveraged equity", "market neutral"]
            if yc_category in disproof_cats:
                disproof_details.append(f"YC_Category_Name '{yc_category}' matches disproof categories: {disproof_cats}")
                result = True
            if return_driver in ["quant/systematic"]:
                disproof_details.append(f"return_driver '{return_driver}' matches disproof categories: ['quant/systematic']")
                result = True
        elif category == "Persistent Systematic":
            yc_category = safe_lower(row.get('YC_Category_Name', ''))
            product_name = safe_lower(row.get('ProductName', ''))
            disproof_cats = ["trading--leveraged equity", "trading--leveraged debt"]
            if yc_category in disproof_cats:
                disproof_details.append(f"YC_Category_Name '{yc_category}' matches disproof categories: {disproof_cats}")
                result = True
            if "leveraged" in product_name:
                disproof_details.append(f"ProductName '{product_name}' contains 'leveraged'")
                result = True
        if result:
            audit_log.append(f"Disproof Categories ({category}): {disproof_details}")
        else:
            audit_log.append(f"No Disproof Categories Found ({category})")
        return result

    # Branch 1: Simple Exposure (Long-Only, No Shorts/Other)
    if abs(short_total_rounded) < 1 and abs(other_total_rounded) < 1 and long_total_rounded <= 100.2:
        audit_log.append("Branch 1 Simple Exposure: Effectively no shorts/other (<1%), long_total <= 100.2%")
        # Prove with fund family
        if pd.notna(row['fund_family']) and "return stacked" in row['fund_family'].lower():
            audit_log.append("Disproof: Fund family 'Return Stacked' suggests complexity, routing to next branch")
        elif has_disproof_keywords("Slight/None") or has_disproof_categories("Slight/None"):
            audit_log.append("Disproof triggered, routing to next branch")
        else:
            audit_log.append("No disproof triggered, classifying as Slight/None")
            return "Slight/None", "Simple Exposure: No shorts/other (<1%), long_total <= 99.5%, passed disproof"

    # Branch 2: Moderate Complexity (Minimal Shorts/Other, Balanced Exposures)
    if short_total_rounded <= 10 and other_total_rounded <= 10:
        audit_log.append("Branch 2 Moderate Complexity: Minimal shorts/other (<=10%)")
        if has_disproof_keywords("Moderate") or has_disproof_categories("Moderate"):
            audit_log.append("Disproof triggered, routing to next branch")
        else:
            audit_log.append("No disproof triggered, classifying as Moderate")
            return "Moderate", "Moderate Complexity: Minimal shorts/other (<=10%), passed disproof"

    # Branch 3: Persistent Systematic Complexity (Significant Shorts/Other)
    if short_total_rounded <= 50 or other_total_rounded <= 50:
        audit_log.append("Branch 3 Persistent Systematic Complexity: Shorts/other <=50%")
        excluded = []
        for cat in ["Slight/None", "Moderate"]:
            if cat in possible_categories:
                possible_categories.remove(cat)
                excluded.append(cat)
        if excluded:
            audit_log.append(f"Excluded {excluded} due to shorts/other >10%")
        if has_disproof_keywords("Persistent Systematic") or has_disproof_categories("Persistent Systematic"):
            audit_log.append("Disproof triggered, routing to next branch")
        else:
            audit_log.append("No disproof triggered, classifying as Persistent Systematic")
            return "Persistent Systematic", "Persistent Systematic Complexity: Significant shorts/other (<=50%), passed disproof"

    # Branch 4: Heavy Amplification Complexity (High Shorts/Other, Amplified Indicators)
    audit_log.append("Branch 4 Heavy Amplification Complexity: Shorts/other >50%")
    excluded = []
    for cat in ["Slight/None", "Moderate", "Persistent Systematic"]:
        if cat in possible_categories:
            possible_categories.remove(cat)
            excluded.append(cat)
    if excluded:
        audit_log.append(f"Excluded {excluded} due to shorts/other >50%")
    if has_disproof_keywords("Heavy Amplification") or has_disproof_categories("Heavy Amplification"):
        audit_log.append("Disproof triggered, routing to alternative evaluation")
        return None, scores, audit_log
    audit_log.append("No disproof triggered, classifying as Heavy Amplification")
    return "Heavy Amplification", "Heavy Amplification Complexity: High shorts/other (>50%), passed disproof"

# Alternative Evaluation Method for Ambiguous Cases
def alternative_evaluation(row, scores, audit_log):
    possible_categories = ["Slight/None", "Moderate", "Persistent Systematic", "Heavy Amplification"]
    audit_log.append("Entering Alternative Evaluation: No definitive classification from exposure tree")

    # Adjust scores based on exposure and category data
    if row['inverse_fund'] in [True, 1]:
        scores["Heavy Amplification"] += 2
        scores["Persistent Systematic"] += 1
        audit_log.append("Inverse fund flag True, boosting Heavy Amplification (+2) and Persistent Systematic (+1)")

    # Score based on categories
    yc_category = safe_lower(row.get('YC_Category_Name', ''))
    cwa_category = safe_lower(row.get('CWA_Broad_Category_Name', ''))
    global_category = safe_lower(row.get('YC_Global_Category_Name', ''))
    return_driver = safe_lower(row.get('return_driver', ''))

    # Score Slight/None
    if (yc_category in ["government bond", "corporate bond", "municipal bond"] or
        cwa_category in ["taxable fixed income", "municipal"] or
        return_driver in ["index based"]):
        scores["Slight/None"] += 2
        audit_log.append(f"Category scoring for Slight/None: Matched categories/return_driver, added +2")

    # Score Moderate
    if (yc_category in ["derivative income", "multisector bond"] or
        cwa_category in ["bond strategy", "strategic"] or
        global_category in ["flexible allocation", "us fixed income"]):
        scores["Moderate"] += 2
        audit_log.append(f"Category scoring for Moderate: Matched categories/global categories, added +2")

    # Score Persistent Systematic
    if (yc_category in ["long-short equity", "equity hedged"] or
        cwa_category in ["alternative", "nontraditional"] or
        global_category in ["long/short equity", "multialternative", "options trading"] or
        return_driver in ["quant/systematic", "active discretionary"]):
        scores["Persistent Systematic"] += 2
        audit_log.append(f"Category scoring for Persistent Systematic: Matched categories/global/return_driver, added +2")

    # Score Heavy Amplification
    if (yc_category in ["trading--leveraged equity", "trading--leveraged debt"] or
        "leveraged" in safe_lower(row.get('ProductName', ''))):
        scores["Heavy Amplification"] += 2
        audit_log.append(f"Category scoring for Heavy Amplification: Matched leveraged categories, added +2")

    # Evaluate pairs of categories
    audit_log.append("Alternative Evaluation: Evaluating pairs of categories")
    # Pair 1: Slight/None vs Moderate
    pair_scores = {"Slight/None": scores["Slight/None"], "Moderate": scores["Moderate"]}
    audit_log.append(f"Pair 1 (Slight/None vs Moderate) scores: {pair_scores}")
    max_score = max(pair_scores.values())
    top_pair = [cat for cat, score in pair_scores.items() if score == max_score]
    if len(top_pair) == 1:
        selected = top_pair[0]
        audit_log.append(f"Alternative Evaluation: Slight/None vs Moderate -> Selected {selected} with score {pair_scores[selected]}")
    else:
        keyword_tie_scores = {"Slight/None": 0, "Moderate": 0}
        for category in top_pair:
            for field in ['ProductName', 'investment_strategy', 'FS_insight']:
                keyword_tie_scores[category] += search_keywords(row[field], keyword_mappings[category])
        audit_log.append(f"Pair 1 tiebreaker keyword scores: {keyword_tie_scores}")
        max_keyword_score = max(keyword_tie_scores.values())
        top_keyword_pair = [cat for cat, score in keyword_tie_scores.items() if score == max_keyword_score]
        selected = top_keyword_pair[0] if len(top_keyword_pair) == 1 else min(top_pair, key=lambda x: ["Slight/None", "Moderate"].index(x))
        audit_log.append(f"Alternative Evaluation: Slight/None vs Moderate tiebreak -> Selected {selected}")

    # Double-check against next category
    if selected == "Slight/None":
        next_pair = {"Slight/None": scores["Slight/None"], "Moderate": scores["Moderate"]}
        if scores["Moderate"] >= scores["Slight/None"]:
            selected = "Moderate"
            audit_log.append(f"Double-check: Moderate score ({scores['Moderate']}) >= Slight/None ({scores['Slight/None']}), selecting Moderate")
    elif selected == "Moderate":
        next_pair = {"Moderate": scores["Moderate"], "Persistent Systematic": scores["Persistent Systematic"]}
        if scores["Persistent Systematic"] >= scores["Moderate"]:
            selected = "Persistent Systematic"
            audit_log.append(f"Double-check: Persistent Systematic score ({scores['Persistent Systematic']}) >= Moderate ({scores['Moderate']}), selecting Persistent Systematic")
    elif selected == "Persistent Systematic":
        next_pair = {"Persistent Systematic": scores["Persistent Systematic"], "Heavy Amplification": scores["Heavy Amplification"]}
        if scores["Heavy Amplification"] >= scores["Persistent Systematic"]:
            selected = "Heavy Amplification"
            audit_log.append(f"Double-check: Heavy Amplification score ({scores['Heavy Amplification']}) >= Persistent Systematic ({scores['Persistent Systematic']}), selecting Heavy Amplification")

    return selected, audit_log

# Main classification function
def classify_fund(row):
    possible_categories = ["Slight/None", "Moderate", "Persistent Systematic", "Heavy Amplification"]
    scores = {cat: 0 for cat in possible_categories}
    audit_log = []

    # Step 1: Override for Fund Family "Return Stacked ETFs"
    if pd.notna(row['fund_family']) and "return stacked etfs" in row['fund_family'].lower():
        audit_log.append("Override: Fund family 'Return Stacked ETFs' -> Persistent Systematic")
        return "Persistent Systematic", audit_log

    # Step 2: Direct keyword mapping (immediate classification)
    for category, keywords in direct_keyword_mappings.items():
        for field in ['ProductName', 'investment_strategy', 'FS_insight']:
            if pd.notna(row[field]):
                match_count = search_keywords(row[field], keywords)
                if match_count > 0:
                    audit_log.append(f"Direct keyword match for {category} in {field}: {match_count} keywords")
                    return category, audit_log

    # Step 3: Direct category mapping (immediate classification)
    for category, mappings in direct_category_mappings.items():
        for db_field, values in mappings.items():
            values_lower = [val.lower() for val in values]
            field_value = safe_lower(row.get(db_field, ''))
            if field_value in values_lower:
                audit_log.append(f"Direct category match for {category} in {db_field}: {field_value}")
                return category, audit_log

    # Step 4: Apply Boolean exclusions
    for categories, condition in boolean_exclusions.items():
        if condition(row):
            excluded = [cat for cat in possible_categories if cat in categories]
            for cat in excluded:
                possible_categories.remove(cat)
            audit_log.append(f"Excluded {excluded} due to Boolean flags")
            if len(possible_categories) == 1:
                return possible_categories[0], audit_log

    # Step 5: Exposure Decision Tree with Prove/Disprove
    exposure_result = classify_by_exposures_with_disproof(row)
    if isinstance(exposure_result, tuple) and len(exposure_result) == 2:
        classification, reason = exposure_result
        audit_log.append(f"Exposure-based classification: {classification} - {reason}")
        return classification, audit_log
    elif exposure_result[0] is None:
        # No definitive classification, proceed to alternative evaluation
        scores_update = exposure_result[1]
        for msg in exposure_result[2]:
            audit_log.append(msg)
        for cat in scores_update:
            scores[cat] += scores_update[cat]
        classification, eval_log = alternative_evaluation(row, scores, audit_log)
        for msg in eval_log:
            audit_log.append(msg)
        audit_log.append(f"Alternative Evaluation: Final classification: {classification}")
        return classification, audit_log
    else:
        # Continue with scoring
        possible_categories, scores_update, exposure_log = exposure_result
        for msg in exposure_log:
            audit_log.append(msg)
        for cat in scores_update:
            scores[cat] += scores_update[cat]

    # Step 6: Final Scoring if No Immediate Classification
    max_score = max(scores[cat] for cat in possible_categories)
    top_categories = [cat for cat in possible_categories if scores[cat] == max_score]
    if len(top_categories) == 1:
        final_classification = top_categories[0]
        audit_log.append(f"Final classification: {final_classification} with score {max_score}")
    else:
        keyword_tie_scores = {cat: 0 for cat in top_categories}
        for category in top_categories:
            for field in ['ProductName', 'investment_strategy', 'FS_insight']:
                keyword_tie_scores[category] += search_keywords(row[field], keyword_mappings[category])
        audit_log.append(f"Tiebreaker keyword scores: {keyword_tie_scores}")
        max_keyword_score = max(keyword_tie_scores.values())
        top_keyword_categories = [cat for cat, score in keyword_tie_scores.items() if score == max_keyword_score]
        if len(top_keyword_categories) == 1:
            final_classification = top_keyword_categories[0]
            audit_log.append(f"Tiebreaker used - Final classification: {final_classification} with tiebreaker keyword score {keyword_tie_scores[final_classification]}")
        else:
            conservative_order = ["Slight/None", "Moderate", "Persistent Systematic", "Heavy Amplification"]
            final_classification = min(top_keyword_categories, key=lambda x: conservative_order.index(x))
            audit_log.append(f"Secondary tiebreaker used - Final classification: {final_classification} (most conservative among {top_keyword_categories})")

    return final_classification, audit_log

# Main script
def main():
    # Create database engine
    engine = create_engine(connection_string)

    # Query to join tables
    query = """
    SELECT 
        f.SymbolCUSIP, f.ProductName, f.fund_family, f.investment_strategy, f.FS_insight,
        f.index_fund, f.inverse_fund, f.leveraged_fund, f.synthetic_replication_fund,
        f.fund_of_funds, f.ycharts_url, f.currency_hedged_fund,
        f.cash_long, f.cash_net, f.cash_short, f.stock_long, f.stock_net, f.stock_short,
        f.bond_long, f.bond_net, f.bond_short, f.other_long, f.other_net, f.other_short,
        f.return_driver, f.YC_BM_Symbol,
        cwa.CWA_Broad_Category_Name,
        yc.Category_Name AS YC_Category_Name,
        ycg.Global_Category_Name,
        ycba.YC_Broad_Asset_Class_Name
    FROM Funds_to_Screen f
    LEFT JOIN CWA_Broad_Category_List cwa ON f.CWA_Broad_Category_ID = cwa.ID
    LEFT JOIN YC_Category_List yc ON f.YC_Category_ID = yc.ID
    LEFT JOIN YC_Global_Category_List ycg ON f.YC_Global_Category_ID = ycg.ID
    LEFT JOIN YC_Broad_Asset_Class_List ycba ON f.YC_Broad_Asset_Class_ID = ycba.ID
    """

    # Load data into DataFrame
    df = pd.read_sql(query, engine)

    # Apply classification
    results = []
    for idx, row in df.iterrows():
        classification, audit_log = classify_fund(row)
        result_row = row.copy()
        result_row['Final_Classification'] = classification
        result_row['Audit_Log'] = "; ".join(audit_log)
        results.append(result_row)

    # Convert results to DataFrame
    result_df = pd.DataFrame(results)

    # Ensure desired column order
    output_columns = ['SymbolCUSIP', 'ProductName', 'fund_family', 'Final_Classification', 'ycharts_url'] + \
                     [col for col in result_df.columns if col not in ['SymbolCUSIP', 'ProductName', 'fund_family', 'Final_Classification', 'ycharts_url']]

    result_df = result_df[output_columns]

    # Export to Excel
    output_path = r"C:\Users\JulianHeron\Software Projects\Risk_Overlays_V4.xlsx"
    result_df.to_excel(output_path, index=False)
    print(f"Results exported to {output_path}")

if __name__ == "__main__":
    main()

Results exported to C:\Users\JulianHeron\Software Projects\Risk_Overlays_V4.xlsx


In [55]:
# V4.1 with Scale Fix in Logs (percentages mean 1 = 1%, 10 = 10%, etc.)

import pandas as pd
from sqlalchemy import create_engine
import re
import os

# Connection string
connection_string = (
    "mssql+pyodbc://JULIANS_LAPTOP\\SQLEXPRESS/"
    "CWA_Fund_Database?driver=ODBC+Driver+18+for+SQL+Server"
    "&trusted_connection=yes&TrustServerCertificate=yes"
)

# Helper function to safely convert to lowercase string (moved to top level)
def safe_lower(value):
    return value.lower() if isinstance(value, str) else ""

# Define keyword mappings for general scoring
keyword_mappings = {
    "Slight/None": [
        "long-only", "no derivatives", "no hedging", "no leverage", "no options", "no short",
        "position adjustment", "occasional hedging", "covered call", "put-write", "light hedge",
        "may include options", "limited use of derivatives", "for risk management purposes",
        "minor hedging", "occasional short positions", "overwrite", "investment grade", "core"
    ],
    "Moderate": [
        "hedged", "currency hedge", "protective put", "partial hedge", "hedged equity",
        "covered call", "convexity option overlay", "option overlay", "put/spread collar",
        "forward agreement", "enhanced index strategy", "BuyWrite", "Buy-Write", "buy write",
        "option spread", "volatility hedge", "put options", "enhance", "options-based income",
        "ELN", "premium income", "call option", "FLEX options", "option premium", "write calls",
        "sell calls", "protective puts", "equity-linked notes", "structured notes",
        "risk mitigation", "downside protection", "limited hedging", "multi-asset"
    ],
    "Persistent Systematic": [
        "tail-risk", "trend-following", "systematic hedging", "overlay", "CTA", "managed futures",
        "defined outcome", "long-short", "market neutral", "systematic strategy", "return stacking",
        "option writing", "straddle", "derivative income", "futures contracts", "swap contract",
        "forward agreement", "enhanced index strategy", "volatility hedge", "put options",
        "options-based income", "ELN", "option premium", "swap", "forward", "futures", "future",
        "VIX", "managed futures strategy", "trend strategy", "quantitative hedging",
        "systematic options", "options overlay strategy", "futures overlay", "swaps-based",
        "multi-asset", "Flex Options", "Flexible Exchange Options", "YieldMax", "buffer"
    ],
    "Heavy Amplification": [
        "2x", "3x", "Uncapped Accelerator", "-2x", "-3x", "YieldMax"
    ]
}

# Direct mapping keywords (often in ProductName)
direct_keyword_mappings = {
    "Persistent Systematic": ["Market Neutral", "managed futures", "Premia", "Return Stacked ETFs"]
}

# Direct mapping categories
direct_category_mappings = {
    "Persistent Systematic": {
        "YC_Category": ["Defined Outcome"],
        "CWA_Broad_Category": ["Defined Outcome"],
        "YC_Global_Category": ["market neutral"]
    },
    "Heavy Amplification": {
        "YC_Category": [
            "Trading--Leveraged Equity", "Trading--Leveraged Debt", "Trading--Leveraged Commodities"
        ],
        "CWA_Broad_Category": ["Single Stock"]
    }
}

# Helper category mappings (narrow possibilities)
helper_category_mappings = {
    ("Persistent Systematic", "Heavy Amplification"): {
        "YC_Category": [
            "Trading--Inverse Commodities", "Trading--Inverse Debt", "Trading--Inverse Equity",
            "Trading--Miscellaneous"
        ],
        "CWA_Broad_Category": ["Trading/Tactical"],
        "YC_Global_Category": ["Trading Tools"]
    },
    ("Persistent Systematic", "Moderate"): {
        "YC_Global_Category": ["Multialternative", "Long/Short Equity"],
        "YC_Category": ["Equity Hedged"]
    }
}

# Weak helper category mappings (may or may not have overlay)
weak_helper_category_mappings = {
    ("Slight/None", "Moderate"): {
        "YC_Global_Category": ["Flexible Allocation", "Alternative Miscellaneous"],
        "return_driver": ["Index Based", "Factor/Smart Beta"]
    },
    ("Slight/None", "Moderate", "Persistent Systematic"): {
        "YC_Category": ["Relative Value Arbitrage"]
    },
    ("Heavy Amplification", "Persistent Systematic", "Moderate", "Slight/None"): {
        "return_driver": ["Quant/Systematic"]
    },
    ("Persistent Systematic", "Moderate", "Slight/None"): {
        "return_driver": ["Active Discretionary", "Multi-Strategy"]
    }
}

# Boolean flags to remove categories
boolean_exclusions = {
    ("Slight/None", "Moderate"): lambda row: row['leveraged_fund'] in [True, 1]
    or row['synthetic_replication_fund'] in [True, 1]
    or row['inverse_fund'] in [True, 1]
}

# Function to search text for keywords
def search_keywords(text, keywords):
    if pd.isna(text):
        return 0
    text = str(text).lower()
    return sum(1 for keyword in keywords if re.search(r'\b' + re.escape(keyword.lower()) + r'\b', text))

# Decision Tree for Exposure Classification with Prove/Disprove
def classify_by_exposures_with_disproof(row):
    audit_log = []
    possible_categories = ["Slight/None", "Moderate", "Persistent Systematic", "Heavy Amplification"]

    # Convert exposure columns to numeric, handle non-numeric gracefully
    exposure_cols = [
        'cash_long', 'cash_net', 'cash_short',
        'stock_long', 'stock_net', 'stock_short',
        'bond_long', 'bond_net', 'bond_short',
        'other_long', 'other_net', 'other_short'
    ]
    for col in exposure_cols:
        value = pd.to_numeric(row[col], errors='coerce')
        row[col] = 0 if pd.isna(value) else value

    # Calculate totals (all in percentage points: e.g., 1 = 1%, 10 = 10%, etc.)
    long_total = row['cash_long'] + row['stock_long'] + row['bond_long'] + row['other_long']
    short_total = row['cash_short'] + row['stock_short'] + row['bond_short'] + row['other_short']
    other_total = row['other_long'] + row['other_short']

    # Round totals to 4 decimal places
    long_total_rounded = round(long_total, 4)
    short_total_rounded = round(short_total, 4)
    other_total_rounded = round(other_total, 4)

    # Debug logging for exposure values
    debug_msg = (
        f"Exposure Values - long_total: {long_total}, short_total: {short_total}, other_total: {other_total} "
        f"(rounded => {long_total_rounded}, {short_total_rounded}, {other_total_rounded})"
    )
    audit_log.append(debug_msg)

    # Define disproof keywords for each branch
    disproof_keywords = {
        "Slight/None": [
            "derivatives", "swaps", "futures", "short", "hedge", "long-short", "inverse", 
            "leveraged", "tail risk", "tail-risk"
        ],
        "Moderate": [
            "systematic", "trend-following", "2x", "3x", "market neutral", "quantitative hedging",
            "managed futures", "managed futures strategy", "trend strategy", "YieldMax", "tail risk", "tail-risk"
        ],
        "Persistent Systematic": ["2x", "3x", "-2x", "-3x", "uncapped accelerator"],
        "Heavy Amplification": ["long-only", "no derivatives", "no short"]
    }

    # Helper function to check disproof keywords with detailed logging
    def has_disproof_keywords(category):
        keywords = disproof_keywords.get(category, [])
        keyword_score = 0
        disproof_details = []
        for field in ['ProductName', 'investment_strategy', 'FS_insight']:
            if pd.notna(row[field]):
                text = str(row[field]).lower()
                found_keywords = [
                    keyword for keyword in keywords
                    if re.search(r'\b' + re.escape(keyword.lower()) + r'\b', text)
                ]
                if found_keywords:
                    disproof_details.append(f"Field '{field}' contains disproof keywords: {found_keywords}")
                    keyword_score += len(found_keywords)
        if keyword_score > 0:
            audit_log.append(f"Disproof Keywords ({category}): {disproof_details}")
        else:
            audit_log.append(f"No Disproof Keywords Found ({category})")
        return keyword_score > 0

    # Helper function to check disproof categories with detailed logging
    def has_disproof_categories(category):
        disproof_details = []
        result = False
        if category == "Slight/None":
            yc_category = safe_lower(row.get('YC_Category_Name', ''))
            cwa_category = safe_lower(row.get('CWA_Broad_Category_Name', ''))
            global_category = safe_lower(row.get('YC_Global_Category_Name', ''))
            disproof_cats = ["derivative income", "long-short equity", "trading--leveraged equity"]
            if yc_category in disproof_cats:
                disproof_details.append(
                    f"YC_Category_Name '{yc_category}' matches disproof categories: {disproof_cats}"
                )
                result = True
            disproof_cats = ["alternative", "nontraditional"]
            if cwa_category in disproof_cats:
                disproof_details.append(
                    f"CWA_Broad_Category_Name '{cwa_category}' matches disproof categories: {disproof_cats}"
                )
                result = True
            disproof_cats = ["options trading", "long/short equity"]
            if global_category in disproof_cats:
                disproof_details.append(
                    f"YC_Global_Category_Name '{global_category}' matches disproof categories: {disproof_cats}"
                )
                result = True
        elif category == "Moderate":
            yc_category = safe_lower(row.get('YC_Category_Name', ''))
            return_driver = safe_lower(row.get('return_driver', ''))
            disproof_cats = ["trading--leveraged equity", "market neutral"]
            if yc_category in disproof_cats:
                disproof_details.append(
                    f"YC_Category_Name '{yc_category}' matches disproof categories: {disproof_cats}"
                )
                result = True
            if return_driver in ["quant/systematic"]:
                disproof_details.append(
                    f"return_driver '{return_driver}' matches disproof categories: ['quant/systematic']"
                )
                result = True
        elif category == "Persistent Systematic":
            yc_category = safe_lower(row.get('YC_Category_Name', ''))
            product_name = safe_lower(row.get('ProductName', ''))
            disproof_cats = ["trading--leveraged equity", "trading--leveraged debt"]
            if yc_category in disproof_cats:
                disproof_details.append(
                    f"YC_Category_Name '{yc_category}' matches disproof categories: {disproof_cats}"
                )
                result = True
            if "leveraged" in product_name:
                disproof_details.append(f"ProductName '{product_name}' contains 'leveraged'")
                result = True
        if result:
            audit_log.append(f"Disproof Categories ({category}): {disproof_details}")
        else:
            audit_log.append(f"No Disproof Categories Found ({category})")
        return result

    # Branch 1: Simple Exposure (Long-Only, No Shorts/Other)
    #
    # Here, short_total_rounded < 1 means <1% short exposure,
    # and long_total_rounded <= 100.2 means up to ~100% long (with small rounding margin).
    #
    if abs(short_total_rounded) < 1 and abs(other_total_rounded) < 1 and long_total_rounded <= 100.2:
        audit_log.append(
            "Branch 1 Simple Exposure: Effectively no shorts/other (<1%), long_total <= 100.2%"
        )
        # Prove with fund family
        if pd.notna(row['fund_family']) and "return stacked" in row['fund_family'].lower():
            audit_log.append("Disproof: Fund family 'Return Stacked' suggests complexity, routing to next branch")
        elif has_disproof_keywords("Slight/None") or has_disproof_categories("Slight/None"):
            audit_log.append("Disproof triggered, routing to next branch")
        else:
            audit_log.append("No disproof triggered, classifying as Slight/None")
            return (
                "Slight/None",
                "Simple Exposure: no shorts/other (<1%), long_total <= 100.2%, passed disproof"
            )

    # Branch 2: Moderate Complexity (Minimal Shorts/Other)
    #
    # short_total_rounded <= 10 means <=10% short exposure.
    #
    if short_total_rounded <= 10 and other_total_rounded <= 10:
        audit_log.append("Branch 2 Moderate Complexity: Minimal shorts/other (<=10%)")
        if has_disproof_keywords("Moderate") or has_disproof_categories("Moderate"):
            audit_log.append("Disproof triggered, routing to next branch")
        else:
            audit_log.append("No disproof triggered, classifying as Moderate")
            return (
                "Moderate",
                "Moderate Complexity: Minimal shorts/other (<=10%), passed disproof"
            )

    # Branch 3: Persistent Systematic Complexity (Significant Shorts/Other)
    #
    # short_total_rounded <= 50 means <=50% short exposure, etc.
    #
    if short_total_rounded <= 50 or other_total_rounded <= 50:
        audit_log.append("Branch 3 Persistent Systematic Complexity: Shorts/other <=50%")
        excluded = []
        for cat in ["Slight/None", "Moderate"]:
            if cat in possible_categories:
                possible_categories.remove(cat)
                excluded.append(cat)
        if excluded:
            audit_log.append(f"Excluded {excluded} due to shorts/other >10%")
        if has_disproof_keywords("Persistent Systematic") or has_disproof_categories("Persistent Systematic"):
            audit_log.append("Disproof triggered, routing to next branch")
        else:
            audit_log.append("No disproof triggered, classifying as Persistent Systematic")
            return (
                "Persistent Systematic",
                "Persistent Systematic Complexity: Significant shorts/other (<=50%), passed disproof"
            )

    # Branch 4: Heavy Amplification Complexity (High Shorts/Other, Amplified Indicators)
    audit_log.append("Branch 4 Heavy Amplification Complexity: Shorts/other >50%")
    excluded = []
    for cat in ["Slight/None", "Moderate", "Persistent Systematic"]:
        if cat in possible_categories:
            possible_categories.remove(cat)
            excluded.append(cat)
    if excluded:
        audit_log.append(f"Excluded {excluded} due to shorts/other >50%")
    if has_disproof_keywords("Heavy Amplification") or has_disproof_categories("Heavy Amplification"):
        audit_log.append("Disproof triggered, routing to alternative evaluation")
        return None, {}, audit_log
    audit_log.append("No disproof triggered, classifying as Heavy Amplification")
    return (
        "Heavy Amplification",
        "Heavy Amplification Complexity: High shorts/other (>50%), passed disproof"
    )

# Alternative Evaluation Method for Ambiguous Cases
def alternative_evaluation(row, scores, audit_log):
    possible_categories = ["Slight/None", "Moderate", "Persistent Systematic", "Heavy Amplification"]
    audit_log.append("Entering Alternative Evaluation: No definitive classification from exposure tree")

    # Adjust scores based on exposure and category data
    if row['inverse_fund'] in [True, 1]:
        scores["Heavy Amplification"] += 2
        scores["Persistent Systematic"] += 1
        audit_log.append("Inverse fund flag True, boosting Heavy Amplification (+2) and Persistent Systematic (+1)")

    # Score based on categories
    yc_category = safe_lower(row.get('YC_Category_Name', ''))
    cwa_category = safe_lower(row.get('CWA_Broad_Category_Name', ''))
    global_category = safe_lower(row.get('YC_Global_Category_Name', ''))
    return_driver = safe_lower(row.get('return_driver', ''))

    # Score Slight/None
    if (yc_category in ["government bond", "corporate bond", "municipal bond"]
        or cwa_category in ["taxable fixed income", "municipal"]
        or return_driver in ["index based"]):
        scores["Slight/None"] += 2
        audit_log.append("Category scoring for Slight/None: Matched categories/return_driver, +2")

    # Score Moderate
    if (yc_category in ["derivative income", "multisector bond"]
        or cwa_category in ["bond strategy", "strategic"]
        or global_category in ["flexible allocation", "us fixed income"]):
        scores["Moderate"] += 2
        audit_log.append("Category scoring for Moderate: Matched categories/global categories, +2")

    # Score Persistent Systematic
    if (yc_category in ["long-short equity", "equity hedged"]
        or cwa_category in ["alternative", "nontraditional"]
        or global_category in ["long/short equity", "multialternative", "options trading"]
        or return_driver in ["quant/systematic", "active discretionary"]):
        scores["Persistent Systematic"] += 2
        audit_log.append("Category scoring for Persistent Systematic: +2")

    # Score Heavy Amplification
    if (yc_category in ["trading--leveraged equity", "trading--leveraged debt"]
        or "leveraged" in safe_lower(row.get('ProductName', ''))):
        scores["Heavy Amplification"] += 2
        audit_log.append("Category scoring for Heavy Amplification: +2")

    # Evaluate pairs of categories
    audit_log.append("Alternative Evaluation: Evaluating pairs of categories")

    # Pair 1: Slight/None vs Moderate
    pair_scores = {"Slight/None": scores["Slight/None"], "Moderate": scores["Moderate"]}
    audit_log.append(f"Pair 1 (Slight/None vs Moderate) scores: {pair_scores}")
    max_score = max(pair_scores.values())
    top_pair = [cat for cat, sc in pair_scores.items() if sc == max_score]
    if len(top_pair) == 1:
        selected = top_pair[0]
        audit_log.append(f"Alternative Evaluation: Slight/None vs Moderate -> Selected {selected}")
    else:
        keyword_tie_scores = {"Slight/None": 0, "Moderate": 0}
        for category in top_pair:
            for field in ['ProductName', 'investment_strategy', 'FS_insight']:
                keyword_tie_scores[category] += search_keywords(row[field], keyword_mappings[category])
        audit_log.append(f"Pair 1 tiebreaker keyword scores: {keyword_tie_scores}")
        max_keyword_score = max(keyword_tie_scores.values())
        top_keyword_pair = [cat for cat, score in keyword_tie_scores.items() if score == max_keyword_score]
        if len(top_keyword_pair) == 1:
            selected = top_keyword_pair[0]
            audit_log.append(f"Pair 1 tiebreak -> Selected {selected}")
        else:
            # fallback to most conservative (Slight/None < Moderate)
            conservative_order = ["Slight/None", "Moderate"]
            selected = min(top_keyword_pair, key=lambda x: conservative_order.index(x))
            audit_log.append(f"Secondary tiebreak -> Selected {selected}")

    # Double-check next category if necessary
    if selected == "Slight/None":
        if scores["Moderate"] >= scores["Slight/None"]:
            selected = "Moderate"
            audit_log.append(
                f"Double-check: Moderate score ({scores['Moderate']}) >= Slight/None "
                f"({scores['Slight/None']}), selecting Moderate"
            )
    elif selected == "Moderate":
        if scores["Persistent Systematic"] >= scores["Moderate"]:
            selected = "Persistent Systematic"
            audit_log.append(
                f"Double-check: Persistent Systematic score ({scores['Persistent Systematic']}) >= Moderate "
                f"({scores['Moderate']}), selecting Persistent Systematic"
            )
    elif selected == "Persistent Systematic":
        if scores["Heavy Amplification"] >= scores["Persistent Systematic"]:
            selected = "Heavy Amplification"
            audit_log.append(
                f"Double-check: Heavy Amplification score ({scores['Heavy Amplification']}) >= Persistent "
                f"Systematic ({scores['Persistent Systematic']}), selecting Heavy Amplification"
            )

    return selected, audit_log

# Main classification function
def classify_fund(row):
    possible_categories = ["Slight/None", "Moderate", "Persistent Systematic", "Heavy Amplification"]
    scores = {cat: 0 for cat in possible_categories}
    audit_log = []

    # Step 1: Override for Fund Family "Return Stacked ETFs"
    if pd.notna(row['fund_family']) and "return stacked etfs" in row['fund_family'].lower():
        audit_log.append("Override: Fund family 'Return Stacked ETFs' -> Persistent Systematic")
        return "Persistent Systematic", audit_log

    # Step 2: Direct keyword mapping (immediate classification)
    for category, keywords in direct_keyword_mappings.items():
        for field in ['ProductName', 'investment_strategy', 'FS_insight']:
            if pd.notna(row[field]):
                match_count = search_keywords(row[field], keywords)
                if match_count > 0:
                    audit_log.append(f"Direct keyword match for {category} in {field}: {match_count} keywords")
                    return category, audit_log

    # Step 3: Direct category mapping (immediate classification)
    for category, mappings in direct_category_mappings.items():
        for db_field, values in mappings.items():
            values_lower = [val.lower() for val in values]
            field_value = safe_lower(row.get(db_field, ''))
            if field_value in values_lower:
                audit_log.append(f"Direct category match for {category} in {db_field}: {field_value}")
                return category, audit_log

    # Step 4: Apply Boolean exclusions
    for categories, condition in boolean_exclusions.items():
        if condition(row):
            excluded = [cat for cat in possible_categories if cat in categories]
            for cat in excluded:
                possible_categories.remove(cat)
            audit_log.append(f"Excluded {excluded} due to Boolean flags")
            if len(possible_categories) == 1:
                return possible_categories[0], audit_log

    # Step 5: Exposure Decision Tree with Prove/Disprove
    exposure_result = classify_by_exposures_with_disproof(row)
    # If we get a two-element tuple => we have classification + reason
    if isinstance(exposure_result, tuple) and len(exposure_result) == 2:
        classification, reason = exposure_result
        audit_log.append(f"Exposure-based classification: {classification} - {reason}")
        return classification, audit_log
    # If we get None => no classification, let's do alternative
    elif exposure_result[0] is None:
        # exposure_result => (None, scores_dict, audit_log_list)
        scores_update = exposure_result[1]
        for msg in exposure_result[2]:
            audit_log.append(msg)
        for cat in scores_update:
            scores[cat] += scores_update[cat]
        classification, eval_log = alternative_evaluation(row, scores, audit_log)
        for msg in eval_log:
            audit_log.append(msg)
        audit_log.append(f"Alternative Evaluation: Final classification: {classification}")
        return classification, audit_log
    else:
        # Possibly we get (possible_categories, scores_update, exposure_log)
        # but we've now standardized the function to return classification or None
        # We'll keep this in case the structure changes
        possible_categories, scores_update, exposure_log = exposure_result
        for msg in exposure_log:
            audit_log.append(msg)
        for cat in scores_update:
            scores[cat] += scores_update[cat]

    # Step 6: Final Scoring if No Immediate Classification
    max_score = max(scores[cat] for cat in possible_categories)
    top_categories = [cat for cat in possible_categories if scores[cat] == max_score]
    if len(top_categories) == 1:
        final_classification = top_categories[0]
        audit_log.append(f"Final classification: {final_classification} with score {max_score}")
    else:
        keyword_tie_scores = {cat: 0 for cat in top_categories}
        for category in top_categories:
            for field in ['ProductName', 'investment_strategy', 'FS_insight']:
                keyword_tie_scores[category] += search_keywords(row[field], keyword_mappings[category])
        audit_log.append(f"Tiebreaker keyword scores: {keyword_tie_scores}")
        max_keyword_score = max(keyword_tie_scores.values())
        top_keyword_categories = [cat for cat, score in keyword_tie_scores.items() if score == max_keyword_score]
        if len(top_keyword_categories) == 1:
            final_classification = top_keyword_categories[0]
            audit_log.append(
                f"Tiebreaker used - Final classification: {final_classification} with tiebreaker keyword score "
                f"{keyword_tie_scores[final_classification]}"
            )
        else:
            # fallback to conservative ordering
            conservative_order = ["Slight/None", "Moderate", "Persistent Systematic", "Heavy Amplification"]
            final_classification = min(
                top_keyword_categories,
                key=lambda x: conservative_order.index(x)
            )
            audit_log.append(
                f"Secondary tiebreaker used - Final classification: {final_classification} "
                f"(most conservative among {top_keyword_categories})"
            )

    return final_classification, audit_log

# Main script
def main():
    # Create database engine
    engine = create_engine(connection_string)

    # Query to join tables
    query = """
    SELECT 
        f.SymbolCUSIP, f.ProductName, f.fund_family, f.investment_strategy, f.FS_insight,
        f.index_fund, f.inverse_fund, f.leveraged_fund, f.synthetic_replication_fund,
        f.fund_of_funds, f.ycharts_url, f.currency_hedged_fund,
        f.cash_long, f.cash_net, f.cash_short, f.stock_long, f.stock_net, f.stock_short,
        f.bond_long, f.bond_net, f.bond_short, f.other_long, f.other_net, f.other_short,
        f.return_driver, f.YC_BM_Symbol,
        cwa.CWA_Broad_Category_Name,
        yc.Category_Name AS YC_Category_Name,
        ycg.Global_Category_Name,
        ycba.YC_Broad_Asset_Class_Name
    FROM Funds_to_Screen f
    LEFT JOIN CWA_Broad_Category_List cwa ON f.CWA_Broad_Category_ID = cwa.ID
    LEFT JOIN YC_Category_List yc ON f.YC_Category_ID = yc.ID
    LEFT JOIN YC_Global_Category_List ycg ON f.YC_Global_Category_ID = ycg.ID
    LEFT JOIN YC_Broad_Asset_Class_List ycba ON f.YC_Broad_Asset_Class_ID = ycba.ID
    """

    # Load data into DataFrame
    df = pd.read_sql(query, engine)

    # Apply classification
    results = []
    for idx, row in df.iterrows():
        classification, audit_log = classify_fund(row)
        result_row = row.copy()
        result_row['Final_Classification'] = classification
        result_row['Audit_Log'] = "; ".join(audit_log)
        results.append(result_row)

    # Convert results to DataFrame
    result_df = pd.DataFrame(results)

    # Ensure desired column order
    output_columns = ['SymbolCUSIP', 'ProductName', 'fund_family', 'Final_Classification', 'ycharts_url'] + [
        col for col in result_df.columns
        if col not in ['SymbolCUSIP', 'ProductName', 'fund_family', 'Final_Classification', 'ycharts_url']
    ]

    result_df = result_df[output_columns]

    # Export to Excel
    output_path = r"C:\Users\JulianHeron\Software Projects\Risk_Overlays_V4.1.xlsx"
    result_df.to_excel(output_path, index=False)
    print(f"Results exported to {output_path}")

if __name__ == "__main__":
    main()


Results exported to C:\Users\JulianHeron\Software Projects\Risk_Overlays_V4.1.xlsx


In [63]:
# V4.2 with Actual Fix for Numeric Scale (multiplying exposures by 100)
# This ensures that a value of 1.0 is treated as 100%, not 1%.

import pandas as pd
from sqlalchemy import create_engine
import re
import os

# Connection string
connection_string = (
    "mssql+pyodbc://JULIANS_LAPTOP\\SQLEXPRESS/"
    "CWA_Fund_Database?driver=ODBC+Driver+18+for+SQL+Server"
    "&trusted_connection=yes&TrustServerCertificate=yes"
)

def safe_lower(value):
    return value.lower() if isinstance(value, str) else ""

# Define keyword mappings for general scoring
keyword_mappings = {
    "Slight/None": [
        "long-only", "no derivatives", "no hedging", "no leverage", "no options", "no short",
        "position adjustment", "occasional hedging", "covered call", "put-write", "light hedge",
        "may include options", "limited use of derivatives", "for risk management purposes",
        "minor hedging", "occasional short positions", "overwrite", "investment grade", "core"
    ],
    "Moderate": [
        "hedged", "currency hedge", "protective put", "partial hedge", "hedged equity",
        "covered call", "convexity option overlay", "option overlay", "put/spread collar",
        "forward agreement", "enhanced index strategy", "BuyWrite", "Buy-Write", "buy write",
        "option spread", "volatility hedge", "put options", "enhance", "options-based income",
        "ELN", "premium income", "call option", "FLEX options", "option premium", "write calls",
        "sell calls", "protective puts", "equity-linked notes", "structured notes",
        "risk mitigation", "downside protection", "limited hedging", "multi-asset"
    ],
    "Persistent Systematic": [
        "tail-risk", "trend-following", "systematic hedging", "overlay", "CTA", "managed futures",
        "defined outcome", "long-short", "market neutral", "systematic strategy", "return stacking",
        "option writing", "straddle", "derivative income", "futures contracts", "swap contract",
        "forward agreement", "enhanced index strategy", "volatility hedge", "put options",
        "options-based income", "ELN", "option premium", "swap", "forward", "futures", "future",
        "VIX", "managed futures strategy", "trend strategy", "quantitative hedging",
        "systematic options", "options overlay strategy", "futures overlay", "swaps-based",
        "multi-asset", "Flex Options", "Flexible Exchange Options", "YieldMax", "buffer"
    ],
    "Heavy Amplification": [
        "2x", "3x", "Uncapped Accelerator", "-2x", "-3x", "YieldMax"
    ]
}

# Direct mapping keywords (often in ProductName)
direct_keyword_mappings = {
    "Persistent Systematic": ["Market Neutral", "managed futures", "Premia", "Return Stacked ETFs"]
}

# Direct mapping categories
direct_category_mappings = {
    "Persistent Systematic": {
        "YC_Category": ["Defined Outcome"],
        "CWA_Broad_Category": ["Defined Outcome"],
        "YC_Global_Category": ["market neutral"]
    },
    "Heavy Amplification": {
        "YC_Category": [
            "Trading--Leveraged Equity", "Trading--Leveraged Debt", "Trading--Leveraged Commodities"
        ],
        "CWA_Broad_Category": ["Single Stock"]
    }
}

# Helper category mappings (narrow possibilities)
helper_category_mappings = {
    ("Persistent Systematic", "Heavy Amplification"): {
        "YC_Category": [
            "Trading--Inverse Commodities", "Trading--Inverse Debt", "Trading--Inverse Equity",
            "Trading--Miscellaneous"
        ],
        "CWA_Broad_Category": ["Trading/Tactical"],
        "YC_Global_Category": ["Trading Tools"]
    },
    ("Persistent Systematic", "Moderate"): {
        "YC_Global_Category": ["Multialternative", "Long/Short Equity"],
        "YC_Category": ["Equity Hedged"]
    }
}

# Weak helper category mappings (may or may not have overlay)
weak_helper_category_mappings = {
    ("Slight/None", "Moderate"): {
        "YC_Global_Category": ["Flexible Allocation", "Alternative Miscellaneous"],
        "return_driver": ["Index Based", "Factor/Smart Beta"]
    },
    ("Slight/None", "Moderate", "Persistent Systematic"): {
        "YC_Category": ["Relative Value Arbitrage"]
    },
    ("Heavy Amplification", "Persistent Systematic", "Moderate", "Slight/None"): {
        "return_driver": ["Quant/Systematic"]
    },
    ("Persistent Systematic", "Moderate", "Slight/None"): {
        "return_driver": ["Active Discretionary", "Multi-Strategy"]
    }
}

# Boolean flags to remove categories
boolean_exclusions = {
    ("Slight/None", "Moderate"): lambda row: row['leveraged_fund'] in [True, 1]
    or row['synthetic_replication_fund'] in [True, 1]
    or row['inverse_fund'] in [True, 1]
}

# Function to search text for keywords
def search_keywords(text, keywords):
    if pd.isna(text):
        return 0
    text = str(text).lower()
    return sum(1 for keyword in keywords if re.search(r'\b' + re.escape(keyword.lower()) + r'\b', text))

# Decision Tree for Exposure Classification with Prove/Disprove
def classify_by_exposures_with_disproof(row):
    audit_log = []
    possible_categories = ["Slight/None", "Moderate", "Persistent Systematic", "Heavy Amplification"]

    # Convert exposure columns to numeric, handle non-numeric gracefully
    exposure_cols = [
        'cash_long', 'cash_net', 'cash_short',
        'stock_long', 'stock_net', 'stock_short',
        'bond_long', 'bond_net', 'bond_short',
        'other_long', 'other_net', 'other_short'
    ]
    for col in exposure_cols:
        value = pd.to_numeric(row[col], errors='coerce')
        row[col] = 0 if pd.isna(value) else value

    # Calculate totals (incoming data presumably 1.0 => 100%)
    long_total = row['cash_long'] + row['stock_long'] + row['bond_long'] + row['other_long']
    short_total = row['cash_short'] + row['stock_short'] + row['bond_short'] + row['other_short']
    other_total = row['other_long'] + row['other_short']

    # ---- KEY FIX: Multiply by 100 so that 1.0 becomes 100% ----
    long_total *= 100
    short_total *= 100
    other_total *= 100

    # Round totals
    long_total_rounded = round(long_total, 4)
    short_total_rounded = round(short_total, 4)
    other_total_rounded = round(other_total, 4)

    debug_msg = (
        f"[Exposure after *100] long_total={long_total_rounded}%, "
        f"short_total={short_total_rounded}%, other_total={other_total_rounded}%"
    )
    audit_log.append(debug_msg)

    # Define disproof keywords for each branch
    disproof_keywords = {
        "Slight/None": [
            "derivatives", "swaps", "futures", "short", "hedge", "long-short", "inverse", 
            "leveraged", "tail risk", "tail-risk"
        ],
        "Moderate": [
            "systematic", "trend-following", "2x", "3x", "market neutral",
            "quantitative hedging", "managed futures", "managed futures strategy",
            "trend strategy", "yieldmax", "tail risk", "tail-risk"
        ],
        "Persistent Systematic": ["2x", "3x", "-2x", "-3x", "uncapped accelerator"],
        "Heavy Amplification": ["long-only", "no derivatives", "no short"]
    }

    def has_disproof_keywords(category):
        keywords = disproof_keywords.get(category, [])
        keyword_score = 0
        disproof_details = []
        for field in ['ProductName', 'investment_strategy', 'FS_insight']:
            if pd.notna(row[field]):
                text = str(row[field]).lower()
                found_keywords = [
                    kw for kw in keywords
                    if re.search(r'\b' + re.escape(kw.lower()) + r'\b', text)
                ]
                if found_keywords:
                    disproof_details.append(f"Field '{field}' has {found_keywords}")
                    keyword_score += len(found_keywords)
        if keyword_score > 0:
            audit_log.append(f"Disproof Keywords ({category}): {disproof_details}")
        else:
            audit_log.append(f"No Disproof Keywords Found ({category})")
        return keyword_score > 0

    def has_disproof_categories(category):
        # If you need to replicate prior disproof logic, you can do that here.
        # For brevity, returning False unless you want the full checks.
        return False

    # Branch 1: Simple Exposure (Slight/None)
    #
    # short/other < 1 => <1%, and long <= 100.2 => ~100%
    if abs(short_total_rounded) < 1 and abs(other_total_rounded) < 1 and long_total_rounded <= 100.2:
        audit_log.append(
            "Branch 1: Simple Exposure => short/other <1%, long <= ~100%"
        )
        if pd.notna(row['fund_family']) and "return stacked" in row['fund_family'].lower():
            audit_log.append("Disproof: 'return stacked' fund_family => next branch")
        elif has_disproof_keywords("Slight/None") or has_disproof_categories("Slight/None"):
            audit_log.append("Disproof triggered => next branch")
        else:
            audit_log.append("No disproof => Slight/None")
            return "Slight/None", "Simple: short/other <1%, ~100% long"

    # Branch 2: Moderate
    #
    # short/other <= 10 => <=10%
    if short_total_rounded <= 10 and other_total_rounded <= 10:
        audit_log.append("Branch 2: moderate => short/other <=10%")
        if has_disproof_keywords("Moderate") or has_disproof_categories("Moderate"):
            audit_log.append("Disproof triggered => next branch")
        else:
            audit_log.append("No disproof => Moderate")
            return "Moderate", "Moderate: short/other <=10%"

    # Branch 3: Persistent Systematic
    #
    # short/other <= 50 => <=50%
    if short_total_rounded <= 50 or other_total_rounded <= 50:
        audit_log.append("Branch 3: persistent systematic => short/other <=50%")
        return "Persistent Systematic", "Persistent Systematic: short/other <=50%"

    # Branch 4: Heavy Amplification
    #
    audit_log.append("Branch 4: heavy amplification => short/other >50%")
    return "Heavy Amplification", "Heavy Amplification: short/other >50%"

# Alternative evaluation for ambiguous cases
def alternative_evaluation(row, scores, audit_log):
    possible_categories = ["Slight/None", "Moderate", "Persistent Systematic", "Heavy Amplification"]
    audit_log.append("Entering Alternative Evaluation")

    # If it's an inverse fund
    if row['inverse_fund'] in [True, 1]:
        scores["Heavy Amplification"] += 2
        scores["Persistent Systematic"] += 1
        audit_log.append("Inverse fund => +2 Heavy Amplification, +1 Persistent Systematic")

    # Score based on categories
    yc_category = safe_lower(row.get('YC_Category_Name', ''))
    cwa_category = safe_lower(row.get('CWA_Broad_Category_Name', ''))
    global_category = safe_lower(row.get('YC_Global_Category_Name', ''))
    return_driver = safe_lower(row.get('return_driver', ''))

    # Score Slight/None
    if (yc_category in ["government bond", "corporate bond", "municipal bond"]
        or cwa_category in ["taxable fixed income", "municipal"]
        or return_driver in ["index based"]):
        scores["Slight/None"] += 2
        audit_log.append("Scoring +2 for Slight/None based on category/return_driver")

    # Score Moderate
    if (yc_category in ["derivative income", "multisector bond"]
        or cwa_category in ["bond strategy", "strategic"]
        or global_category in ["flexible allocation", "us fixed income"]):
        scores["Moderate"] += 2
        audit_log.append("Scoring +2 for Moderate")

    # Score Persistent Systematic
    if (yc_category in ["long-short equity", "equity hedged"]
        or cwa_category in ["alternative", "nontraditional"]
        or global_category in ["long/short equity", "multialternative", "options trading"]
        or return_driver in ["quant/systematic", "active discretionary"]):
        scores["Persistent Systematic"] += 2
        audit_log.append("Scoring +2 for Persistent Systematic")

    # Score Heavy Amplification
    if (yc_category in ["trading--leveraged equity", "trading--leveraged debt"]
        or "leveraged" in safe_lower(row.get('ProductName', ''))):
        scores["Heavy Amplification"] += 2
        audit_log.append("Scoring +2 for Heavy Amplification")

    # Evaluate pairs of categories
    audit_log.append("Evaluating pairs to see if a single category emerges...")

    # For example, you can do your pairwise logic here. 
    # We'll keep a simplified version that picks the max among the four.
    # Then use a tiebreaker on keywords.

    return None, audit_log  # or your logic to choose one

def classify_fund(row):
    possible_categories = ["Slight/None", "Moderate", "Persistent Systematic", "Heavy Amplification"]
    scores = {cat: 0 for cat in possible_categories}
    audit_log = []

    # Step 1: Override for Fund Family "Return Stacked ETFs"
    if pd.notna(row['fund_family']) and "return stacked etfs" in row['fund_family'].lower():
        audit_log.append("Override => 'Return Stacked ETFs' => Persistent Systematic")
        return "Persistent Systematic", audit_log

    # Step 2: Direct keyword mapping
    for category, keywords in direct_keyword_mappings.items():
        for field in ['ProductName', 'investment_strategy', 'FS_insight']:
            if pd.notna(row[field]):
                match_count = search_keywords(row[field], keywords)
                if match_count > 0:
                    audit_log.append(f"Direct keyword => {category} in {field}")
                    return category, audit_log

    # Step 3: Direct category mapping
    for category, mappings in direct_category_mappings.items():
        for db_field, values in mappings.items():
            values_lower = [val.lower() for val in values]
            field_value = safe_lower(row.get(db_field, ''))
            if field_value in values_lower:
                audit_log.append(f"Direct category => {category} for {field_value} in {db_field}")
                return category, audit_log

    # Step 4: Apply Boolean exclusions
    for categories, condition in boolean_exclusions.items():
        if condition(row):
            excluded = [cat for cat in possible_categories if cat in categories]
            for cat in excluded:
                possible_categories.remove(cat)
            audit_log.append(f"Excluded {excluded} due to Boolean flags")
            if len(possible_categories) == 1:
                return possible_categories[0], audit_log

    # Step 5: Exposure Decision Tree
    exposure_result = classify_by_exposures_with_disproof(row)

    if isinstance(exposure_result, tuple) and len(exposure_result) == 2:
        classification, reason = exposure_result
        audit_log.append(f"Exposure-based => {classification} - {reason}")
        return classification, audit_log

    # If None or ambiguous, do alternative
    elif exposure_result[0] is None:
        scores_update = exposure_result[1]
        for msg in exposure_result[2]:
            audit_log.append(msg)
        for cat in scores_update:
            scores[cat] += scores_update[cat]
        classification, eval_log = alternative_evaluation(row, scores, audit_log)
        for msg in eval_log:
            audit_log.append(msg)
        if classification:
            audit_log.append(f"Alternative => {classification}")
            return classification, audit_log

    # Step 6: Final scoring if no immediate classification
    max_score = max(scores[cat] for cat in possible_categories)
    top_categories = [cat for cat in possible_categories if scores[cat] == max_score]
    if len(top_categories) == 1:
        final_classification = top_categories[0]
        audit_log.append(f"Final => {final_classification}")
    else:
        # Tiebreaker on keywords
        keyword_tie_scores = {cat: 0 for cat in top_categories}
        for category in top_categories:
            for field in ['ProductName', 'investment_strategy', 'FS_insight']:
                keyword_tie_scores[category] += search_keywords(row[field], keyword_mappings[category])
        audit_log.append(f"Tiebreaker => {keyword_tie_scores}")
        max_keyword_score = max(keyword_tie_scores.values())
        top_keyword_categories = [
            cat for cat, score in keyword_tie_scores.items()
            if score == max_keyword_score
        ]
        if len(top_keyword_categories) == 1:
            final_classification = top_keyword_categories[0]
            audit_log.append(f"Tiebreaker => {final_classification}")
        else:
            # fallback to conservative
            conservative_order = ["Slight/None", "Moderate", "Persistent Systematic", "Heavy Amplification"]
            final_classification = min(
                top_keyword_categories,
                key=lambda x: conservative_order.index(x)
            )
            audit_log.append(f"Conservative fallback => {final_classification}")

    return final_classification, audit_log

def main():
    # Create database engine
    engine = create_engine(connection_string)

    # Query to join tables
    query = """
    SELECT 
        f.SymbolCUSIP, f.ProductName, f.fund_family, f.investment_strategy, f.FS_insight,
        f.index_fund, f.inverse_fund, f.leveraged_fund, f.synthetic_replication_fund,
        f.fund_of_funds, f.ycharts_url, f.currency_hedged_fund,
        f.cash_long, f.cash_net, f.cash_short, f.stock_long, f.stock_net, f.stock_short,
        f.bond_long, f.bond_net, f.bond_short, f.other_long, f.other_net, f.other_short,
        f.return_driver, f.YC_BM_Symbol,
        cwa.CWA_Broad_Category_Name,
        yc.Category_Name AS YC_Category_Name,
        ycg.Global_Category_Name,
        ycba.YC_Broad_Asset_Class_Name
    FROM Funds_to_Screen f
    LEFT JOIN CWA_Broad_Category_List cwa ON f.CWA_Broad_Category_ID = cwa.ID
    LEFT JOIN YC_Category_List yc ON f.YC_Category_ID = yc.ID
    LEFT JOIN YC_Global_Category_List ycg ON f.YC_Global_Category_ID = ycg.ID
    LEFT JOIN YC_Broad_Asset_Class_List ycba ON f.YC_Broad_Asset_Class_ID = ycba.ID
    """

    df = pd.read_sql(query, engine)

    # Apply classification
    results = []
    for idx, row in df.iterrows():
        classification, audit_log = classify_fund(row)
        result_row = row.copy()
        result_row['Final_Classification'] = classification
        result_row['Audit_Log'] = "; ".join(audit_log)
        results.append(result_row)

    result_df = pd.DataFrame(results)

    # Ensure desired column order
    output_columns = [
        'SymbolCUSIP', 'ProductName', 'fund_family', 'Final_Classification', 'ycharts_url'
    ] + [
        col for col in result_df.columns
        if col not in ['SymbolCUSIP', 'ProductName', 'fund_family', 'Final_Classification', 'ycharts_url']
    ]

    result_df = result_df[output_columns]

    # Export to Excel - V4.2
    output_path = r"C:\Users\JulianHeron\Software Projects\Risk_Overlays_V4.2T2.xlsx"
    result_df.to_excel(output_path, index=False)
    print(f"Results exported to {output_path}")

if __name__ == "__main__":
    main()


Results exported to C:\Users\JulianHeron\Software Projects\Risk_Overlays_V4.2T2.xlsx


In [60]:
# This code seems stable, test then run in DB
# V4.2 with Extra Debug Logging for Exposures
import pandas as pd
from sqlalchemy import create_engine
import re
import os

connection_string = (
    "mssql+pyodbc://JULIANS_LAPTOP\\SQLEXPRESS/"
    "CWA_Fund_Database?driver=ODBC+Driver+18+for+SQL+Server"
    "&trusted_connection=yes&TrustServerCertificate=yes"
)

def safe_lower(value):
    return value.lower() if isinstance(value, str) else ""

keyword_mappings = {
    "Slight/None": [
        "long-only", "no derivatives", "no hedging", "no leverage", "no options", "no short",
        "position adjustment", "occasional hedging", "covered call", "put-write", "light hedge",
        "may include options", "limited use of derivatives", "for risk management purposes",
        "minor hedging", "occasional short positions", "overwrite", "investment grade", "core"
    ],
    "Moderate": [
        "hedged", "currency hedge", "protective put", "partial hedge", "hedged equity",
        "covered call", "convexity option overlay", "option overlay", "put/spread collar",
        "forward agreement", "enhanced index strategy", "BuyWrite", "Buy-Write", "buy write",
        "option spread", "volatility hedge", "put options", "enhance", "options-based income",
        "ELN", "premium income", "call option", "FLEX options", "option premium", "write calls",
        "sell calls", "protective puts", "equity-linked notes", "structured notes",
        "risk mitigation", "downside protection", "limited hedging", "multi-asset"
    ],
    "Persistent Systematic": [
        "tail-risk", "trend-following", "systematic hedging", "overlay", "CTA", "managed futures",
        "defined outcome", "long-short", "market neutral", "systematic strategy", "return stacking",
        "option writing", "straddle", "derivative income", "futures contracts", "swap contract",
        "forward agreement", "enhanced index strategy", "volatility hedge", "put options",
        "options-based income", "ELN", "option premium", "swap", "forward", "futures", "future",
        "VIX", "managed futures strategy", "trend strategy", "quantitative hedging",
        "systematic options", "options overlay strategy", "futures overlay", "swaps-based",
        "multi-asset", "Flex Options", "Flexible Exchange Options", "YieldMax", "buffer"
    ],
    "Heavy Amplification": [
        "2x", "3x", "Uncapped Accelerator", "-2x", "-3x", "YieldMax"
    ]
}

direct_keyword_mappings = {
    "Persistent Systematic": ["Market Neutral", "managed futures", "Premia", "Return Stacked ETFs"]
}

direct_category_mappings = {
    "Persistent Systematic": {
        "YC_Category": ["Defined Outcome"],
        "CWA_Broad_Category": ["Defined Outcome"],
        "YC_Global_Category": ["market neutral"]
    },
    "Heavy Amplification": {
        "YC_Category": [
            "Trading--Leveraged Equity", "Trading--Leveraged Debt", "Trading--Leveraged Commodities"
        ],
        "CWA_Broad_Category": ["Single Stock"]
    }
}

boolean_exclusions = {
    ("Slight/None", "Moderate"): lambda row: row['leveraged_fund'] in [True, 1]
    or row['synthetic_replication_fund'] in [True, 1]
    or row['inverse_fund'] in [True, 1]
}

def search_keywords(text, keywords):
    if pd.isna(text):
        return 0
    text = str(text).lower()
    return sum(1 for keyword in keywords if re.search(r'\b' + re.escape(keyword.lower()) + r'\b', text))

def classify_by_exposures_with_disproof(row):
    audit_log = []

    # Extra debug: Show the raw values from the row *before* we do anything
    audit_log.append(
        f"DEBUG: Symbol={row['SymbolCUSIP']}, "
        f"cash_long={row['cash_long']}, cash_net={row['cash_net']}, cash_short={row['cash_short']}, "
        f"stock_long={row['stock_long']}, stock_net={row['stock_net']}, stock_short={row['stock_short']}, "
        f"bond_long={row['bond_long']}, bond_net={row['bond_net']}, bond_short={row['bond_short']}, "
        f"other_long={row['other_long']}, other_net={row['other_net']}, other_short={row['other_short']}"
    )

    possible_categories = ["Slight/None", "Moderate", "Persistent Systematic", "Heavy Amplification"]

    # Convert to numeric
    exposure_cols = [
        'cash_long', 'cash_net', 'cash_short',
        'stock_long', 'stock_net', 'stock_short',
        'bond_long', 'bond_net', 'bond_short',
        'other_long', 'other_net', 'other_short'
    ]
    for col in exposure_cols:
        value = pd.to_numeric(row[col], errors='coerce')
        row[col] = 0 if pd.isna(value) else value

    # Calculate totals
    long_total = row['cash_long'] + row['stock_long'] + row['bond_long'] + row['other_long']
    short_total = row['cash_short'] + row['stock_short'] + row['bond_short'] + row['other_short']
    other_total = row['other_long'] + row['other_short']

    # Multiply by 100 so that 1.0 => 100%
    long_total *= 100
    short_total *= 100
    other_total *= 100

    # Round
    long_total_rounded = round(long_total, 4)
    short_total_rounded = round(short_total, 4)
    other_total_rounded = round(other_total, 4)

    audit_log.append(
        f"[After *100] long_total={long_total_rounded}%, short_total={short_total_rounded}%, other_total={other_total_rounded}%"
    )

    # Disproof keywords as before
    disproof_keywords = {
        "Slight/None": [
            "derivatives", "swaps", "futures", "short", "hedge", "long-short", "inverse", 
            "leveraged", "tail risk", "tail-risk"
        ],
        "Moderate": [
            "systematic", "trend-following", "2x", "3x", "market neutral",
            "quantitative hedging", "managed futures", "managed futures strategy",
            "trend strategy", "yieldmax", "tail risk", "tail-risk"
        ],
        "Persistent Systematic": ["2x", "3x", "-2x", "-3x", "uncapped accelerator"],
        "Heavy Amplification": ["long-only", "no derivatives", "no short"]
    }

    def has_disproof_keywords(category):
        # ...
        return False

    def has_disproof_categories(category):
        # ...
        return False

    # Branch 1: Slight/None
    if abs(short_total_rounded) < 1 and abs(other_total_rounded) < 1 and long_total_rounded <= 100.2:
        audit_log.append("Branch 1 => Slight/None")
        return "Slight/None", "Simple"

    # Branch 2: Moderate
    if short_total_rounded <= 10 and other_total_rounded <= 10:
        audit_log.append("Branch 2 => Moderate (<=10% short/other)")
        return "Moderate", "Moderate"

    # Branch 3: Persistent Systematic
    if short_total_rounded <= 50 or other_total_rounded <= 50:
        audit_log.append("Branch 3 => Persistent Systematic (<=50% short/other)")
        return "Persistent Systematic", "Persistent Systematic"

    # Branch 4: Heavy Amplification
    audit_log.append("Branch 4 => Heavy Amplification (>50% short/other)")
    return "Heavy Amplification", "Heavy Amplification"

def classify_fund(row):
    audit_log = []

    # Potential direct checks / boolean exclusions omitted for brevity
    classification, detail = classify_by_exposures_with_disproof(row)
    audit_log.append(f"Exposure-based => {classification}: {detail}")
    return classification, audit_log

def main():
    engine = create_engine(connection_string)

    query = """
    SELECT 
        f.SymbolCUSIP, f.ProductName, f.fund_family, f.investment_strategy, f.FS_insight,
        f.index_fund, f.inverse_fund, f.leveraged_fund, f.synthetic_replication_fund,
        f.fund_of_funds, f.ycharts_url, f.currency_hedged_fund,
        f.cash_long, f.cash_net, f.cash_short, f.stock_long, f.stock_net, f.stock_short,
        f.bond_long, f.bond_net, f.bond_short, f.other_long, f.other_net, f.other_short,
        f.return_driver, f.YC_BM_Symbol,
        cwa.CWA_Broad_Category_Name,
        yc.Category_Name AS YC_Category_Name,
        ycg.Global_Category_Name,
        ycba.YC_Broad_Asset_Class_Name
    FROM Funds_to_Screen f
    LEFT JOIN CWA_Broad_Category_List cwa ON f.CWA_Broad_Category_ID = cwa.ID
    LEFT JOIN YC_Category_List yc ON f.YC_Category_ID = yc.ID
    LEFT JOIN YC_Global_Category_List ycg ON f.YC_Global_Category_ID = ycg.ID
    LEFT JOIN YC_Broad_Asset_Class_List ycba ON f.YC_Broad_Asset_Class_ID = ycba.ID
    """

    df = pd.read_sql(query, engine)

    results = []
    for idx, row in df.iterrows():
        classification, local_log = classify_fund(row)
        newrow = row.copy()
        newrow['Final_Classification'] = classification
        newrow['Audit_Log'] = "; ".join(local_log)
        results.append(newrow)

    outdf = pd.DataFrame(results)

    columns_front = [
        'SymbolCUSIP','ProductName','fund_family','Final_Classification','ycharts_url'
    ]
    other_cols = [c for c in outdf.columns if c not in columns_front]
    outdf = outdf[columns_front + other_cols]

    # v4.2 with debug
    output_path = r"C:\Users\JulianHeron\Software Projects\Risk_Overlays_V4.2_DebugT3.xlsx"
    outdf.to_excel(output_path, index=False)
    print(f"Exported debug version to {output_path}")

if __name__ == "__main__":
    main()


Exported debug version to C:\Users\JulianHeron\Software Projects\Risk_Overlays_V4.2_DebugT3.xlsx


In [61]:
# extended debugging before finalizing

import pandas as pd
from sqlalchemy import create_engine
import re
import os

# Connection string
connection_string = (
    "mssql+pyodbc://JULIANS_LAPTOP\\SQLEXPRESS/"
    "CWA_Fund_Database?driver=ODBC+Driver+18+for+SQL+Server"
    "&trusted_connection=yes&TrustServerCertificate=yes"
)

def safe_lower(value):
    return value.lower() if isinstance(value, str) else ""

keyword_mappings = {
    "Slight/None": [
        "long-only", "no derivatives", "no hedging", "no leverage", "no options", "no short",
        "position adjustment", "occasional hedging", "covered call", "put-write", "light hedge",
        "may include options", "limited use of derivatives", "for risk management purposes",
        "minor hedging", "occasional short positions", "overwrite", "investment grade", "core"
    ],
    "Moderate": [
        "hedged", "currency hedge", "protective put", "partial hedge", "hedged equity",
        "covered call", "convexity option overlay", "option overlay", "put/spread collar",
        "forward agreement", "enhanced index strategy", "BuyWrite", "Buy-Write", "buy write",
        "option spread", "volatility hedge", "put options", "enhance", "options-based income",
        "ELN", "premium income", "call option", "FLEX options", "option premium", "write calls",
        "sell calls", "protective puts", "equity-linked notes", "structured notes",
        "risk mitigation", "downside protection", "limited hedging", "multi-asset"
    ],
    "Persistent Systematic": [
        "tail-risk", "trend-following", "systematic hedging", "overlay", "CTA", "managed futures",
        "defined outcome", "long-short", "market neutral", "systematic strategy", "return stacking",
        "option writing", "straddle", "derivative income", "futures contracts", "swap contract",
        "forward agreement", "enhanced index strategy", "volatility hedge", "put options",
        "options-based income", "ELN", "option premium", "swap", "forward", "futures", "future",
        "VIX", "managed futures strategy", "trend strategy", "quantitative hedging",
        "systematic options", "options overlay strategy", "futures overlay", "swaps-based",
        "multi-asset", "Flex Options", "Flexible Exchange Options", "YieldMax", "buffer"
    ],
    "Heavy Amplification": [
        "2x", "3x", "Uncapped Accelerator", "-2x", "-3x", "YieldMax"
    ]
}

# Direct keyword mappings (immediate classification)
direct_keyword_mappings = {
    "Persistent Systematic": [
        "Market Neutral", 
        "managed futures", 
        "Premia", 
        "Return Stacked ETFs"
    ]
}

# Direct category mappings (immediate classification)
direct_category_mappings = {
    "Persistent Systematic": {
        "YC_Category": ["Defined Outcome"],
        "CWA_Broad_Category": ["Defined Outcome"],
        "YC_Global_Category": ["market neutral"]
    },
    "Heavy Amplification": {
        "YC_Category": [
            "Trading--Leveraged Equity", 
            "Trading--Leveraged Debt", 
            "Trading--Leveraged Commodities"
        ],
        "CWA_Broad_Category": ["Single Stock"]
    }
}

# Boolean flags to remove categories
boolean_exclusions = {
    ("Slight/None", "Moderate"): lambda row: row['leveraged_fund'] in [True, 1]
        or row['synthetic_replication_fund'] in [True, 1]
        or row['inverse_fund'] in [True, 1]
}

def search_keywords(text, keywords):
    if pd.isna(text):
        return 0
    text = str(text).lower()
    return sum(1 for kw in keywords if re.search(r'\b' + re.escape(kw.lower()) + r'\b', text))

def classify_by_exposures_with_disproof(row, master_log):
    """
    Extended-logging version:
    master_log is the same list we pass around to accumulate all messages,
    so we can see each branch check.
    """
    master_log.append("===== BEGIN EXPOSURE DECISION TREE =====")
    
    # Convert to numeric
    exposure_cols = [
        'cash_long', 'cash_net', 'cash_short',
        'stock_long', 'stock_net', 'stock_short',
        'bond_long', 'bond_net', 'bond_short',
        'other_long', 'other_net', 'other_short'
    ]
    for col in exposure_cols:
        val = pd.to_numeric(row[col], errors='coerce')
        row[col] = 0 if pd.isna(val) else val

    # Sum
    long_total = (row['cash_long'] + row['stock_long'] 
                  + row['bond_long'] + row['other_long'])
    short_total = (row['cash_short'] + row['stock_short'] 
                   + row['bond_short'] + row['other_short'])
    other_total = (row['other_long'] + row['other_short'])

    # Multiply by 100 => 1.0 => 100%
    long_total_100 = long_total * 100
    short_total_100 = short_total * 100
    other_total_100 = other_total * 100

    # Round
    long_r = round(long_total_100, 4)
    short_r = round(short_total_100, 4)
    other_r = round(other_total_100, 4)

    master_log.append(f"Exposures => long={long_r}%, short={short_r}%, other={other_r}%")

    # Branch 1: "Slight/None"
    master_log.append(
        f"Branch 1 check: short<{1}?, other<{1}?, long<={100.2}? => short={short_r}, other={other_r}, long={long_r}"
    )
    if abs(short_r) < 1 and abs(other_r) < 1 and long_r <= 100.2:
        master_log.append("=> PASSED => 'Slight/None' (simple long-only exposure)")
        return "Slight/None", "Simple"

    master_log.append("=> FAILED => next branch...")

    # Branch 2: "Moderate"
    master_log.append(
        f"Branch 2 check: short<={10}?, other<={10}? => short={short_r}, other={other_r}"
    )
    if short_r <= 10 and other_r <= 10:
        master_log.append("=> PASSED => 'Moderate'")
        return "Moderate", "Moderate"

    master_log.append("=> FAILED => next branch...")

    # Branch 3: "Persistent Systematic"
    master_log.append(
        f"Branch 3 check: short<={50}? or other<={50}? => short={short_r}, other={other_r}"
    )
    if short_r <= 50 or other_r <= 50:
        master_log.append("=> PASSED => 'Persistent Systematic'")
        return "Persistent Systematic", "Persistent Systematic"

    master_log.append("=> FAILED => next branch...")

    # Branch 4: "Heavy Amplification"
    master_log.append(
        f"Branch 4 => short/other > 50% => => 'Heavy Amplification'"
    )
    return "Heavy Amplification", "Heavy Amplification"

def alternative_evaluation(row, scores, master_log):
    # If you have alternative logic, put extended logging here.
    master_log.append("===== BEGIN ALTERNATIVE EVALUATION =====")
    # Return (None, master_log) or a final classification with reason
    return (None, master_log)

def classify_fund(row):
    audit_log = []

    # 1) Fund family override
    if pd.notna(row['fund_family']) and ("return stacked etfs" in row['fund_family'].lower()):
        audit_log.append(
            "FUND FAMILY OVERRIDE => 'Return Stacked ETFs' => Persistent Systematic"
        )
        return "Persistent Systematic", audit_log

    # 2) Direct keyword mapping
    for category, keywords in direct_keyword_mappings.items():
        for field in ['ProductName', 'investment_strategy', 'FS_insight']:
            if pd.notna(row[field]):
                count = search_keywords(row[field], keywords)
                if count > 0:
                    audit_log.append(f"DIRECT KEYWORD => {category} from {field} match")
                    return category, audit_log

    # 3) Direct category mapping
    for category, mappings in direct_category_mappings.items():
        for db_field, values in mappings.items():
            vals_lower = [v.lower() for v in values]
            field_val = safe_lower(row.get(db_field, ''))
            if field_val in vals_lower:
                audit_log.append(f"DIRECT CATEGORY => {category} from {db_field}={field_val}")
                return category, audit_log

    # 4) Boolean exclusions
    possible_categories = ["Slight/None", "Moderate", "Persistent Systematic", "Heavy Amplification"]
    for cats_to_exclude, condition_fn in boolean_exclusions.items():
        if condition_fn(row):
            # i.e. if fund is leveraged => exclude "Slight/None" and "Moderate"
            excluded = [c for c in cats_to_exclude if c in possible_categories]
            for c in excluded:
                possible_categories.remove(c)
            audit_log.append(
                f"BOOLEAN EXCLUSION => Excluding {excluded} for leveraged/inverse/synthetic"
            )
            if len(possible_categories) == 1:
                # Only one left
                final_cat = possible_categories[0]
                audit_log.append(f"Only {final_cat} remains => final classification")
                return final_cat, audit_log

    # 5) Exposure-based classification
    classification, reason = classify_by_exposures_with_disproof(row, audit_log)
    audit_log.append(f"Exposure-based => {classification} => {reason}")
    return classification, audit_log

def main():
    engine = create_engine(connection_string)

    query = """
    SELECT 
        f.SymbolCUSIP, f.ProductName, f.fund_family, f.investment_strategy, f.FS_insight,
        f.index_fund, f.inverse_fund, f.leveraged_fund, f.synthetic_replication_fund,
        f.fund_of_funds, f.ycharts_url, f.currency_hedged_fund,
        f.cash_long, f.cash_net, f.cash_short, f.stock_long, f.stock_net, f.stock_short,
        f.bond_long, f.bond_net, f.bond_short, f.other_long, f.other_net, f.other_short,
        f.return_driver, f.YC_BM_Symbol,
        cwa.CWA_Broad_Category_Name,
        yc.Category_Name AS YC_Category_Name,
        ycg.Global_Category_Name,
        ycba.YC_Broad_Asset_Class_Name
    FROM Funds_to_Screen f
    LEFT JOIN CWA_Broad_Category_List cwa ON f.CWA_Broad_Category_ID = cwa.ID
    LEFT JOIN YC_Category_List yc ON f.YC_Category_ID = yc.ID
    LEFT JOIN YC_Global_Category_List ycg ON f.YC_Global_Category_ID = ycg.ID
    LEFT JOIN YC_Broad_Asset_Class_List ycba ON f.YC_Broad_Asset_Class_ID = ycba.ID
    """

    df = pd.read_sql(query, engine)

    results = []
    for idx, row in df.iterrows():
        classification, local_log = classify_fund(row)
        newrow = row.copy()
        newrow['Final_Classification'] = classification
        newrow['Audit_Log'] = "; ".join(local_log)
        results.append(newrow)

    outdf = pd.DataFrame(results)

    # Order columns
    columns_front = [
        'SymbolCUSIP','ProductName','fund_family','Final_Classification','ycharts_url'
    ]
    other_cols = [c for c in outdf.columns if c not in columns_front]
    outdf = outdf[columns_front + other_cols]

    output_path = r"C:\Users\JulianHeron\Software Projects\Risk_Overlays_V4.3_ExtendedLog.xlsx"
    outdf.to_excel(output_path, index=False)
    print(f"Exported v4.3 Extended Logging => {output_path}")

if __name__ == "__main__":
    main()


Exported v4.3 Extended Logging => C:\Users\JulianHeron\Software Projects\Risk_Overlays_V4.3_ExtendedLog.xlsx


In [65]:
# Extended Debugging, but adding back in proof/ disproof logic in tier one exposures

# extended debugging before finalizing

import pandas as pd
from sqlalchemy import create_engine
import re
import os

# Connection string
connection_string = (
    "mssql+pyodbc://JULIANS_LAPTOP\\SQLEXPRESS/"
    "CWA_Fund_Database?driver=ODBC+Driver+18+for+SQL+Server"
    "&trusted_connection=yes&TrustServerCertificate=yes"
)

def safe_lower(value):
    return value.lower() if isinstance(value, str) else ""

keyword_mappings = {
    "Slight/None": [
        "long-only", "no derivatives", "no hedging", "no leverage", "no options", "no short",
        "position adjustment", "occasional hedging", "covered call", "put-write", "light hedge",
        "may include options", "limited use of derivatives", "for risk management purposes",
        "minor hedging", "occasional short positions", "overwrite", "investment grade", "core"
    ],
    "Moderate": [
        "hedged", "currency hedge", "protective put", "partial hedge", "hedged equity",
        "covered call", "convexity option overlay", "option overlay", "put/spread collar",
        "forward agreement", "enhanced index strategy", "BuyWrite", "Buy-Write", "buy write",
        "option spread", "volatility hedge", "put options", "enhance", "options-based income",
        "ELN", "premium income", "call option", "FLEX options", "option premium", "write calls",
        "sell calls", "protective puts", "equity-linked notes", "structured notes",
        "risk mitigation", "downside protection", "limited hedging", "multi-asset"
    ],
    "Persistent Systematic": [
        "tail-risk", "trend-following", "systematic hedging", "overlay", "CTA", "managed futures",
        "defined outcome", "long-short", "market neutral", "systematic strategy", "return stacking",
        "option writing", "straddle", "derivative income", "futures contracts", "swap contract",
        "forward agreement", "enhanced index strategy", "volatility hedge", "put options",
        "options-based income", "ELN", "option premium", "swap", "forward", "futures", "future",
        "VIX", "managed futures strategy", "trend strategy", "quantitative hedging",
        "systematic options", "options overlay strategy", "futures overlay", "swaps-based",
        "multi-asset", "Flex Options", "Flexible Exchange Options", "YieldMax", "buffer"
    ],
    "Heavy Amplification": [
        "2x", "3x", "Uncapped Accelerator", "-2x", "-3x", "YieldMax"
    ]
}

# Proof/ Disproof definitions for tier 1
slight_proof_phrases = [
    "used for minor duration or risk tweaks",
    "occasional use for limited exposure adjustments",
    "used for position adjustments on a case-by-case basis",
    "applied sparingly to fine-tune risk",
    "used on an ad hoc basis for hedging",
    "employed occasionally for cash management",
    "derivatives used optionally for risk",
    "can utilize swaps for adjustments",
    "can use futures to track index",
    "can employ derivatives occasionally",
    "derivatives used discretionarily",
    "derivatives permitted for limited purposes",
    "may invest in derivatives sparingly",
    "may employ futures for cash flow",
    "uses derivatives to adjust exposure",
    "utilizes futures contracts to equitize cash",
    "will not use it to increase leveraged exposure",
    "use of derivatives is permitted within limits",
    "may use derivatives",
    "may invest in derivatives",
    "derivatives only to mitigate",
    "may utilize derivatives for managing duration, sector exposure, yield curve and risk mitigation",
    "may utilize derivatives for managing duration",
    "may invest in derivatives, including foreign currency derivatives",
    "may also invest in futures contracts and options to manage market exposure",
    "may use derivatives to leverage exposure or manage cash",
    "may also use derivatives to leverage or hedge exposure",
    "may hedge foreign currency exposure through derivatives, although it is not required to do so"
]

slight_disproof_phrases = [
    "employs currency forward contracts to hedge exposure",
    "derivatives are integral to its hedging strategy",
    "systematically uses derivatives",
    "uses a quantitative model to generate derivative signals",
    "systematic use of derivatives",
    "derivatives are central",
    "invests primarily in futures, call options, and put options",
    "hedges currency exposure with derivatives",
    "writes call options on index",
    "invests in futures to offset risk",
    "enters swap transactions for protection",
    "uses futures to enhance exposure",
    "invests in derivatives through subsidiary",
    "hedges interest rates with options",
    "allocates assets to options strategy",
    "employs leverage through inverse floaters",
    "employs options strategies regularly",
    "rolled according to a fixed schedule",
    "currency‐related derivatives to hedge",
    "may invest up to 15% of its total assets in credit default swaps",
    "applies an options collar strategy",
    "derivatives to hedge currency exposure",
    "writing covered calls",
    "selects put options through a laddered approach that rolls monthly"
]


# Direct keyword mappings (immediate classification)
direct_keyword_mappings = {
    "Persistent Systematic": [
        "Market Neutral", 
        "managed futures", 
        "Premia", 
        "Return Stacked ETFs"
    ]
}

# Direct category mappings (immediate classification)
direct_category_mappings = {
    "Persistent Systematic": {
        "YC_Category": ["Defined Outcome"],
        "CWA_Broad_Category": ["Defined Outcome"],
        "YC_Global_Category": ["market neutral"]
    },
    "Heavy Amplification": {
        "YC_Category": [
            "Trading--Leveraged Equity", 
            "Trading--Leveraged Debt", 
            "Trading--Leveraged Commodities"
        ],
        "CWA_Broad_Category": ["Single Stock"]
    }
}

# Boolean flags to remove categories
boolean_exclusions = {
    ("Slight/None", "Moderate"): lambda row: row['leveraged_fund'] in [True, 1]
        or row['synthetic_replication_fund'] in [True, 1]
        or row['inverse_fund'] in [True, 1]
}

# Function to define Proof/ Disproof for tier 1
def tally_slight_none_proof_disproof(row):
    """
    Returns (proof_score, disproof_score) by checking for the presence 
    of each phrase in [ProductName, investment_strategy, FS_insight].
    Ties go to 'Slight/None'.
    """
    proof_score = 0
    disproof_score = 0

    fields_to_check = ["ProductName", "investment_strategy", "FS_insight"]
    for field in fields_to_check:
        if pd.notna(row[field]):
            text = row[field].lower()

            for phrase in slight_proof_phrases:
                if phrase in text:
                    proof_score += 1

            for phrase in slight_disproof_phrases:
                if phrase in text:
                    disproof_score += 1

    return proof_score, disproof_score


# Function to search keywords
def search_keywords(text, keywords):
    if pd.isna(text):
        return 0
    text = str(text).lower()
    return sum(1 for kw in keywords if re.search(r'\b' + re.escape(kw.lower()) + r'\b', text))

#Function to classify exposures and check for disproof
def classify_by_exposures_with_disproof(row, master_log):
    """
    Extended-logging version:
    master_log is the same list we pass around to accumulate all messages,
    so we can see each branch check.
    """
    master_log.append("===== BEGIN EXPOSURE DECISION TREE =====")
    
    # Convert to numeric
    exposure_cols = [
        'cash_long', 'cash_net', 'cash_short',
        'stock_long', 'stock_net', 'stock_short',
        'bond_long', 'bond_net', 'bond_short',
        'other_long', 'other_net', 'other_short'
    ]
    for col in exposure_cols:
        val = pd.to_numeric(row[col], errors='coerce')
        row[col] = 0 if pd.isna(val) else val

    # Sum
    long_total = (row['cash_long'] + row['stock_long'] 
                  + row['bond_long'] + row['other_long'])
    short_total = (row['cash_short'] + row['stock_short'] 
                   + row['bond_short'] + row['other_short'])
    other_total = (row['other_long'] + row['other_short'])

    # Multiply by 100 => 1.0 => 100%
    long_total_100 = long_total * 100
    short_total_100 = short_total * 100
    other_total_100 = other_total * 100

    # Round
    long_r = round(long_total_100, 4)
    short_r = round(short_total_100, 4)
    other_r = round(other_total_100, 4)

    master_log.append(f"Exposures => long={long_r}%, short={short_r}%, other={other_r}%")

    # Branch 1: "Slight/None" with proof vs. disproof
    master_log.append(
        f"Branch 1 check: short<{1}?, other<{1}?, long<={100.2}? => short={short_r}, other={other_r}, long={long_r}"
    )
    if abs(short_r) < 1 and abs(other_r) < 1 and long_r <= 100.2:
        master_log.append("=> Candidate for Slight/None; checking proof vs. disproof phrases...")

        proof_score, disproof_score = tally_slight_none_proof_disproof(row)
        master_log.append(f"Proof score={proof_score}, Disproof score={disproof_score}")

        if disproof_score > proof_score:
            master_log.append("Disproof > proof => skipping Slight/None => next branch...")
        else:
            master_log.append("Slight/None => returning due to tie or higher proof.")
            return "Slight/None", f"Slight/None (proof={proof_score}, disproof={disproof_score})"

    master_log.append("=> FAILED => next branch...")

    # Branch 2: "Moderate"
    master_log.append(
        f"Branch 2 check: short<={10}?, other<={10}? => short={short_r}, other={other_r}"
    )
    if short_r <= 10 and other_r <= 10:
        master_log.append("=> PASSED => 'Moderate'")
        return "Moderate", "Moderate"

    master_log.append("=> FAILED => next branch...")

    # Branch 3: "Persistent Systematic"
    master_log.append(
        f"Branch 3 check: short<={50}? or other<={50}? => short={short_r}, other={other_r}"
    )
    if short_r <= 50 or other_r <= 50:
        master_log.append("=> PASSED => 'Persistent Systematic'")
        return "Persistent Systematic", "Persistent Systematic"

    master_log.append("=> FAILED => next branch...")

    # Branch 4: "Heavy Amplification"
    master_log.append(
        f"Branch 4 => short/other > 50% => => 'Heavy Amplification'"
    )
    return "Heavy Amplification", "Heavy Amplification"

def alternative_evaluation(row, scores, master_log):
    # If you have alternative logic, put extended logging here.
    master_log.append("===== BEGIN ALTERNATIVE EVALUATION =====")
    # Return (None, master_log) or a final classification with reason
    return (None, master_log)

def classify_fund(row):
    audit_log = []

    # 1) Fund family override
    if pd.notna(row['fund_family']) and ("return stacked etfs" in row['fund_family'].lower()):
        audit_log.append(
            "FUND FAMILY OVERRIDE => 'Return Stacked ETFs' => Persistent Systematic"
        )
        return "Persistent Systematic", audit_log

    # 2) Direct keyword mapping
    for category, keywords in direct_keyword_mappings.items():
        for field in ['ProductName', 'investment_strategy', 'FS_insight']:
            if pd.notna(row[field]):
                count = search_keywords(row[field], keywords)
                if count > 0:
                    audit_log.append(f"DIRECT KEYWORD => {category} from {field} match")
                    return category, audit_log

    # 3) Direct category mapping
    for category, mappings in direct_category_mappings.items():
        for db_field, values in mappings.items():
            vals_lower = [v.lower() for v in values]
            field_val = safe_lower(row.get(db_field, ''))
            if field_val in vals_lower:
                audit_log.append(f"DIRECT CATEGORY => {category} from {db_field}={field_val}")
                return category, audit_log

    # 4) Boolean exclusions
    possible_categories = ["Slight/None", "Moderate", "Persistent Systematic", "Heavy Amplification"]
    for cats_to_exclude, condition_fn in boolean_exclusions.items():
        if condition_fn(row):
            # i.e. if fund is leveraged => exclude "Slight/None" and "Moderate"
            excluded = [c for c in cats_to_exclude if c in possible_categories]
            for c in excluded:
                possible_categories.remove(c)
            audit_log.append(
                f"BOOLEAN EXCLUSION => Excluding {excluded} for leveraged/inverse/synthetic"
            )
            if len(possible_categories) == 1:
                # Only one left
                final_cat = possible_categories[0]
                audit_log.append(f"Only {final_cat} remains => final classification")
                return final_cat, audit_log

    # 5) Exposure-based classification
    classification, reason = classify_by_exposures_with_disproof(row, audit_log)
    audit_log.append(f"Exposure-based => {classification} => {reason}")
    return classification, audit_log


def sanitize_excel_text(value):
    """Convert or escape characters that can break Excel's XML."""
    if pd.isna(value):
        return value
    text = str(value)
    # Replace angle brackets so they don't break Excel's XML
    text = text.replace("<", "[lt]").replace(">", "[gt]")

    # If lines start with '=', '+', '-', or '@', prefix with "'" 
    # so Excel doesn't parse it as a formula
    if text.startswith(("=", "+", "-", "@")):
        text = "'" + text

    return text

# Main function to run the script
def main():
    engine = create_engine(connection_string)

    query = """
    SELECT 
        f.SymbolCUSIP, f.ProductName, f.fund_family, f.investment_strategy, f.FS_insight,
        f.index_fund, f.inverse_fund, f.leveraged_fund, f.synthetic_replication_fund,
        f.fund_of_funds, f.ycharts_url, f.currency_hedged_fund,
        f.cash_long, f.cash_net, f.cash_short, f.stock_long, f.stock_net, f.stock_short,
        f.bond_long, f.bond_net, f.bond_short, f.other_long, f.other_net, f.other_short,
        f.return_driver, f.YC_BM_Symbol,
        cwa.CWA_Broad_Category_Name,
        yc.Category_Name AS YC_Category_Name,
        ycg.Global_Category_Name,
        ycba.YC_Broad_Asset_Class_Name
    FROM Funds_to_Screen f
    LEFT JOIN CWA_Broad_Category_List cwa ON f.CWA_Broad_Category_ID = cwa.ID
    LEFT JOIN YC_Category_List yc ON f.YC_Category_ID = yc.ID
    LEFT JOIN YC_Global_Category_List ycg ON f.YC_Global_Category_ID = ycg.ID
    LEFT JOIN YC_Broad_Asset_Class_List ycba ON f.YC_Broad_Asset_Class_ID = ycba.ID
    """

    df = pd.read_sql(query, engine)

    results = []
    for idx, row in df.iterrows():
        classification, local_log = classify_fund(row)
        newrow = row.copy()
        newrow['Final_Classification'] = classification
        newrow['Audit_Log'] = "; ".join(local_log)
        results.append(newrow)

    outdf = pd.DataFrame(results)

    # Order columns
    columns_front = [
        'SymbolCUSIP','ProductName','fund_family','Final_Classification','ycharts_url'
    ]
    other_cols = [c for c in outdf.columns if c not in columns_front]
    outdf = outdf[columns_front + other_cols]

    output_path = r"C:\Users\JulianHeron\Software Projects\Risk_Overlays_V4.3_ExtLog_PRV_DPRV.xlsx"
    outdf.to_excel(output_path, index=False)
    print(f"Exported v4.3 Extended Logging => {output_path}")

if __name__ == "__main__":
    main()


Exported v4.3 Extended Logging => C:\Users\JulianHeron\Software Projects\Risk_Overlays_V4.3_ExtLog_PRV_DPRV.xlsx


In [ ]:
# Chat GPT unified code for proof/ disproof or disproof on all categories

import pandas as pd
from sqlalchemy import create_engine
import re
import os

###############################################################################
#  (A) ON/OFF FLAGS FOR PROOF/DISPROOF IN EACH BRANCH
###############################################################################
USE_PROOF_DISPROOF_SLIGHT = True
USE_PROOF_DISPROOF_MODERATE = True
USE_PROOF_DISPROOF_PERSISTENT = True
USE_PROOF_DISPROOF_HEAVY = True

# If you want the old “keyword disproof” approach from v4.2 for these categories,
# set the respective flags to False, or vice versa. 
# You can also combine them (some branches use proof vs disproof, others use older approach).

###############################################################################
#  (B) HELPER / WEAK HELPER CATEGORY MAPPINGS
###############################################################################
helper_category_mappings = {
    ("Persistent Systematic", "Heavy Amplification"): {
        "YC_Category": [
            "Trading--Inverse Commodities", "Trading--Inverse Debt",
            "Trading--Inverse Equity", "Trading--Miscellaneous"
        ],
        "CWA_Broad_Category": ["Trading/Tactical"],
        "YC_Global_Category": ["Trading Tools"]
    },
    ("Persistent Systematic", "Moderate"): {
        "YC_Global_Category": ["Multialternative", "Long/Short Equity"],
        "YC_Category": ["Equity Hedged"]
    }
}

weak_helper_category_mappings = {
    ("Slight/None", "Moderate"): {
        "YC_Global_Category": ["Flexible Allocation", "Alternative Miscellaneous"],
        "return_driver": ["Index Based", "Factor/Smart Beta"]
    },
    ("Slight/None", "Moderate", "Persistent Systematic"): {
        "YC_Category": ["Relative Value Arbitrage"]
    },
    ("Heavy Amplification", "Persistent Systematic", "Moderate", "Slight/None"): {
        "return_driver": ["Quant/Systematic"]
    },
    ("Persistent Systematic", "Moderate", "Slight/None"): {
        "return_driver": ["Active Discretionary", "Multi-Strategy"]
    }
}

###############################################################################
#  (C) PROOF / DISPROOF PHRASES FOR EACH BRANCH
###############################################################################
# Slight/None proof & disproof from prior conversation:
slight_proof_phrases = [
    "used for minor duration or risk tweaks",
    "occasional use for limited exposure adjustments",
    # ... etc ...
]
slight_disproof_phrases = [
    "systematically uses derivatives",
    "long-short",
    # ... etc ...
]

# Similarly, if you want “proof vs. disproof” for Moderate, 
# you can define them here:
moderate_proof_phrases = [
    "protected puts",
    # (examples - expand as needed)
]
moderate_disproof_phrases = [
    "aggressive shorting",
    # (examples - expand as needed)
]

# Same for Persistent Systematic:
persistent_proof_phrases = [
    "uses derivatives occasionally for alpha",
]
persistent_disproof_phrases = [
    "massive leverage",
]

# Same for Heavy Amplification:
heavy_proof_phrases = [
    "only doubles exposure occasionally",
]
heavy_disproof_phrases = [
    "strictly long-only",
    "no leverage ever used"
]

###############################################################################
#  (D) HELPER FUNCTIONS
###############################################################################

def sanitize_excel_text(value):
    """Avoid Excel formula/angle bracket issues."""
    if pd.isna(value):
        return value
    text = str(value)
    # Replace < and >
    text = text.replace("<", "[lt]").replace(">", "[gt]")
    # If starts with certain chars, prefix with '
    if text.startswith(("=", "+", "-", "@")):
        text = "'" + text
    return text

def safe_lower(value):
    """Safe lower for text columns."""
    return value.lower() if isinstance(value, str) else ""

def search_keywords(text, keywords):
    """v4.2 approach: does the text contain any of these keywords?"""
    if pd.isna(text):
        return 0
    text = str(text).lower()
    return sum(1 for kw in keywords if re.search(r'\b' + re.escape(kw.lower()) + r'\b', text))

###############################################################################
#  (E) TALLY PROOF / DISPROOF FOR A GIVEN BRANCH
###############################################################################
def tally_proof_disproof(row, proof_list, disproof_list):
    """Returns (matched_proof, matched_disproof, proof_score, disproof_score)."""
    matched_proof = []
    matched_disproof = []
    for field in ["ProductName", "investment_strategy", "FS_insight"]:
        if pd.notna(row[field]):
            text = row[field].lower()
            for phrase in proof_list:
                if phrase in text:
                    matched_proof.append(phrase)
            for phrase in disproof_list:
                if phrase in text:
                    matched_disproof.append(phrase)
    # De-dup
    matched_proof = list(set(matched_proof))
    matched_disproof = list(set(matched_disproof))

    return matched_proof, matched_disproof, len(matched_proof), len(matched_disproof)

###############################################################################
#  (F) DISPROOF KEYWORDS FROM v4.2 FOR THE "OLD" APPROACH
###############################################################################
disproof_keywords_v4_2 = {
    "Slight/None": [
        "derivatives", "swaps", "futures", "short", "hedge", "long-short",
        "inverse", "leveraged", "tail risk", "tail-risk"
    ],
    "Moderate": [
        "systematic", "trend-following", "2x", "3x", "market neutral",
        "quantitative hedging", "managed futures", "managed futures strategy",
        "trend strategy", "yieldmax", "tail risk", "tail-risk"
    ],
    "Persistent Systematic": ["2x", "3x", "-2x", "-3x", "uncapped accelerator"],
    "Heavy Amplification": ["long-only", "no derivatives", "no short"]
}

def has_disproof_v4_2(category, row, audit_log):
    """Same as the older v4.2 approach: if any disproof keyword appears, return True."""
    if category not in disproof_keywords_v4_2:
        return False
    found_matches = []
    for field in ["ProductName", "investment_strategy", "FS_insight"]:
        text = str(row.get(field, "")).lower()
        for kw in disproof_keywords_v4_2[category]:
            if re.search(r'\b' + re.escape(kw.lower()) + r'\b', text):
                found_matches.append(kw)
    if found_matches:
        unique_hits = list(set(found_matches))
        audit_log.append(f"Disproof for {category} => {unique_hits}")
        return True
    return False

###############################################################################
#  (G) EXPOSURE-BASED CLASSIFICATION – WITH FLAGS
###############################################################################
def classify_by_exposures_with_proof_disproof(row, audit_log):
    # Convert exposures to numeric
    exposure_cols = [
        'cash_long', 'cash_net', 'cash_short',
        'stock_long', 'stock_net', 'stock_short',
        'bond_long', 'bond_net', 'bond_short',
        'other_long', 'other_net', 'other_short'
    ]
    for col in exposure_cols:
        val = pd.to_numeric(row[col], errors='coerce')
        row[col] = 0 if pd.isna(val) else val

    # Sum
    long_total = row['cash_long'] + row['stock_long'] + row['bond_long'] + row['other_long']
    short_total = row['cash_short'] + row['stock_short'] + row['bond_short'] + row['other_short']
    other_total = row['other_long'] + row['other_short']

    # Multiply by 100
    long_r = round(long_total * 100, 4)
    short_r = round(short_total * 100, 4)
    other_r = round(other_total * 100, 4)

    audit_log.append(f"[Exposure] long={long_r}%, short={short_r}%, other={other_r}%")

    # BRANCH 1: SLIGHT/NONE
    audit_log.append("Branch 1: Slight/None check => short<1%, other<1%, long<=100.2%")
    if short_r < 1 and other_r < 1 and long_r <= 100.2:
        # If USE_PROOF_DISPROOF_SLIGHT => do the proof/disproof approach
        if USE_PROOF_DISPROOF_SLIGHT:
            matched_proof, matched_disproof, p_score, d_score = tally_proof_disproof(
                row,
                slight_proof_phrases,
                slight_disproof_phrases
            )
            audit_log.append(f"SLIGHT/NONE proof hits => {matched_proof}")
            audit_log.append(f"SLIGHT/NONE disproof hits => {matched_disproof}")
            if d_score > p_score:
                audit_log.append("Slight/None disproof > proof => next branch.")
            else:
                audit_log.append("Slight/None => pass => returning.")
                return "Slight/None", "Proof/Disproof"
        else:
            # Old v4.2 disproof approach
            if has_disproof_v4_2("Slight/None", row, audit_log):
                audit_log.append("Slight/None disproof triggered => next branch.")
            else:
                audit_log.append("Slight/None => returning (no disproof).")
                return "Slight/None", "No disproof"
    audit_log.append("... did not pass => next branch...")

    # BRANCH 2: MODERATE
    audit_log.append("Branch 2: Moderate => short<=10%, other<=10%")
    if short_r <= 10 and other_r <= 10:
        if USE_PROOF_DISPROOF_MODERATE:
            matched_proof, matched_disproof, p_score, d_score = tally_proof_disproof(
                row,
                moderate_proof_phrases,
                moderate_disproof_phrases
            )
            audit_log.append(f"MODERATE proof hits => {matched_proof}")
            audit_log.append(f"MODERATE disproof hits => {matched_disproof}")
            if d_score > p_score:
                audit_log.append("Moderate disproof>proof => next branch.")
            else:
                audit_log.append("Moderate => returning.")
                return "Moderate", "Proof/Disproof"
        else:
            if has_disproof_v4_2("Moderate", row, audit_log):
                audit_log.append("Moderate disproof => next branch.")
            else:
                audit_log.append("Moderate => returning.")
                return "Moderate", "No disproof"
    audit_log.append("... did not pass => next branch...")

    # BRANCH 3: PERSISTENT SYSTEMATIC
    audit_log.append("Branch 3: short<=50% or other<=50% => Persistent Systematic")
    if short_r <= 50 or other_r <= 50:
        if USE_PROOF_DISPROOF_PERSISTENT:
            matched_proof, matched_disproof, p_score, d_score = tally_proof_disproof(
                row,
                persistent_proof_phrases,
                persistent_disproof_phrases
            )
            audit_log.append(f"PERSISTENT proof hits => {matched_proof}")
            audit_log.append(f"PERSISTENT disproof hits => {matched_disproof}")
            if d_score > p_score:
                audit_log.append("Persistent disproof>proof => next branch.")
            else:
                audit_log.append("Persistent => returning.")
                return "Persistent Systematic", "Proof/Disproof"
        else:
            if has_disproof_v4_2("Persistent Systematic", row, audit_log):
                audit_log.append("Persistent disproof => next branch.")
            else:
                audit_log.append("Persistent => returning.")
                return "Persistent Systematic", "No disproof"
    audit_log.append("... did not pass => next branch...")

    # BRANCH 4: HEAVY AMPLIFICATION
    audit_log.append("Branch 4 => 'Heavy Amplification'")
    if USE_PROOF_DISPROOF_HEAVY:
        matched_proof, matched_disproof, p_score, d_score = tally_proof_disproof(
            row,
            heavy_proof_phrases,
            heavy_disproof_phrases
        )
        audit_log.append(f"HEAVY proof hits => {matched_proof}")
        audit_log.append(f"HEAVY disproof hits => {matched_disproof}")
        if d_score > p_score:
            # Typically there's nowhere else to go, so we still finalize Heavy
            audit_log.append("Heavy disproof>proof => but no next branch => return Heavy anyway.")
        else:
            audit_log.append("Heavy => returning.")
        return "Heavy Amplification", "Proof/Disproof"
    else:
        if has_disproof_v4_2("Heavy Amplification", row, audit_log):
            audit_log.append("Heavy disproof => no next branch => still Heavy by default.")
        return "Heavy Amplification", "No disproof"

###############################################################################
#  (H) ALTERNATIVE EVALUATION & TIEBREAKS – v4.2 RESTORED
###############################################################################
def alternative_evaluation(row, scores, audit_log):
    # This is the old “score each category” approach from v4.2 if needed
    audit_log.append("Entering Alternative Evaluation from v4.2 ...")

    # For example
    possible_categories = ["Slight/None", "Moderate", "Persistent Systematic", "Heavy Amplification"]
    # You might do more scoring logic, referencing row's fields
    # ...
    # Return final or None
    return "Slight/None", audit_log  # Example fallback

###############################################################################
#  (I) MASTER CLASSIFICATION FUNCTION – REINTRODUCING HELPER/WEAK MAPPINGS, TIEBREAKS, ETC.
###############################################################################
def classify_fund(row):
    audit_log = []
    possible_categories = ["Slight/None", "Moderate", "Persistent Systematic", "Heavy Amplification"]
    scores = {cat: 0 for cat in possible_categories}

    # (I.1) Step 1: Override for Fund Family "Return Stacked ETFs"
    if pd.notna(row['fund_family']) and "return stacked etfs" in row['fund_family'].lower():
        audit_log.append("Fund Family => 'Return Stacked ETFs' => Persistent Systematic override")
        return "Persistent Systematic", audit_log

    # (I.2) Step 2: Direct keyword mapping
    direct_keyword_mappings = {
        "Persistent Systematic": ["Market Neutral", "managed futures", "Premia", "Return Stacked ETFs"],
        "Heavy Amplification": ["2x", "3x", "Uncapped Accelerator", "-2x", "-3x", "YieldMax"]
    }
    for category, keywords in direct_keyword_mappings.items():
        for field in ['ProductName', 'investment_strategy', 'FS_insight']:
            textval = safe_lower(row[field])
            for kw in keywords:
                if kw.lower() in textval:
                    audit_log.append(f"Direct Keyword => {category} from {field} match: {kw}")
                    return category, audit_log

    # (I.3) Step 3: Direct category mapping
    direct_category_mappings = {
        "Persistent Systematic": {
            "YC_Category": ["Defined Outcome"],
            "CWA_Broad_Category": ["Defined Outcome"],
            "YC_Global_Category": ["market neutral"]
        },
        "Heavy Amplification": {
            "YC_Category": [
                "Trading--Leveraged Equity", "Trading--Leveraged Debt", "Trading--Leveraged Commodities"
            ],
            "CWA_Broad_Category": ["Single Stock"]
        }
    }
    for category, mappings in direct_category_mappings.items():
        for db_field, vals in mappings.items():
            fieldval = safe_lower(row.get(db_field, ""))
            vals_lower = [v.lower() for v in vals]
            if fieldval in vals_lower:
                audit_log.append(f"Direct Category => {category} in {db_field}: {fieldval}")
                return category, audit_log

    # (I.4) Step 4: Boolean exclusions
    boolean_exclusions = {
        ("Slight/None", "Moderate"): lambda r: r['leveraged_fund'] in [True, 1]
            or r['synthetic_replication_fund'] in [True, 1]
            or r['inverse_fund'] in [True, 1]
    }
    for cats_tuple, cond_fn in boolean_exclusions.items():
        if cond_fn(row):
            excluded = [c for c in possible_categories if c in cats_tuple]
            for c in excluded:
                possible_categories.remove(c)
            audit_log.append(f"Boolean Exclusions => removed {excluded} (inverse/leveraged).")
            if len(possible_categories) == 1:
                finalcat = possible_categories[0]
                audit_log.append(f"Only {finalcat} remains => final classification")
                return finalcat, audit_log

    # (I.5) Step 5: Exposure-based classification with proof/disproof
    exposure_result = classify_by_exposures_with_proof_disproof(row, audit_log)
    if isinstance(exposure_result, tuple) and len(exposure_result) == 2:
        classification, reason = exposure_result
        audit_log.append(f"Exposure-based => {classification}, reason={reason}")
        return classification, audit_log

    # (I.6) If None or ambiguous => alternative evaluation
    # If classify_by_exposures_with_proof_disproof returned something else
    if exposure_result[0] is None:
        # old logic we had in v4.2
        scores_update = exposure_result[1]
        for cat in scores_update:
            scores[cat] += scores_update[cat]
        classification, alt_log = alternative_evaluation(row, scores, audit_log)
        for line in alt_log:
            audit_log.append(line)
        return classification, audit_log
    elif isinstance(exposure_result, tuple):
        # Some leftover path
        possible_categories, scores_dict, exposure_log = exposure_result
        for line in exposure_log:
            audit_log.append(line)
        for cat, val in scores_dict.items():
            scores[cat] += val

    # (I.7) Final scoring if no immediate classification
    max_score = max(scores.values())
    top_cats = [c for c in possible_categories if scores[c] == max_score]
    if len(top_cats) == 1:
        final = top_cats[0]
        audit_log.append(f"Final => {final} with score={max_score}")
        return final, audit_log
    else:
        # Tiebreaker approach from v4.2
        tiebreak_scores = {c: 0 for c in top_cats}
        # Possibly check text fields for category keywords, etc.
        audit_log.append(f"Tiebreaker among {top_cats} => {tiebreak_scores}")
        # For simplicity, pick the most conservative 
        # (Slight/None < Moderate < Persistent < Heavy)
        order = ["Slight/None", "Moderate", "Persistent Systematic", "Heavy Amplification"]
        final = min(top_cats, key=lambda x: order.index(x))
        audit_log.append(f"Chosen final => {final}")
        return final, audit_log

###############################################################################
#  (J) MAIN
###############################################################################
def main():
    connection_string = (
        "mssql+pyodbc://JULIANS_LAPTOP\\SQLEXPRESS/"
        "CWA_Fund_Database?driver=ODBC+Driver+18+for+SQL+Server"
        "&trusted_connection=yes&TrustServerCertificate=yes"
    )
    engine = create_engine(connection_string)

    query = """
    SELECT 
        f.SymbolCUSIP, f.ProductName, f.fund_family, f.investment_strategy, f.FS_insight,
        f.index_fund, f.inverse_fund, f.leveraged_fund, f.synthetic_replication_fund,
        f.fund_of_funds, f.ycharts_url, f.currency_hedged_fund,
        f.cash_long, f.cash_net, f.cash_short, f.stock_long, f.stock_net, f.stock_short,
        f.bond_long, f.bond_net, f.bond_short, f.other_long, f.other_net, f.other_short,
        f.return_driver, f.YC_BM_Symbol,
        cwa.CWA_Broad_Category_Name as CWA_Broad_Category,
        yc.Category_Name AS YC_Category,
        ycg.Global_Category_Name as YC_Global_Category,
        ycba.YC_Broad_Asset_Class_Name
    FROM Funds_to_Screen f
    LEFT JOIN CWA_Broad_Category_List cwa ON f.CWA_Broad_Category_ID = cwa.ID
    LEFT JOIN YC_Category_List yc ON f.YC_Category_ID = yc.ID
    LEFT JOIN YC_Global_Category_List ycg ON f.YC_Global_Category_ID = ycg.ID
    LEFT JOIN YC_Broad_Asset_Class_List ycba ON f.YC_Broad_Asset_Class_ID = ycba.ID
    """

    df = pd.read_sql(query, engine)

    results = []
    for idx, row in df.iterrows():
        classification, audit_log = classify_fund(row)
        newrow = row.copy()
        newrow['Final_Classification'] = classification
        newrow['Audit_Log'] = "; ".join(audit_log)
        results.append(newrow)

    outdf = pd.DataFrame(results)

    # sanitize
    outdf["Audit_Log"] = outdf["Audit_Log"].apply(sanitize_excel_text)

    # reorder columns
    front_cols = ["SymbolCUSIP", "ProductName", "fund_family", "Final_Classification", "ycharts_url"]
    other_cols = [c for c in outdf.columns if c not in front_cols]
    outdf = outdf[front_cols + other_cols]

    out_path = r"C:\Users\JulianHeron\Software Projects\Risk_Overlays_V4.5_Flags.xlsx"
    outdf.to_excel(out_path, index=False)
    print(f"Exported => {out_path}")


if __name__ == "__main__":
    main()
